In [2]:
import carla
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
from PIL import Image
import time
import os
import random
import cv2
import gc
import imageio.v2 as imageio
import signal
import math

# Environment setup
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24,36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36,48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48,64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64,64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape, device=next(self.parameters()).device)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


def preprocess_image_stack(frame_stack, device):
    """
    Preprocess a stack of 3 RGB frames for the model.
    Matches the exact preprocessing used during training.
    """
    if len(frame_stack) != 3:
        return None
    
    processed_frames = []
    
    # Process each frame individually first (as they would be during training)
    for frame in frame_stack:
        # Resize if needed (should already be 120x160)
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        
        # Convert to float32 and normalize to [0, 1]
        frame = frame.astype(np.float32) / 255.0
        
        # Normalize to [-1, 1] range (standard ImageNet-style normalization)
        frame = (frame - 0.5) / 0.5
        
        processed_frames.append(frame)
    
    # Stack the processed frames along channel dimension
    # This creates a (120, 160, 9) array
    stacked = np.concatenate(processed_frames, axis=2)
    
    # Convert to CHW format for PyTorch (9, 120, 160)
    stacked = np.transpose(stacked, (2, 0, 1))
    
    # Convert to tensor and add batch dimension
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)  # (1, 9, 120, 160)
    
    return tensor.to(device)


class ImprovedPIDController:
    """Enhanced PID controller with better tuning and safeguards"""
    
    def __init__(self):
        # PID gains - tuned for better lane keeping
        self.Kp = 0.8  # Increased proportional gain
        self.Ki = 0.02  # Reduced integral gain to prevent windup
        self.Kd = 0.15  # Increased derivative gain for stability
        
        # State variables
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        
        # Anti-windup limits
        self.integral_limit = 0.5
        self.output_limit = 0.6  # Reduced steering limit
        
    def update(self, error, current_time):
        """Update PID controller with current error"""
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05  # Default dt
        else:
            dt = max(current_time - self.prev_time, 0.001)  # Prevent division by zero
            self.prev_time = current_time
        
        # Proportional term
        P = self.Kp * error
        
        # Integral term with anti-windup
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        
        # Derivative term with smoothing
        if dt > 0:
            derivative = (error - self.prev_error) / dt
            D = self.Kd * derivative
        else:
            D = 0.0
        
        self.prev_error = error
        
        # Combine and limit output
        output = P + I + D
        return np.clip(output, -self.output_limit, self.output_limit)
    
    def reset(self):
        """Reset PID controller state"""
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


class ClosedLoopEvaluator:
    def __init__(self, model_path, host="localhost", port=2000):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        # Enhanced control system
        self.pid_controller = ImprovedPIDController()
        self.throttle_smooth = 0.3
        self.steer_smooth = 0.0
        
        # Control smoothing parameters
        self.steer_alpha = 0.3  # Steering smoothing factor
        self.throttle_alpha = 0.2  # Throttle smoothing factor
        
        # Lane keeping parameters
        self.target_speed_kmh = 15  # Target speed in km/h
        self.max_lane_offset = 2.0  # Maximum allowed lane offset before correction
        
        # Statistics
        self.collision_count = 0
        self.collision_times = []
        self.deviations = []
        self.controls_history = []
        self.trajectory = []
        
        # Setup signal handler for graceful shutdown
        signal.signal(signal.SIGINT, self.signal_handler)

    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        """Calculate lateral error from lane center using CARLA waypoints"""
        try:
            # Get the nearest waypoint
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            
            if waypoint is None:
                return 0.0
            
            # Calculate lateral offset
            lane_center = waypoint.transform.location
            
            # Vector from lane center to vehicle
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            
            # Get lane direction (perpendicular to road direction)
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            
            # Calculate cross-track error (positive = right of center, negative = left)
            cross_track_error = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            
            # Calculate heading error (difference between vehicle and lane heading)
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            
            # Normalize heading error to [-pi, pi]
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            
            # Combine cross-track and heading errors
            # Weight heading error more heavily for better stability
            total_error = cross_track_error + 0.5 * heading_error
            
            return total_error
            
        except Exception as e:
            print(f"Error calculating lane error: {e}")
            return 0.0

    def adaptive_speed_control(self, current_speed_mps, lane_offset, steering_magnitude):
        """Adaptive speed control based on lane position and steering input"""
        target_speed_mps = self.target_speed_kmh / 3.6
        
        # Reduce speed when far from lane center or steering heavily
        offset_factor = max(0.7, 1.0 - abs(lane_offset) / 3.0)  # Slow down if offset > 1.5m
        steering_factor = max(0.8, 1.0 - abs(steering_magnitude) / 0.5)  # Slow down during sharp turns
        
        adjusted_target = target_speed_mps * offset_factor * steering_factor
        
        # Simple speed controller
        speed_error = adjusted_target - current_speed_mps
        
        if speed_error > 0:
            throttle = min(0.7, 0.4 + speed_error * 0.3)
        else:
            throttle = max(0.2, 0.4 + speed_error * 0.2)
            
        return throttle

    def signal_handler(self, sig, frame):
        print("\nGraceful shutdown initiated...")
        self.running = False
        
    def connect_carla(self):
        """Connect to CARLA simulator"""
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        # self.world = self.client.load_world("Town05")
        self.map = self.world.get_map()
        
        # Set synchronous mode for stable evaluation
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05  # 20 FPS
        self.world.apply_settings(settings)
        
    def spawn_vehicle(self):
        """Spawn vehicle at random location"""
        blueprint_library = self.world.get_blueprint_library()
        
        # Use a standard vehicle
        vehicle_bp = blueprint_library.filter("vehicle.mercedes.coupe_2020")[0]
        if not vehicle_bp:
            vehicle_bp = blueprint_library.filter("vehicle.*")[0]
        
        # Get spawn point
        spawn_points = self.map.get_spawn_points()
        spawn_point = random.choice(spawn_points)
        spawn_point.location.z += 0.5
        
        self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        print(f"Spawned vehicle at: ({spawn_point.location.x:.2f}, {spawn_point.location.y:.2f})")
        
        return spawn_point.location
        
    def setup_sensors(self):
        """Setup camera and collision sensors"""
        blueprint_library = self.world.get_blueprint_library()
        
        # Camera sensor (set exactly like CNN script: 800x600, fov 120, camera at x=-4 pitch=-5)
        camera_bp = blueprint_library.find("sensor.camera.rgb")
        camera_bp.set_attribute("image_size_x", "800")
        camera_bp.set_attribute("image_size_y", "600")
        camera_bp.set_attribute("fov", "120")
        
        camera_transform = carla.Transform(carla.Location(x=-4, y=0, z=2), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(camera_bp, camera_transform, attach_to=self.vehicle)
        
        # Collision sensor
        collision_bp = blueprint_library.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(
            collision_bp, carla.Transform(), attach_to=self.vehicle
        )
        
        # Setup callbacks
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        
        self.camera.listen(self.camera_callback)
        self.collision_sensor.listen(self.collision_callback)
        
    def camera_callback(self, image):
        """Process camera images (match CNN script format: RGB)"""
        array = np.frombuffer(image.raw_data, dtype=np.uint8)
        array = array.reshape((image.height, image.width, 4))[:, :, :3]  # BGRA -> BGR
        array = array[:, :, ::-1]  # BGR -> RGB
        self.latest_image = array
        self.new_image_available = True
        
    def collision_callback(self, event):
        """Handle collision events"""
        self.collision_count += 1
        collision_time = time.time() - self.start_time
        self.collision_times.append(collision_time)
        print(f"\n*** Collision #{self.collision_count} at {collision_time:.2f}s ***")
        print(f"    Impact with: {event.other_actor.type_id}")
        self.collision_occurred = True

        if self.collision_occurred:
            # Reset speed to 0 to stop the vehicle
            self.vehicle.set_velocity(carla.Vector3D(0, 0, 0))
            
            # Reset PID controller
            self.pid_controller.reset()
            
            # Neutralize steering for recovery
            control = carla.VehicleControl()
            control.steer = 0.0
            self.vehicle.apply_control(control)
            
            # Wait before continuing
            time.sleep(1)
            self.collision_occurred = False

    def load_model(self):
        """Load the trained model"""
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        # Initialize model
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True).to(self.device)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device) 
        
        # Load weights
        if os.path.exists(self.model_path):
            checkpoint = torch.load(self.model_path, map_location=self.device, weights_only=False)
            self.model.load_state_dict(checkpoint)
            print(f"Model loaded successfully from {self.model_path}")
        else:
            raise FileNotFoundError(f"Model checkpoint not found at {self.model_path}")
            
        self.model.eval()
        
    def run_evaluation(self):
        """Main evaluation loop with enhanced lane keeping"""
        print("\n" + "="*60)
        print("ENHANCED CLOSED-LOOP EVALUATION STARTED")
        print("="*60)
        print("Press Ctrl+C to stop evaluation gracefully\n")
        
        # Initialize
        self.connect_carla()
        start_location = self.spawn_vehicle()
        self.setup_sensors()
        self.load_model()
        
        # Setup spectator
        spectator = self.world.get_spectator()
        
        # Initialize control
        control = carla.VehicleControl()
        control.manual_gear_shift = False
        
        # Wait for initial frames
        print("Warming up sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)
        
        # Initialize variables
        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log_time = time.time()
        
        # Open log file
        log_filename = f"enhanced_evaluation_log_{int(self.start_time)}.csv"
        log_file = open(log_filename, "w")
        log_file.write("timestamp,frame,x,y,z,yaw,speed_mps,speed_kmh,model_steer,final_steer,throttle,lane_error,distance_traveled\n")
        
        # Image directory for video frames (using CNN-style naming)
        image_dir = f"enhanced_eval_images_{int(self.start_time)}"
        os.makedirs(image_dir, exist_ok=True)
        
        print("Starting enhanced autonomous driving...\n")
        
        try:
            while self.running and not self.collision_occurred:
                # Update frame stack
                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False
                
                # Process when we have 3 frames
                if len(frame_stack) == 3:
                    # Get vehicle state
                    transform = self.vehicle.get_transform()
                    location = transform.location
                    rotation = transform.rotation
                    
                    # Calculate speed
                    velocity = self.vehicle.get_velocity()
                    speed_mps = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
                    speed_kmh = speed_mps * 3.6
                    
                    # Calculate lane error
                    lane_error = self.calculate_lane_error(location, rotation)
                    
                    # Preprocess frames for model
                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    
                    if input_tensor is not None:
                        # Prepare speed input
                        speed_normalized = min(speed_mps / 30.0, 1.0)
                        speed_tensor = torch.tensor([[speed_normalized]], dtype=torch.float32, device=self.device)
                        
                        # Model inference
                        with torch.no_grad():
                            output = self.model(input_tensor, aux=speed_tensor)
                        
                        # Extract model outputs
                        model_steer, model_throttle, model_brake = output[0].cpu().numpy()
                        
                        # Enhanced steering control combining model output and lane keeping
                        current_time = time.time()
                        
                        # Use PID controller to correct lane error
                        pid_correction = self.pid_controller.update(lane_error, current_time)
                        
                        # Combine model steering with PID correction
                        # Give more weight to PID when lane error is large
                        error_weight = min(abs(lane_error) / 1.0, 0.8)  # Max 80% PID weight
                        combined_steer = (1 - error_weight) * model_steer + error_weight * pid_correction
                        
                        # Apply steering smoothing
                        final_steer = (self.steer_alpha * combined_steer + 
                                     (1 - self.steer_alpha) * self.steer_smooth)
                        self.steer_smooth = final_steer
                        
                        # Clamp steering to safe limits
                        final_steer = np.clip(final_steer, -0.6, 0.6)
                        
                        # Adaptive throttle control
                        adaptive_throttle = self.adaptive_speed_control(speed_mps, lane_error, abs(final_steer))
                        
                        # Apply throttle smoothing
                        final_throttle = (self.throttle_alpha * adaptive_throttle + 
                                        (1 - self.throttle_alpha) * self.throttle_smooth)
                        self.throttle_smooth = final_throttle
                        
                        # Ensure minimum throttle to prevent stalling
                        final_throttle = max(final_throttle, 0.2)
                        final_throttle = min(final_throttle, 0.8)
                        
                        # Apply controls
                        control.steer = float(final_steer)
                        control.throttle = float(final_throttle)
                        control.brake = 0.0
                        control.hand_brake = False
                        self.vehicle.apply_control(control)
                        
                        # Save frame exactly like CNN script
                        try:
                            frame_filename = os.path.join(image_dir, f"{frame_count:06d}.png")
                            cv2.imwrite(frame_filename, cv2.cvtColor(self.latest_image, cv2.COLOR_RGB2BGR))
                        except Exception as e:
                            print(f"Warning: failed to save frame {frame_count}: {e}")
                    
                        # Calculate metrics
                        distance_traveled = location.distance(start_location)
                        
                        # Store data
                        self.deviations.append(abs(lane_error))
                        self.controls_history.append({
                            'frame': frame_count,
                            'model_steer': model_steer,
                            'final_steer': final_steer,
                            'throttle': final_throttle,
                            'speed': speed_mps,
                            'lane_error': lane_error
                        })
                        self.trajectory.append({
                            'x': location.x,
                            'y': location.y,
                            'z': location.z,
                            'yaw': rotation.yaw
                        })
                        
                        # Log to file
                        timestamp = time.time() - self.start_time
                        log_file.write(
                            f"{timestamp:.3f},{frame_count},{location.x:.2f},{location.y:.2f},{location.z:.2f},"
                            f"{rotation.yaw:.2f},{speed_mps:.3f},{speed_kmh:.1f},{model_steer:.4f},"
                            f"{final_steer:.4f},{final_throttle:.4f},{lane_error:.4f},{distance_traveled:.2f}\n"
                        )
                        log_file.flush()
                        
                        # Update spectator view (use CNN-style spectator follow: -6 and z=3, pitch=-15)
                        transform = self.vehicle.get_transform()
                        loc = transform.location
                        rot = transform.rotation
                        spectator.set_transform(carla.Transform(loc + transform.get_forward_vector()*-6 + carla.Location(z=3),
                                                                carla.Rotation(pitch=-15, yaw=rot.yaw)))
                        
                        # Print status every second
                        current_time = time.time()
                        if current_time - last_log_time >= 1.0:
                            print(f"[{timestamp:6.1f}s] Frame {frame_count:5d} | "
                                  f"Speed: {speed_kmh:5.1f} km/h | "
                                  f"Distance: {distance_traveled:6.1f}m | "
                                  f"Lane error: {lane_error:+.3f}m | "
                                  f"Steering (M/F): {model_steer:+.3f}/{final_steer:+.3f}")
                            last_log_time = current_time
                    
                    frame_count += 1
                
                # Tick the simulation
                self.world.tick()

        except Exception as e:
            print(f"\nError during evaluation: {e}")
            
        finally:
            # Close log file
            log_file.close()
            
            # Create video from saved frames (exact CNN-style logic)
            try:
                images = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
                if images:
                    out_video = f"enhanced_evaluation_video_{int(self.start_time)}.mp4"
                    with imageio.get_writer(out_video, fps=30) as writer:
                        for img in images:
                            writer.append_data(imageio.imread(img))
                    print(f"Enhanced evaluation video saved as {out_video}")
            except Exception as e:
                print(f"Warning: failed to create video: {e}")
            finally:
                try:
                    gc.collect()
                except:
                    pass
            
            # Print final statistics
            self.print_statistics(frame_count)
            
            # Cleanup
            self.cleanup()

    def print_statistics(self, frame_count):
        """Print enhanced evaluation statistics"""
        elapsed_time = time.time() - self.start_time
        
        print("\n" + "="*60)
        print("ENHANCED EVALUATION STATISTICS")
        print("="*60)
        
        print(f"Total frames processed: {frame_count}")
        print(f"Total time: {elapsed_time:.2f} seconds")
        print(f"Average FPS: {frame_count/elapsed_time:.2f}")
        print(f"Total collisions: {self.collision_count}")
        
        if len(self.deviations) > 0:
            print(f"\nLane keeping performance:")
            print(f"  Average offset: {np.mean(self.deviations):.3f} meters")
            print(f"  Std deviation: {np.std(self.deviations):.3f} meters")
            print(f"  Max offset: {np.max(self.deviations):.3f} meters")
            print(f"  95th percentile: {np.percentile(self.deviations, 95):.3f} meters")
            
            # Calculate percentage of time within lane bounds
            within_lane = np.sum(np.array(self.deviations) <= 1.5) / len(self.deviations) * 100
            print(f"  Time within 1.5m of lane center: {within_lane:.1f}%")
            
        if len(self.trajectory) > 0:
            first_pos = self.trajectory[0]
            last_pos = self.trajectory[-1]
            total_distance = np.sqrt(
                (last_pos['x'] - first_pos['x'])**2 + 
                (last_pos['y'] - first_pos['y'])**2
            )
            print(f"\nTotal distance traveled: {total_distance:.2f} meters")
            
        if len(self.controls_history) > 0:
            all_speeds = [c['speed'] for c in self.controls_history]
            all_lane_errors = [c['lane_error'] for c in self.controls_history]
            
            print(f"\nSpeed statistics:")
            print(f"  Average speed: {np.mean(all_speeds)*3.6:.2f} km/h")
            print(f"  Max speed: {np.max(all_speeds)*3.6:.2f} km/h")
            
            print(f"\nLane error statistics:")
            print(f"  Mean absolute error: {np.mean(np.abs(all_lane_errors)):.3f} meters")
            print(f"  RMS error: {np.sqrt(np.mean(np.array(all_lane_errors)**2)):.3f} meters")
                
        print("="*60)
        
    def cleanup(self):
        """Clean up CARLA actors and reset settings"""
        print("\nCleaning up...")
        
        # Reset to asynchronous mode
        if hasattr(self, 'world'):
            settings = self.world.get_settings()
            settings.synchronous_mode = False
            self.world.apply_settings(settings)
        
        # Destroy actors
        actors_to_destroy = []
        if hasattr(self, 'camera'):
            try:
                self.camera.stop()
            except:
                pass
            actors_to_destroy.append(self.camera)
        if hasattr(self, 'collision_sensor'):
            try:
                self.collision_sensor.stop()
            except:
                pass
            actors_to_destroy.append(self.collision_sensor)
        if hasattr(self, 'vehicle'):
            actors_to_destroy.append(self.vehicle)
            
        for actor in actors_to_destroy:
            if actor is not None:
                try:
                    actor.destroy()
                except:
                    pass
                
        print("Cleanup complete.")


def main():
    # Path to your trained model
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    
    # Create evaluator and run
    evaluator = ClosedLoopEvaluator(model_path=MODEL_PATH)
    evaluator.run_evaluation()


if __name__ == "__main__":
    main()



ENHANCED CLOSED-LOOP EVALUATION STARTED
Press Ctrl+C to stop evaluation gracefully

Spawned vehicle at: (102.98, -22.71)
Using device: cuda
Model loaded successfully from /home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth
Warming up sensors...
Starting enhanced autonomous driving...

[   1.0s] Frame    25 | Speed:   0.0 km/h | Distance:    1.1m | Lane error: +0.000m | Steering (M/F): +0.007/+0.007
[   2.0s] Frame    56 | Speed:  13.6 km/h | Distance:    3.6m | Lane error: -0.051m | Steering (M/F): +0.018/+0.020
[   3.1s] Frame    85 | Speed:  12.9 km/h | Distance:    8.8m | Lane error: -0.229m | Steering (M/F): +0.034/-0.013
[   4.1s] Frame   114 | Speed:  12.1 km/h | Distance:   13.6m | Lane error: -0.168m | Steering (M/F): +0.012/-0.016
[   5.1s] Frame   142 | Speed:  13.2 km/h | Distance:   18.7m | Lane error: -0.002m | Steering (M/F): +0.020/+0.025
[   6.1s] Frame   171 | Speed:  13.3 km/h | Distance:   24.0m | Lane error: -0.058m | Steering (M/F): +0.0

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Graceful shutdown initiated...
Enhanced evaluation video saved as enhanced_evaluation_video_1760304382.mp4

ENHANCED EVALUATION STATISTICS
Total frames processed: 411
Total time: 17.24 seconds
Average FPS: 23.84
Total collisions: 0

Lane keeping performance:
  Average offset: 0.102 meters
  Std deviation: 0.099 meters
  Max offset: 1.239 meters
  95th percentile: 0.248 meters
  Time within 1.5m of lane center: 100.0%

Total distance traveled: 66.87 meters

Speed statistics:
  Average speed: 11.74 km/h
  Max speed: 14.48 km/h

Lane error statistics:
  Mean absolute error: 0.102 meters
  RMS error: 0.142 meters

Cleaning up...
Cleanup complete.


# Lane

In [1]:
import carla
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import cv2
import gc
import imageio.v2 as imageio
import signal
import math
import traceback

# Environment setup
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


# ----------------- Utilities -----------------
def preprocess_image_stack(frame_stack, device):
    """Preprocess a stack of 3 RGB frames for the model (returns tensor on device)."""
    if len(frame_stack) != 3:
        return None
    processed_frames = []
    for frame in frame_stack:
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        frame = frame.astype(np.float32) / 255.0
        frame = (frame - 0.5) / 0.5
        processed_frames.append(frame)
    stacked = np.concatenate(processed_frames, axis=2)  # (120,160,9)
    stacked = np.transpose(stacked, (2, 0, 1))  # (9,120,160)
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)
    return tensor.to(device)


class ImprovedPIDController:
    """PID for lateral correction (keeps vehicle centered in lane)."""
    def __init__(self):
        self.Kp = 0.8
        self.Ki = 0.02
        self.Kd = 0.15
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        self.integral_limit = 0.5
        self.output_limit = 0.6

    def update(self, error, current_time):
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05
        else:
            dt = max(current_time - self.prev_time, 1e-3)
            self.prev_time = current_time
        P = self.Kp * error
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        D = self.Kd * derivative
        self.prev_error = error
        output = P + I + D
        return float(np.clip(output, -self.output_limit, self.output_limit))

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


# ----------------- Main evaluator -----------------
class ClosedLoopEvaluator:
    def __init__(self, model_path,
                 host="localhost", port=2000,
                 detect_yellow=False,
                 traffic_light_distance=25.0):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        # controllers & smoothing
        self.pid_controller = ImprovedPIDController()
        self.steer_smooth = 0.0
        self.throttle_smooth = 0.3
        self.steer_alpha = 0.3
        self.throttle_alpha = 0.2

        # speed / thresholds
        self.target_speed_kmh = 15.0
        self.max_lane_offset = 2.0

        # lane-change state
        self.lane_change_active = False
        self.lane_change_waypoints = []
        self.lane_change_start_time = None
        self.lane_change_duration = 2.5
        self.lane_change_finish_threshold = 0.25

        # traffic-light settings
        self.DETECT_YELLOW = detect_yellow
        self.TRAFFIC_LIGHT_DIST = traffic_light_distance  # meters: how far ahead to consider lights

        # stats & actors
        self.collision_count = 0
        self.deviations = []
        self.controls_history = []
        self.trajectory = []

        # placeholders
        self.client = None
        self.world = None
        self.map = None
        self.vehicle = None
        self.camera = None
        self.collision_sensor = None
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        self.start_time = None
        self.original_settings = None

        # graceful shutdown
        signal.signal(signal.SIGINT, self.signal_handler)

    def signal_handler(self, sig, frame):
        print("\nGraceful shutdown requested.")
        self.running = False

    def connect_carla(self):
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        self.original_settings = self.world.get_settings()
        self.map = self.world.get_map()
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)

    def spawn_vehicle(self):
        bp_lib = self.world.get_blueprint_library()
        candidates = bp_lib.filter("vehicle.*")
        if not candidates:
            raise RuntimeError("No vehicle blueprints found.")
        # pick a common vehicle for stability
        vehicle_bp = None
        for name in ("vehicle.tesla.model3", "vehicle.mercedes.coupe_2020", "vehicle.audi.tt"):
            items = bp_lib.filter(name)
            if items:
                vehicle_bp = items[0]
                break
        if vehicle_bp is None:
            vehicle_bp = random.choice(candidates)
        spawn_points = self.map.get_spawn_points()
        if not spawn_points:
            raise RuntimeError("No spawn points available.")
        sp = random.choice(spawn_points)
        sp.location.z += 0.5
        self.vehicle = self.world.spawn_actor(vehicle_bp, sp)
        print(f"Spawned vehicle at ({sp.location.x:.2f}, {sp.location.y:.2f})")
        return sp.location

    def setup_sensors(self):
        bp_lib = self.world.get_blueprint_library()
        cam_bp = bp_lib.find("sensor.camera.rgb")
        cam_bp.set_attribute("image_size_x", "800")
        cam_bp.set_attribute("image_size_y", "600")
        cam_bp.set_attribute("fov", "120")
        cam_transform = carla.Transform(carla.Location(x=-4, z=2, y=0), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)

        col_bp = bp_lib.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(col_bp, carla.Transform(), attach_to=self.vehicle)

        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False

        self.camera.listen(self.camera_callback)
        self.collision_sensor.listen(self.collision_callback)

    def camera_callback(self, image):
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]  # BGR->RGB
            small = cv2.resize(arr, (160, 120))
            self.latest_image = small
            self.new_image_available = True
        except Exception:
            pass

    def collision_callback(self, event):
        self.collision_count += 1
        t = time.time() - (self.start_time or time.time())
        print(f"\n*** Collision #{self.collision_count} at {t:.2f}s ***")
        try:
            print("    Impact with:", event.other_actor.type_id)
        except Exception:
            pass
        self.collision_occurred = True
        try:
            self.vehicle.set_velocity(carla.Vector3D(0, 0, 0))
        except Exception:
            pass
        self.pid_controller.reset()
        try:
            ctrl = carla.VehicleControl()
            ctrl.steer = 0.0
            ctrl.throttle = 0.0
            ctrl.brake = 1.0
            self.vehicle.apply_control(ctrl)
        except Exception:
            pass
        time.sleep(1.0)
        self.collision_occurred = False

    def load_model(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Using device:", self.device)
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device)
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model not found: {self.model_path}")
        loaded = torch.load(self.model_path, map_location=self.device)
        # handle state_dict or checkpoint
        if isinstance(loaded, dict) and 'state_dict' in loaded:
            state = loaded['state_dict']
        else:
            state = loaded
        try:
            # try loading directly or handle module. prefix
            try:
                self.model.load_state_dict(state)
            except RuntimeError:
                new_state = {}
                for k, v in state.items():
                    nk = k[7:] if k.startswith("module.") else k
                    new_state[nk] = v
                self.model.load_state_dict(new_state)
            print("Model loaded:", self.model_path)
        except Exception as e:
            raise RuntimeError("Failed to load model state dict: " + str(e))
        self.model.eval()

    # ------------- lane/traffic helpers -------------
    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        try:
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            if waypoint is None:
                return 0.0
            lane_center = waypoint.transform.location
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            cross_track_error = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            total_error = cross_track_error + 0.5 * heading_error
            return total_error
        except Exception:
            return 0.0

    def get_forward_vector(self, transform):
        f = transform.get_forward_vector()
        return np.array([f.x, f.y, f.z], dtype=np.float32)

    def nearest_traffic_light_ahead(self):
        """
        Find a traffic light ahead of the vehicle within TRAFFIC_LIGHT_DIST and within forward cone.
        Returns (traffic_light_actor, distance) or (None, None).
        """
        if self.vehicle is None or self.world is None:
            return None, None
        try:
            veh_tf = self.vehicle.get_transform()
            veh_loc = veh_tf.location
            veh_fwd = self.get_forward_vector(veh_tf)
            lights = self.world.get_actors().filter("traffic.traffic_light*")
            best = None
            best_dist = None
            for tl in lights:
                tl_loc = tl.get_transform().location
                vec = np.array([tl_loc.x - veh_loc.x, tl_loc.y - veh_loc.y, tl_loc.z - veh_loc.z], dtype=np.float32)
                dist = float(np.linalg.norm(vec))
                if dist > self.TRAFFIC_LIGHT_DIST:
                    continue
                if dist < 1e-3:
                    continue
                vec_norm = vec / dist
                dot = float(np.dot(vec_norm, veh_fwd))
                # ahead if dot ~ 0.5..1.0 (tweakable). use 0.5 to be permissive
                if dot < 0.5:
                    continue
                # prefer closest ahead
                if best is None or dist < best_dist:
                    best = tl
                    best_dist = dist
            return best, best_dist
        except Exception:
            return None, None

    def can_initiate_lane_change(self, waypoint, direction):
        try:
            if direction == "left":
                adj = waypoint.get_left_lane()
            else:
                adj = waypoint.get_right_lane()
            if adj is None:
                return False
            return adj.lane_type == carla.LaneType.Driving
        except Exception:
            return False

    def initiate_lane_change_path(self, direction_prefer=None):
        """
        Build a waypoint path in an adjacent lane (left/right). Sets lane change active if successful.
        direction_prefer: "left" / "right" / None (random)
        """
        try:
            loc = self.vehicle.get_location()
            wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
            if wp is None:
                return False
            directions = ["left", "right"] if direction_prefer is None else [direction_prefer]
            # if preferred fails, try the other
            for dirc in directions + (["left", "right"] if direction_prefer else []):
                if not self.can_initiate_lane_change(wp, dirc):
                    continue
                target = wp.get_left_lane() if dirc == "left" else wp.get_right_lane()
                if target is None:
                    continue
                # step forward a little to avoid sharp cross
                for _ in range(3):
                    nxts = target.next(2.0)
                    if nxts:
                        target = nxts[0]
                waypoints = []
                cur = target
                waypoints.append(cur)
                for _ in range(20):
                    nxts = cur.next(2.0)
                    if not nxts:
                        break
                    cur = nxts[0]
                    waypoints.append(cur)
                if len(waypoints) * 2.0 < 8.0:
                    continue
                # activate lane change
                self.lane_change_active = True
                self.lane_change_waypoints = waypoints
                self.lane_change_start_time = time.time()
                print(f"Initiated lane change to {dirc} lane (path length ~{len(waypoints)*2:.1f}m).")
                return True
            return False
        except Exception:
            traceback.print_exc()
            return False

    def execute_lane_change(self, vehicle_loc, vehicle_rot, current_steer, current_throttle):
        """
        Blend steering/throttle with lane-change target.
        Return (blended_steer, blended_throttle, finished_bool)
        """
        try:
            if not self.lane_change_active or not self.lane_change_waypoints:
                return current_steer, current_throttle, True
            # pick a look-ahead waypoint index
            idx = min(6, len(self.lane_change_waypoints)-1)
            target_wp = self.lane_change_waypoints[idx]
            tloc = target_wp.transform.location
            target_yaw = math.radians(target_wp.transform.rotation.yaw)
            vehicle_yaw = math.radians(vehicle_rot.yaw)
            yaw_diff = target_yaw - vehicle_yaw
            while yaw_diff > math.pi:
                yaw_diff -= 2*math.pi
            while yaw_diff < -math.pi:
                yaw_diff += 2*math.pi
            yaw_error = math.sin(yaw_diff)
            steer_cmd = float(np.clip(1.0 * yaw_error, -0.6, 0.6))
            # lateral error (approx) using target location
            vx = tloc.x - vehicle_loc.x
            vy = tloc.y - vehicle_loc.y
            fwd = vehicle_rot.get_forward_vector()
            lateral_error = vx * fwd.y - vy * fwd.x
            error_weight = float(np.clip(abs(lateral_error) / 1.0, 0.0, 0.9))
            blended = (1 - error_weight) * current_steer + error_weight * steer_cmd
            blended = 0.7 * blended + 0.3 * current_steer  # smoothing
            blended_throttle = min(current_throttle + 0.05, 0.75)
            lane_center_error = abs(self.calculate_lane_error(vehicle_loc, vehicle_rot))
            if lane_center_error < self.lane_change_finish_threshold and abs(yaw_diff) < math.radians(8):
                self.lane_change_active = False
                self.lane_change_waypoints = []
                self.lane_change_start_time = None
                print("✅ Lane change complete (centered & aligned).")
                return blended, blended_throttle, True
            # timeout condition
            if time.time() - (self.lane_change_start_time or time.time()) > (self.lane_change_duration + 4.0):
                print("⚠️ Lane change timeout; aborting.")
                self.lane_change_active = False
                self.lane_change_waypoints = []
                self.lane_change_start_time = None
                return blended, blended_throttle, True
            return blended, blended_throttle, False
        except Exception:
            traceback.print_exc()
            self.lane_change_active = False
            self.lane_change_waypoints = []
            self.lane_change_start_time = None
            return current_steer, current_throttle, True

    # ------------- main loop -------------
    def adaptive_speed_control(self, current_speed_mps, lane_offset, steering_magnitude):
        target_speed_mps = self.target_speed_kmh / 3.6
        offset_factor = max(0.7, 1.0 - abs(lane_offset) / 3.0)
        steering_factor = max(0.8, 1.0 - abs(steering_magnitude) / 0.5)
        adjusted_target = target_speed_mps * offset_factor * steering_factor
        speed_error = adjusted_target - current_speed_mps
        if speed_error > 0:
            throttle = min(0.7, 0.4 + speed_error * 0.3)
        else:
            throttle = max(0.0, 0.4 + speed_error * 0.2)
        return throttle

    def run_evaluation(self):
        print("="*60)
        print("RUNNING: CNN control + traffic-light-triggered smooth lane change")
        print("="*60)
        try:
            self.connect_carla()
            start_loc = self.spawn_vehicle()
            self.setup_sensors()
            self.load_model()
        except Exception as e:
            print("Init failed:", e)
            traceback.print_exc()
            return

        spectator = self.world.get_spectator()
        ctrl = carla.VehicleControl()
        ctrl.manual_gear_shift = False

        # warmup
        print("Warming sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)

        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log = time.time()

        logname = f"eval_log_{int(self.start_time)}.csv"
        logf = open(logname, "w")
        logf.write("timestamp,frame,x,y,z,yaw,speed_mps,speed_kmh,model_steer,final_steer,throttle,lane_error\n")

        image_dir = f"eval_images_{int(self.start_time)}"
        os.makedirs(image_dir, exist_ok=True)

        try:
            while self.running:
                # gather latest frames
                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False

                # detect traffic light ahead
                tl_actor, tl_dist = self.nearest_traffic_light_ahead()
                tl_state = None
                if tl_actor is not None:
                    try:
                        tl_state = tl_actor.get_state()
                    except Exception:
                        tl_state = None

                # when red light ahead -> try lane change (or stop)
                trigger_lane_change = False
                if tl_actor is not None and tl_state is not None:
                    # compare states
                    if tl_state == carla.TrafficLightState.Red or (self.DETECT_YELLOW and tl_state == carla.TrafficLightState.Yellow):
                        trigger_lane_change = True

                # Process when we have frames
                if len(frame_stack) == 3:
                    tf = self.vehicle.get_transform()
                    loc = tf.location
                    rot = tf.rotation
                    vel = self.vehicle.get_velocity()
                    speed_mps = math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
                    speed_kmh = speed_mps * 3.6

                    lane_error = self.calculate_lane_error(loc, rot)
                    input_tensor = preprocess_image_stack(frame_stack, self.device)

                    if input_tensor is None:
                        continue

                    speed_norm = min(speed_mps / 30.0, 1.0)
                    speed_tensor = torch.tensor([[speed_norm]], dtype=torch.float32, device=self.device)
                    with torch.no_grad():
                        output = self.model(input_tensor, aux=speed_tensor)
                    model_steer, model_throttle, model_brake = output[0].cpu().numpy()

                    now = time.time()
                    pid_corr = self.pid_controller.update(lane_error, now)
                    err_w = min(abs(lane_error) / 1.0, 0.8)
                    combined = (1 - err_w) * model_steer + err_w * pid_corr
                    final_steer = (self.steer_alpha * combined + (1 - self.steer_alpha) * self.steer_smooth)
                    self.steer_smooth = final_steer
                    final_steer = float(np.clip(final_steer, -0.6, 0.6))

                    adaptive_throttle = self.adaptive_speed_control(speed_mps, lane_error, abs(final_steer))
                    final_throttle = (self.throttle_alpha * adaptive_throttle + (1 - self.throttle_alpha) * self.throttle_smooth)
                    self.throttle_smooth = final_throttle
                    final_throttle = float(np.clip(max(final_throttle, 0.0), 0.0, 0.9))

                    # If traffic light ahead and red => attempt lane change (if not already active)
                    if trigger_lane_change and not self.lane_change_active:
                        # attempt lane change with preference: first try left then right
                        ok = self.initiate_lane_change_path(direction_prefer=None)
                        if not ok:
                            # can't change lane: stop before light
                            print("🔴 Red light ahead and no adjacent lane available -> slowing to stop.")
                            final_throttle = 0.0
                            # apply brakes if near
                            if tl_dist is not None and tl_dist < 8.0:
                                ctrl.brake = 1.0
                                final_throttle = 0.0
                        else:
                            ctrl.brake = 0.0

                    # If lane-change active -> override/ blend steering + throttle
                    if self.lane_change_active:
                        final_steer, final_throttle, done = self.execute_lane_change(loc, rot, final_steer, final_throttle)
                        if done:
                            pass  # execute_lane_change handles finishing state

                    # Apply controls
                    ctrl.steer = float(final_steer)
                    ctrl.throttle = float(final_throttle)
                    ctrl.brake = float(0.0)
                    ctrl.hand_brake = False
                    try:
                        self.vehicle.apply_control(ctrl)
                    except Exception:
                        pass

                    # Save frame
                    try:
                        fname = os.path.join(image_dir, f"{frame_count:06d}.png")
                        cv2.imwrite(fname, cv2.cvtColor(self.latest_image, cv2.COLOR_RGB2BGR))
                    except Exception:
                        pass

                    # record stats
                    self.deviations.append(abs(lane_error))
                    self.controls_history.append({'frame': frame_count, 'model_steer': float(model_steer),
                                                  'final_steer': float(final_steer), 'throttle': float(final_throttle),
                                                  'speed': float(speed_mps), 'lane_error': float(lane_error)})
                    self.trajectory.append({'x': loc.x, 'y': loc.y, 'z': loc.z, 'yaw': rot.yaw})

                    tstamp = time.time() - self.start_time
                    logf.write(f"{tstamp:.3f},{frame_count},{loc.x:.2f},{loc.y:.2f},{loc.z:.2f},{rot.yaw:.2f},"
                               f"{speed_mps:.3f},{speed_kmh:.1f},{model_steer:.4f},{final_steer:.4f},{final_throttle:.4f},{lane_error:.4f}\n")
                    logf.flush()

                    # spectator follow
                    try:
                        transform = self.vehicle.get_transform()
                        loc2 = transform.location
                        rot2 = transform.rotation
                        spectator.set_transform(carla.Transform(loc2 + transform.get_forward_vector() * -6 + carla.Location(z=3),
                                                                carla.Rotation(pitch=-15, yaw=rot2.yaw)))
                    except Exception:
                        pass

                    # print periodic status
                    if time.time() - last_log >= 1.0:
                        tl_msg = "None"
                        if tl_actor is not None:
                            try:
                                st = tl_actor.get_state()
                                tl_msg = f"{st.name} @ {tl_dist:.1f}m"
                            except Exception:
                                tl_msg = f"@{tl_dist:.1f}m"
                        print(f"[{tstamp:6.1f}s] Frame {frame_count:5d} | Speed: {speed_kmh:5.1f} km/h | "
                              f"LaneErr: {lane_error:+.3f} m | Steering (M/F): {model_steer:+.3f}/{final_steer:+.3f} | TL: {tl_msg} | LC_active: {self.lane_change_active}")
                        last_log = time.time()

                    frame_count += 1

                # tick
                self.world.tick()

        except Exception as e:
            print("Evaluation error:", e)
            traceback.print_exc()

        finally:
            try:
                logf.close()
            except Exception:
                pass
            # assemble video
            try:
                imgs = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
                if imgs:
                    vid = f"eval_video_{int(self.start_time)}.mp4"
                    with imageio.get_writer(vid, fps=30) as writer:
                        for im in imgs:
                            writer.append_data(imageio.imread(im))
                    print("Saved video:", vid)
            except Exception as e:
                print("Video creation warning:", e)

            try:
                self.print_statistics(frame_count)
            except Exception:
                pass
            self.cleanup()

    def print_statistics(self, frame_count):
        elapsed = time.time() - (self.start_time or time.time())
        print("\n" + "="*40)
        print("EVALUATION STATS")
        print("="*40)
        print("Frames:", frame_count)
        print("Time: %.2f s" % elapsed)
        if elapsed > 0:
            print("Avg FPS:", frame_count / elapsed)
        print("Collisions:", self.collision_count)
        if self.deviations:
            print("Lane avg offset: %.3f m" % (np.mean(self.deviations)))
            print("Lane max offset: %.3f m" % (np.max(self.deviations)))
        print("="*40)

    def cleanup(self):
        print("Cleaning up actors and restoring world settings...")
        try:
            if self.world is not None and self.original_settings is not None:
                self.world.apply_settings(self.original_settings)
        except Exception:
            pass
        actors = []
        for a in (self.camera, self.collision_sensor, self.vehicle):
            try:
                if a is not None:
                    try:
                        a.stop()
                    except Exception:
                        pass
                    actors.append(a)
            except Exception:
                pass
        for a in actors:
            try:
                a.destroy()
            except Exception:
                pass
        try:
            gc.collect()
        except Exception:
            pass
        print("Cleanup done.")


# ----------------- Entrypoint -----------------
def main():
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    evaluator = ClosedLoopEvaluator(model_path=MODEL_PATH,
                                    detect_yellow=False,        # set True if you want yellow to trigger changes
                                    traffic_light_distance=25.0)
    evaluator.run_evaluation()


if __name__ == "__main__":
    main()


RUNNING: CNN control + traffic-light-triggered smooth lane change
Spawned vehicle at (54.47, -64.35)
Using device: cuda
Model loaded: /home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth
Warming sensors...


/tmp/ipykernel_2569467/3403186607.py:279: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded = torch.load(self.model_path, map_location=self.device)


[   1.0s] Frame    50 | Speed:  13.3 km/h | LaneErr: +0.057 m | Steering (M/F): +0.007/+0.011 | TL: None | LC_active: False
[   2.0s] Frame   103 | Speed:  16.2 km/h | LaneErr: -0.085 m | Steering (M/F): +0.012/+0.006 | TL: None | LC_active: False
[   3.0s] Frame   158 | Speed:  16.0 km/h | LaneErr: -0.082 m | Steering (M/F): +0.006/+0.002 | TL: None | LC_active: False
[   4.0s] Frame   212 | Speed:  15.9 km/h | LaneErr: +0.147 m | Steering (M/F): +0.012/+0.031 | TL: None | LC_active: False
[   5.0s] Frame   271 | Speed:  16.0 km/h | LaneErr: -0.138 m | Steering (M/F): +0.006/-0.018 | TL: None | LC_active: False
[   6.0s] Frame   332 | Speed:  16.1 km/h | LaneErr: -0.037 m | Steering (M/F): +0.014/+0.006 | TL: None | LC_active: False
[   7.1s] Frame   397 | Speed:  16.1 km/h | LaneErr: -0.091 m | Steering (M/F): +0.004/-0.006 | TL: Yellow @ 17.2m | LC_active: False
Initiated lane change to left lane (path length ~42.0m).
[   8.1s] Frame   459 | Speed:  16.6 km/h | LaneErr: +0.218 m | S

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (160, 120) to (160, 128) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Graceful shutdown requested.

*** Collision #3 at 13.73s ***
    Impact with: static.pole
Saved video: eval_video_1760294594.mp4

EVALUATION STATS
Frames: 759
Time: 14.04 s
Avg FPS: 54.04386604683012
Collisions: 3
Lane avg offset: 0.758 m
Lane max offset: 4.308 m
Cleaning up actors and restoring world settings...
Cleanup done.


In [ ]:
import carla
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import cv2
import gc
import imageio.v2 as imageio
import signal
import math
import traceback

# Environment setup (limits threads for reproducibility)
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


# ----------------- Utilities -----------------
def preprocess_image_stack(frame_stack, device):
    """Preprocess 3 RGB frames -> (1, 9, 120, 160) tensor on device."""
    if len(frame_stack) != 3:
        return None
    processed_frames = []
    for frame in frame_stack:
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        frame = frame.astype(np.float32) / 255.0
        frame = (frame - 0.5) / 0.5
        processed_frames.append(frame)
    stacked = np.concatenate(processed_frames, axis=2)  # (120,160,9)
    stacked = np.transpose(stacked, (2, 0, 1))  # (9,120,160)
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)
    return tensor.to(device)


class ImprovedPIDController:
    """Softer PID for lateral correction (reduced gains to avoid oscillation)."""
    def __init__(self):
        # Softer gains (reduced from your earlier values)
        self.Kp = 0.45
        self.Ki = 0.005
        self.Kd = 0.04
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        self.integral_limit = 0.5
        self.output_limit = 0.4  # reduce maximum PID steering contribution

    def update(self, error, current_time):
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05
        else:
            dt = max(current_time - self.prev_time, 1e-3)
            self.prev_time = current_time
        P = self.Kp * error
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        D = self.Kd * derivative
        self.prev_error = error
        output = P + I + D
        return float(np.clip(output, -self.output_limit, self.output_limit))

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


# ----------------- Main evaluator -----------------
class ClosedLoopEvaluator:
    def __init__(self, model_path, host="localhost", port=2000, detect_yellow=False, traffic_light_distance=25.0):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        # controllers & smoothing
        self.pid_controller = ImprovedPIDController()
        self.steer_smooth = 0.0
        self.throttle_smooth = 0.3
        # stronger smoothing (lower alpha -> slower changes)
        self.steer_alpha = 0.12
        self.throttle_alpha = 0.15

        # steering rate limit (max change in steer per tick)
        self.max_steer_delta = 0.04  # tweak down for even smoother

        # speed / thresholds
        self.target_speed_kmh = 15.0
        self.max_lane_offset = 2.0

        # lane-change state machine (we'll only change lane/turn after green)
        self.lane_change_active = False
        self.lane_change_waypoints = []
        self.lane_change_start_time = None
        self.lane_change_duration = 2.8
        self.lane_change_finish_threshold = 0.22

        # traffic-light settings
        self.DETECT_YELLOW = detect_yellow
        self.TRAFFIC_LIGHT_DIST = traffic_light_distance

        # red/green behavior states
        self.state = "DRIVING"  # other: 'STOPPING', 'WAITING_GREEN', 'LANE_CHANGE'
        self.stop_start_time = None
        self.stop_speed_threshold = 0.5  # m/s considered stopped
        self.red_stop_distance = 15.0  # distance to start stopping for red (meters)
        self.green_resume_delay = 0.2  # slight delay after green before moving

        # stats & actors
        self.collision_count = 0
        self.deviations = []
        self.controls_history = []
        self.trajectory = []

        # placeholders
        self.client = None
        self.world = None
        self.map = None
        self.vehicle = None
        self.camera = None
        self.collision_sensor = None
        self.latest_image = None     # raw camera frame for model
        self.new_image_available = False
        self.collision_occurred = False
        self.start_time = None
        self.original_settings = None

        # image stabilization for saved video/spectator (exponential moving average)
        self._saved_frame_ema = None
        self._ema_alpha = 0.25  # smoothing for saved video frames

        # graceful shutdown
        signal.signal(signal.SIGINT, self.signal_handler)

    def signal_handler(self, sig, frame):
        print("\nGraceful shutdown requested.")
        self.running = False

    def connect_carla(self):
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        self.original_settings = self.world.get_settings()
        self.map = self.world.get_map()

        # enable synchronous mode
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)

    def spawn_vehicle(self):
        bp_lib = self.world.get_blueprint_library()
        candidates = bp_lib.filter("vehicle.*")
        if not candidates:
            raise RuntimeError("No vehicle blueprints found.")
        vehicle_bp = None
        for name in ("vehicle.tesla.model3", "vehicle.mercedes.coupe_2020", "vehicle.audi.tt"):
            items = bp_lib.filter(name)
            if items:
                vehicle_bp = items[0]
                break
        if vehicle_bp is None:
            vehicle_bp = random.choice(candidates)
        spawn_points = self.map.get_spawn_points()
        if not spawn_points:
            raise RuntimeError("No spawn points available.")
        sp = random.choice(spawn_points)
        sp.location.z += 0.5
        self.vehicle = self.world.spawn_actor(vehicle_bp, sp)
        print(f"Spawned vehicle at ({sp.location.x:.2f}, {sp.location.y:.2f})")
        return sp.location

    def setup_sensors(self):
        bp_lib = self.world.get_blueprint_library()
        cam_bp = bp_lib.find("sensor.camera.rgb")
        # keep camera attached for model input, but use small fov and stable mount to reduce shake
        cam_bp.set_attribute("image_size_x", "800")
        cam_bp.set_attribute("image_size_y", "600")
        cam_bp.set_attribute("fov", "100")
        # mount camera near hood, slightly forward and up (more stable center of mass)
        cam_transform = carla.Transform(carla.Location(x=1.3, y=0.0, z=1.8), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)

        col_bp = bp_lib.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(col_bp, carla.Transform(), attach_to=self.vehicle)

        # init placeholders
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False

        # callbacks
        self.camera.listen(self.camera_callback)
        self.collision_sensor.listen(self.collision_callback)

    def camera_callback(self, image):
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]  # BGRA -> BGR
            arr = arr[:, :, ::-1]  # BGR -> RGB
            # create small frame for model (120x160) and keep original sized for saved/stabilized video
            small = cv2.resize(arr, (160, 120))
            self.latest_image = small
            self.new_image_available = True

            # update EMA stabilized frame for saving/visualization
            # convert to uint8 BGR for saving
            save_frame = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
            if self._saved_frame_ema is None:
                self._saved_frame_ema = save_frame.astype(np.float32)
            else:
                # EMA to smooth apparent camera jitter in saved video
                self._saved_frame_ema = (self._ema_alpha * save_frame.astype(np.float32) +
                                         (1 - self._ema_alpha) * self._saved_frame_ema)
        except Exception:
            pass

    def collision_callback(self, event):
        self.collision_count += 1
        t = time.time() - (self.start_time or time.time())
        print(f"\n*** Collision #{self.collision_count} at {t:.2f}s ***")
        try:
            print("    Impact with:", event.other_actor.type_id)
        except Exception:
            pass
        self.collision_occurred = True
        try:
            # emergency stop
            self.vehicle.set_velocity(carla.Vector3D(0, 0, 0))
        except Exception:
            pass
        self.pid_controller.reset()
        try:
            ctrl = carla.VehicleControl()
            ctrl.steer = 0.0
            ctrl.throttle = 0.0
            ctrl.brake = 1.0
            self.vehicle.apply_control(ctrl)
        except Exception:
            pass
        time.sleep(1.0)
        self.collision_occurred = False

    def load_model(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Using device:", self.device)
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device)
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model not found: {self.model_path}")
        loaded = torch.load(self.model_path, map_location=self.device)
        if isinstance(loaded, dict) and 'state_dict' in loaded:
            state = loaded['state_dict']
        else:
            state = loaded
        try:
            try:
                self.model.load_state_dict(state)
            except RuntimeError:
                new_state = {}
                for k, v in state.items():
                    nk = k[7:] if k.startswith("module.") else k
                    new_state[nk] = v
                self.model.load_state_dict(new_state)
            print("Model loaded:", self.model_path)
        except Exception as e:
            raise RuntimeError("Failed to load model state dict: " + str(e))
        self.model.eval()

    # ---------------- helpers ----------------
    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        """Cross-track + heading error to lane center (meters / radians weighted)"""
        try:
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            if waypoint is None:
                return 0.0
            lane_center = waypoint.transform.location
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            cross_track_error = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            total_error = cross_track_error + 0.5 * heading_error
            return total_error
        except Exception:
            return 0.0

    def get_forward_vector(self, transform):
        f = transform.get_forward_vector()
        return np.array([f.x, f.y, f.z], dtype=np.float32)

    def nearest_traffic_light_ahead(self):
        """Return (traffic_light_actor, distance) if one exists ahead in forward cone and distance, else (None, None)."""
        if self.vehicle is None or self.world is None:
            return None, None
        try:
            veh_tf = self.vehicle.get_transform()
            veh_loc = veh_tf.location
            veh_fwd = self.get_forward_vector(veh_tf)
            lights = self.world.get_actors().filter("traffic.traffic_light*")
            best = None
            best_dist = None
            for tl in lights:
                tl_loc = tl.get_transform().location
                vec = np.array([tl_loc.x - veh_loc.x, tl_loc.y - veh_loc.y, tl_loc.z - veh_loc.z], dtype=np.float32)
                dist = float(np.linalg.norm(vec))
                if dist > self.TRAFFIC_LIGHT_DIST:
                    continue
                if dist < 1e-3:
                    continue
                vec_norm = vec / dist
                dot = float(np.dot(vec_norm, veh_fwd))
                # use a strict forward cone so we don't detect side signals
                if dot < 0.7:
                    continue
                if best is None or dist < best_dist:
                    best = tl
                    best_dist = dist
            return best, best_dist
        except Exception:
            return None, None

    def can_initiate_lane_change(self, waypoint, direction):
        try:
            if direction == "left":
                adj = waypoint.get_left_lane()
            else:
                adj = waypoint.get_right_lane()
            if adj is None:
                return False
            return adj.lane_type == carla.LaneType.Driving
        except Exception:
            return False

    def initiate_lane_change_path(self, direction_prefer=None):
        """Build a waypoint path in an adjacent lane. Returns True if successful and activates lane_change state."""
        try:
            loc = self.vehicle.get_location()
            wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
            if wp is None:
                return False
            directions = ["left", "right"] if direction_prefer is None else [direction_prefer]
            for dirc in directions + (["left", "right"] if direction_prefer else []):
                if not self.can_initiate_lane_change(wp, dirc):
                    continue
                target = wp.get_left_lane() if dirc == "left" else wp.get_right_lane()
                if target is None:
                    continue
                # step forward to create smoother path
                for _ in range(3):
                    nxts = target.next(2.0)
                    if nxts:
                        target = nxts[0]
                waypoints = []
                cur = target
                waypoints.append(cur)
                for _ in range(20):
                    nxts = cur.next(2.0)
                    if not nxts:
                        break
                    cur = nxts[0]
                    waypoints.append(cur)
                if len(waypoints) * 2.0 < 8.0:
                    continue
                # activate
                self.lane_change_active = True
                self.lane_change_waypoints = waypoints
                self.lane_change_start_time = time.time()
                print(f"Initiated lane change to {dirc} lane (path length ~{len(waypoints)*2:.1f}m).")
                return True
            return False
        except Exception:
            traceback.print_exc()
            return False

    def execute_lane_change(self, vehicle_loc, vehicle_rot, current_steer, current_throttle):
        """Blend steering/throttle toward lane-change waypoint targets. Returns blended steer, throttle, finished_bool."""
        try:
            if not self.lane_change_active or not self.lane_change_waypoints:
                return current_steer, current_throttle, True
            idx = min(6, len(self.lane_change_waypoints)-1)
            target_wp = self.lane_change_waypoints[idx]
            tloc = target_wp.transform.location
            target_yaw = math.radians(target_wp.transform.rotation.yaw)
            vehicle_yaw = math.radians(vehicle_rot.yaw)
            yaw_diff = target_yaw - vehicle_yaw
            while yaw_diff > math.pi:
                yaw_diff -= 2*math.pi
            while yaw_diff < -math.pi:
                yaw_diff += 2*math.pi
            yaw_error = math.sin(yaw_diff)
            steer_cmd = float(np.clip(0.9 * yaw_error, -0.6, 0.6))
            vx = tloc.x - vehicle_loc.x
            vy = tloc.y - vehicle_loc.y
            fwd = vehicle_rot.get_forward_vector()
            lateral_error = vx * fwd.y - vy * fwd.x
            error_weight = float(np.clip(abs(lateral_error) / 1.0, 0.0, 0.95))
            blended = (1 - error_weight) * current_steer + error_weight * steer_cmd
            blended = 0.75 * blended + 0.25 * current_steer
            blended_throttle = min(current_throttle + 0.05, 0.7)
            lane_center_error = abs(self.calculate_lane_error(vehicle_loc, vehicle_rot))
            if lane_center_error < self.lane_change_finish_threshold and abs(yaw_diff) < math.radians(8):
                self.lane_change_active = False
                self.lane_change_waypoints = []
                self.lane_change_start_time = None
                print("✅ Lane change complete (centered & aligned).")
                return blended, blended_throttle, True
            if time.time() - (self.lane_change_start_time or time.time()) > (self.lane_change_duration + 4.0):
                print("⚠️ Lane change timeout; aborting.")
                self.lane_change_active = False
                self.lane_change_waypoints = []
                self.lane_change_start_time = None
                return blended, blended_throttle, True
            return blended, blended_throttle, False
        except Exception:
            traceback.print_exc()
            self.lane_change_active = False
            self.lane_change_waypoints = []
            self.lane_change_start_time = None
            return current_steer, current_throttle, True

    def adaptive_speed_control(self, current_speed_mps, lane_offset, steering_magnitude):
        """Simple adaptive throttle policy used during driving (not during stopping)."""
        target_speed_mps = self.target_speed_kmh / 3.6
        offset_factor = max(0.7, 1.0 - abs(lane_offset) / 3.0)
        steering_factor = max(0.75, 1.0 - abs(steering_magnitude) / 0.6)
        adjusted_target = target_speed_mps * offset_factor * steering_factor
        speed_error = adjusted_target - current_speed_mps
        if speed_error > 0:
            throttle = min(0.6, 0.38 + speed_error * 0.28)
        else:
            throttle = max(0.0, 0.33 + speed_error * 0.22)
        return float(np.clip(throttle, 0.0, 0.9))

    # ---------------- main loop ----------------
    def run_evaluation(self):
        print("\n" + "="*40)
        print("STARTING: CNN + safe stop at red + controlled lane-change after green")
        print("="*40)
        try:
            self.connect_carla()
            start_location = self.spawn_vehicle()
            self.setup_sensors()
            self.load_model()
        except Exception as e:
            print("Init failed:", e)
            traceback.print_exc()
            return

        spectator = self.world.get_spectator()
        ctrl = carla.VehicleControl()
        ctrl.manual_gear_shift = False

        # warmup ticks
        print("Warming sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)

        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log = time.time()

        log_filename = f"eval_log_{int(self.start_time)}.csv"
        log_file = open(log_filename, "w")
        log_file.write("timestamp,frame,x,y,z,yaw,speed_mps,speed_kmh,model_steer,final_steer,throttle,lane_error,state\n")

        image_dir = f"eval_images_{int(self.start_time)}"
        os.makedirs(image_dir, exist_ok=True)

        try:
            while self.running:
                # tick at top of loop to keep sync (we call world.tick multiple times sometimes)
                self.world.tick()

                # spectator smoothing (visual only)
                try:
                    tf = self.vehicle.get_transform()
                    # smooth spectator by lerping from previous transform
                    sp_loc = tf.location + tf.get_forward_vector() * -6 + carla.Location(z=3.0)
                    sp_rot = carla.Rotation(pitch=-12, yaw=tf.rotation.yaw)
                    spectator.set_transform(carla.Transform(sp_loc, sp_rot))
                except Exception:
                    pass

                # collect frame for model
                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False

                # detect traffic light ahead & its distance/state
                tl_actor, tl_dist = self.nearest_traffic_light_ahead()
                tl_state = None
                if tl_actor is not None:
                    try:
                        tl_state = tl_actor.get_state()
                    except Exception:
                        tl_state = None

                # decide state transitions
                # If red light ahead within red_stop_distance -> STOPPING
                if tl_actor is not None and tl_state is not None and (tl_state == carla.TrafficLightState.Red):
                    if tl_dist is not None and tl_dist <= self.red_stop_distance and self.state == "DRIVING":
                        print(f"Red light detected at {tl_dist:.1f}m -> initiating stop.")
                        self.state = "STOPPING"
                        self.stop_start_time = None
                # If currently waiting and light turned green -> prepare to lane-change
                if self.state == "WAITING_GREEN" and tl_actor is not None and tl_state == carla.TrafficLightState.Green:
                    print("Green detected -> scheduling lane change/turn.")
                    # Initiate lane change path (choose a direction preference: try left then right)
                    ok = self.initiate_lane_change_path(direction_prefer=None)
                    if ok:
                        self.state = "LANE_CHANGE"
                    else:
                        # if no lane change available, just resume DRIVING
                        print("No lane available for lane-change. Resuming driving.")
                        self.state = "DRIVING"

                # Process when we have 3 frames
                if len(frame_stack) == 3:
                    tf = self.vehicle.get_transform()
                    loc = tf.location
                    rot = tf.rotation
                    vel = self.vehicle.get_velocity()
                    speed_mps = math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
                    speed_kmh = speed_mps * 3.6

                    lane_error = self.calculate_lane_error(loc, rot)

                    input_tensor = preprocess_image_stack(frame_stack, self.model.parameters().__iter__().__next__().device if hasattr(self, 'model') else torch.device('cpu'))
                    if input_tensor is None:
                        continue

                    # speed input
                    speed_norm = min(speed_mps / 30.0, 1.0)
                    speed_tensor = torch.tensor([[speed_norm]], dtype=torch.float32, device=self.model.parameters().__iter__().__next__().device)

                    # model inference
                    with torch.no_grad():
                        output = self.model(input_tensor, aux=speed_tensor)
                    model_steer, model_throttle, model_brake = output[0].cpu().numpy()

                    now = time.time()
                    pid_corr = self.pid_controller.update(lane_error, now)

                    # blend model + PID
                    err_w = min(abs(lane_error) / 1.0, 0.7)  # reduce max PID weight
                    combined = (1 - err_w) * model_steer + err_w * pid_corr

                    # steering smoothing (low-pass)
                    final_steer_candidate = (self.steer_alpha * combined + (1 - self.steer_alpha) * self.steer_smooth)

                    # limit steering rate to avoid oscillation / shaking
                    delta = float(final_steer_candidate - self.steer_smooth)
                    delta = np.clip(delta, -self.max_steer_delta, self.max_steer_delta)
                    final_steer = float(self.steer_smooth + delta)
                    # clamp absolute steering
                    final_steer = float(np.clip(final_steer, -0.6, 0.6))
                    # update smoothed steer
                    self.steer_smooth = final_steer

                    # throttle smoothing (normal driving)
                    adaptive_throttle = self.adaptive_speed_control(speed_mps, lane_error, abs(final_steer))
                    final_throttle_candidate = (self.throttle_alpha * adaptive_throttle + (1 - self.throttle_alpha) * self.throttle_smooth)
                    # gentle throttle update (prevent jerk)
                    final_throttle = float(np.clip(final_throttle_candidate, 0.0, 0.9))
                    # update smooth throttle
                    self.throttle_smooth = final_throttle

                    # Default braking off (can be set below)
                    final_brake = 0.0

                    # STATE MACHINE: STOPPING / WAITING_GREEN / LANE_CHANGE / DRIVING
                    if self.state == "STOPPING":
                        # apply brakes to smoothly stop before light
                        # reduce throttle and apply brake proportional to speed
                        if self.stop_start_time is None:
                            self.stop_start_time = time.time()
                            self.pid_controller.reset()  # reset PID so subsequent control is clean
                        # compute required braking: more brake if closer and faster
                        if tl_dist is None:
                            # conservative braking
                            brake_val = min(1.0, 0.6 + (speed_mps / 10.0))
                        else:
                            # proportional to inverse distance
                            approach = max(0.05, (self.red_stop_distance - tl_dist) / max(1.0, self.red_stop_distance))
                            brake_val = np.clip(0.3 + approach * 0.9 + (speed_mps / 20.0), 0.0, 1.0)
                        final_throttle = 0.0
                        final_brake = float(brake_val)
                        # keep steering small and stable during stop
                        final_steer = float(np.clip(final_steer * 0.5, -0.3, 0.3))
                        # consider stopped
                        if speed_mps < self.stop_speed_threshold:
                            print("Vehicle stopped for red light.")
                            self.state = "WAITING_GREEN"
                    elif self.state == "WAITING_GREEN":
                        # Hold vehicle stopped; small braking
                        final_throttle = 0.0
                        final_brake = 1.0
                        # keep steer neutral
                        final_steer = 0.0
                        # remain until we detect green (handled above)
                    elif self.state == "LANE_CHANGE":
                        # execute lane change blending; ensure brakes off
                        final_brake = 0.0
                        final_steer, final_throttle, done = self.execute_lane_change(loc, rot, final_steer, final_throttle)
                        if done:
                            self.state = "DRIVING"
                    else:  # DRIVING
                        # Normal driving behavior (nothing special)
                        final_brake = 0.0

                    # Apply the control
                    ctrl.steer = float(final_steer)
                    ctrl.throttle = float(final_throttle)
                    ctrl.brake = float(final_brake)
                    ctrl.hand_brake = False
                    try:
                        self.vehicle.apply_control(ctrl)
                    except Exception:
                        pass

                    # Save stabilized frame for video (use EMA stored BGR image)
                    try:
                        if self._saved_frame_ema is not None:
                            save_img = np.clip(self._saved_frame_ema, 0, 255).astype(np.uint8)
                            save_path = os.path.join(image_dir, f"{frame_count:06d}.png")
                            cv2.imwrite(save_path, save_img)
                    except Exception:
                        pass

                    # Logging & stats
                    self.deviations.append(abs(lane_error))
                    self.controls_history.append({'frame': frame_count, 'model_steer': float(model_steer),
                                                  'final_steer': float(final_steer), 'throttle': float(final_throttle),
                                                  'speed': float(speed_mps), 'lane_error': float(lane_error),
                                                  'state': self.state})
                    self.trajectory.append({'x': loc.x, 'y': loc.y, 'z': loc.z, 'yaw': rot.yaw})

                    tstamp = time.time() - self.start_time
                    log_file.write(f"{tstamp:.3f},{frame_count},{loc.x:.2f},{loc.y:.2f},{loc.z:.2f},{rot.yaw:.2f},"
                                   f"{speed_mps:.3f},{speed_kmh:.1f},{model_steer:.4f},{final_steer:.4f},{final_throttle:.4f},{lane_error:.4f},{self.state}\n")
                    log_file.flush()

                    # Print periodic status
                    if time.time() - last_log >= 1.0:
                        tl_msg = "None"
                        if tl_actor is not None and tl_dist is not None:
                            try:
                                st = tl_actor.get_state()
                                tl_msg = f"{st.name} @ {tl_dist:.1f}m"
                            except Exception:
                                tl_msg = f"@{tl_dist:.1f}m"
                        print(f"[{tstamp:6.1f}s] Frame {frame_count:5d} | Speed: {speed_kmh:5.1f} km/h | "
                              f"LaneErr: {lane_error:+.3f} m | Steering (M/F): {model_steer:+.3f}/{final_steer:+.3f} | TL: {tl_msg} | STATE: {self.state}")
                        last_log = time.time()

                    frame_count += 1

                # end processing frames

            # end main while
        except Exception as e:
            print("Evaluation error:", e)
            traceback.print_exc()
        finally:
            try:
                log_file.close()
            except Exception:
                pass
            # assemble stabilized video from saved frames
            try:
                imgs = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
                if imgs:
                    out_vid = f"eval_video_{int(self.start_time)}.mp4"
                    with imageio.get_writer(out_vid, fps=30) as writer:
                        for im in imgs:
                            writer.append_data(imageio.imread(im))
                    print("Saved stabilized video:", out_vid)
            except Exception as e:
                print("Video creation warning:", e)

            try:
                self.print_statistics(frame_count)
            except Exception:
                pass
            self.cleanup()

    def print_statistics(self, frame_count):
        elapsed = time.time() - (self.start_time or time.time())
        print("\n" + "="*40)
        print("EVALUATION STATS")
        print("="*40)
        print("Frames:", frame_count)
        print("Time: %.2f s" % elapsed)
        if elapsed > 0:
            print("Avg FPS:", frame_count / elapsed)
        print("Collisions:", self.collision_count)
        if self.deviations:
            print("Lane avg offset: %.3f m" % (np.mean(self.deviations)))
            print("Lane max offset: %.3f m" % (np.max(self.deviations)))
        print("="*40)

    def cleanup(self):
        print("Cleaning up actors and restoring world settings...")
        try:
            if self.world is not None and self.original_settings is not None:
                self.world.apply_settings(self.original_settings)
        except Exception:
            pass
        actors = []
        for a in (self.camera, self.collision_sensor, self.vehicle):
            try:
                if a is not None:
                    try:
                        a.stop()
                    except Exception:
                        pass
                    actors.append(a)
            except Exception:
                pass
        for a in actors:
            try:
                a.destroy()
            except Exception:
                pass
        try:
            gc.collect()
        except Exception:
            pass
        print("Cleanup done.")


# ----------------- Entrypoint -----------------
def main():
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    evaluator = ClosedLoopEvaluator(model_path=MODEL_PATH,
                                    detect_yellow=False,
                                    traffic_light_distance=25.0)
    evaluator.run_evaluation()


if __name__ == "__main__":
    main()



STARTING: CNN + safe stop at red + controlled lane-change after green
Spawned vehicle at (109.52, 89.84)
Using device: cuda
Model loaded: /home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth
Warming sensors...


/tmp/ipykernel_2570726/1584623477.py:309: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded = torch.load(self.model_path, map_location=self.device)


[   1.0s] Frame    40 | Speed:   6.6 km/h | LaneErr: -0.270 m | Steering (M/F): +0.052/+0.006 | TL: None | STATE: DRIVING
[   2.0s] Frame    94 | Speed:  14.8 km/h | LaneErr: -0.045 m | Steering (M/F): +0.005/+0.004 | TL: None | STATE: DRIVING
[   3.0s] Frame   150 | Speed:  14.9 km/h | LaneErr: +0.136 m | Steering (M/F): +0.008/+0.014 | TL: None | STATE: DRIVING
[   4.0s] Frame   207 | Speed:  14.7 km/h | LaneErr: -0.286 m | Steering (M/F): -0.010/-0.025 | TL: Green @ 22.2m | STATE: DRIVING
[   5.0s] Frame   263 | Speed:  15.1 km/h | LaneErr: +0.057 m | Steering (M/F): +0.013/+0.007 | TL: Yellow @ 12.2m | STATE: DRIVING
Red light detected at 14.8m -> initiating stop.
[   6.1s] Frame   317 | Speed:  12.8 km/h | LaneErr: +0.099 m | Steering (M/F): +0.029/+0.014 | TL: Red @ 14.5m | STATE: STOPPING
Vehicle stopped for red light.
[   7.1s] Frame   373 | Speed:   0.0 km/h | LaneErr: +0.041 m | Steering (M/F): -0.002/+0.000 | TL: Red @ 13.6m | STATE: WAITING_GREEN
[   8.1s] Frame   429 | Spe

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Graceful shutdown requested.

*** Collision #18 at 46.96s ***
    Impact with: static.vehicle

*** Collision #19 at 47.97s ***
    Impact with: static.vehicle

*** Collision #20 at 48.97s ***
    Impact with: static.vehicle

*** Collision #21 at 49.97s ***
    Impact with: static.vehicle

*** Collision #22 at 50.97s ***
    Impact with: static.vehicle

*** Collision #23 at 51.97s ***
    Impact with: static.vehicle

*** Collision #24 at 52.97s ***
    Impact with: static.vehicle

*** Collision #25 at 53.97s ***
    Impact with: static.vehicle

*** Collision #26 at 54.98s ***
    Impact with: static.vehicle

*** Collision #27 at 55.98s ***
    Impact with: static.vehicle

*** Collision #28 at 56.98s ***
    Impact with: static.vehicle


In [1]:
import carla
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
import numpy as np
from PIL import Image
import time
import os
import random
import cv2
import gc
import imageio.v2 as imageio
import signal
import math
import traceback

# Environment setup
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24,36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36,48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48,64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64,64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape, device=next(self.parameters()).device)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


# ----------------- Utilities -----------------
def preprocess_image_stack(frame_stack, device):
    """
    Preprocess a stack of 3 RGB frames for the model.
    Matches the exact preprocessing used during training.
    """
    if len(frame_stack) != 3:
        return None
    
    processed_frames = []
    
    # Process each frame individually first (as they would be during training)
    for frame in frame_stack:
        # Resize if needed (should already be 120x160)
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        
        # Convert to float32 and normalize to [0, 1]
        frame = frame.astype(np.float32) / 255.0
        
        # Normalize to [-1, 1] range (standard ImageNet-style normalization)
        frame = (frame - 0.5) / 0.5
        
        processed_frames.append(frame)
    
    # Stack the processed frames along channel dimension
    # This creates a (120, 160, 9) array
    stacked = np.concatenate(processed_frames, axis=2)
    
    # Convert to CHW format for PyTorch (9, 120, 160)
    stacked = np.transpose(stacked, (2, 0, 1))
    
    # Convert to tensor and add batch dimension
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)  # (1, 9, 120, 160)
    
    return tensor.to(device)


class ImprovedPIDController:
    """Enhanced PID controller with better tuning and safeguards (softer to avoid oscillation)"""
    
    def __init__(self):
        # Softer PID gains to reduce over-correction and drift
        self.Kp = 0.45
        self.Ki = 0.008
        self.Kd = 0.06
        
        # State variables
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        
        # Anti-windup limits
        self.integral_limit = 0.5
        self.output_limit = 0.45  # limit PID steering contribution
        
    def update(self, error, current_time):
        """Update PID controller with current error"""
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05  # Default dt
        else:
            dt = max(current_time - self.prev_time, 0.001)  # Prevent division by zero
            self.prev_time = current_time
        
        # Proportional term
        P = self.Kp * error
        
        # Integral term with anti-windup
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        
        # Derivative term with smoothing
        if dt > 0:
            derivative = (error - self.prev_error) / dt
            D = self.Kd * derivative
        else:
            D = 0.0
        
        self.prev_error = error
        
        # Combine and limit output
        output = P + I + D
        return float(np.clip(output, -self.output_limit, self.output_limit))
    
    def reset(self):
        """Reset PID controller state"""
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


# ----------------- Main evaluator -----------------
class ClosedLoopEvaluator:
    def __init__(self, model_path, host="localhost", port=2000):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        # Enhanced control system
        self.pid_controller = ImprovedPIDController()
        self.throttle_smooth = 0.3
        self.steer_smooth = 0.0
        
        # Control smoothing parameters (stronger smoothing to avoid jitter)
        self.steer_alpha = 0.12  # smaller alpha -> more smoothing
        self.throttle_alpha = 0.15
        
        # Steering rate limit (max change per tick)
        self.max_steer_delta = 0.04
        
        # Lane keeping parameters
        self.target_speed_kmh = 15  # Target speed in km/h
        self.max_lane_offset = 2.0  # Maximum allowed lane offset before correction
        
        # Lane-change state machine parameters
        self.state = "DRIVING"  # possible: DRIVING, STOPPING, WAITING_GREEN, LANE_CHANGE
        self.stop_start_time = None
        self.red_stop_distance = 15.0   # meters to start stopping
        self.stop_speed_threshold = 0.5 # m/s to consider vehicle stopped
        
        self.lane_change_active = False
        self.lane_change_waypoints = []
        self.lane_change_start_time = None
        self.lane_change_duration = 2.8
        self.lane_change_finish_threshold = 0.25
        
        # Statistics
        self.collision_count = 0
        self.collision_times = []
        self.deviations = []
        self.controls_history = []
        self.trajectory = []
        
        # Placeholders
        self.client = None
        self.world = None
        self.map = None
        self.vehicle = None
        self.camera = None
        self.collision_sensor = None
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        self.start_time = None
        self.original_settings = None
        
        # Stabilized saved frame EMA for video (optional)
        self._saved_frame_ema = None
        self._ema_alpha = 0.25
        
        # Setup signal handler for graceful shutdown
        signal.signal(signal.SIGINT, self.signal_handler)

    def signal_handler(self, sig, frame):
        print("\nGraceful shutdown initiated...")
        self.running = False
        
    def connect_carla(self):
        """Connect to CARLA simulator"""
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        # self.world = self.client.load_world("Town05")
        self.map = self.world.get_map()
        self.original_settings = self.world.get_settings()
        
        # Set synchronous mode for stable evaluation
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05  # 20 FPS
        self.world.apply_settings(settings)
        
    def spawn_vehicle(self):
        """Spawn vehicle at random location"""
        blueprint_library = self.world.get_blueprint_library()
        
        # Use a standard vehicle
        candidates = blueprint_library.filter("vehicle.*")
        vehicle_bp = None
        for choice in ("vehicle.mercedes.coupe_2020","vehicle.tesla.model3","vehicle.audi.tt"):
            items = blueprint_library.filter(choice)
            if items:
                vehicle_bp = items[0]
                break
        if vehicle_bp is None:
            vehicle_bp = random.choice(candidates)
        
        # Get spawn point
        spawn_points = self.map.get_spawn_points()
        spawn_point = random.choice(spawn_points)
        spawn_point.location.z += 0.5
        
        self.vehicle = self.world.spawn_actor(vehicle_bp, spawn_point)
        print(f"Spawned vehicle at: ({spawn_point.location.x:.2f}, {spawn_point.location.y:.2f})")
        
        return spawn_point.location
        
    def setup_sensors(self):
        """Setup camera and collision sensors"""
        blueprint_library = self.world.get_blueprint_library()
        
        # Camera sensor (mounted near hood for more stable view)
        camera_bp = blueprint_library.find("sensor.camera.rgb")
        camera_bp.set_attribute("image_size_x", "800")
        camera_bp.set_attribute("image_size_y", "600")
        camera_bp.set_attribute("fov", "100")
        
        # mount near hood for less visual jitter
        camera_transform = carla.Transform(carla.Location(x=1.3, y=0, z=1.8), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(camera_bp, camera_transform, attach_to=self.vehicle)
        
        # Collision sensor
        collision_bp = blueprint_library.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(
            collision_bp, carla.Transform(), attach_to=self.vehicle
        )
        
        # Setup callbacks
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        
        self.camera.listen(self.camera_callback)
        self.collision_sensor.listen(self.collision_callback)
        
    def camera_callback(self, image):
        """Process camera images (match CNN script format: RGB)"""
        try:
            array = np.frombuffer(image.raw_data, dtype=np.uint8)
            array = array.reshape((image.height, image.width, 4))[:, :, :3]  # BGRA -> BGR
            array = array[:, :, ::-1]  # BGR -> RGB
            small = cv2.resize(array, (160, 120))
            self.latest_image = small
            self.new_image_available = True
            
            # update EMA for saved frame stabilization
            save_frame = cv2.cvtColor(array, cv2.COLOR_RGB2BGR)
            if self._saved_frame_ema is None:
                self._saved_frame_ema = save_frame.astype(np.float32)
            else:
                self._saved_frame_ema = (self._ema_alpha * save_frame.astype(np.float32) +
                                         (1 - self._ema_alpha) * self._saved_frame_ema)
        except Exception:
            pass
        
    def collision_callback(self, event):
        """Handle collision events"""
        self.collision_count += 1
        collision_time = time.time() - self.start_time
        self.collision_times.append(collision_time)
        print(f"\n*** Collision #{self.collision_count} at {collision_time:.2f}s ***")
        try:
            print(f"    Impact with: {event.other_actor.type_id}")
        except Exception:
            pass
        self.collision_occurred = True

        if self.collision_occurred:
            # Reset speed to 0 to stop the vehicle
            try:
                self.vehicle.set_velocity(carla.Vector3D(0, 0, 0))
            except Exception:
                pass
            
            # Reset PID controller
            self.pid_controller.reset()
            
            # Neutralize steering for recovery
            control = carla.VehicleControl()
            control.steer = 0.0
            control.throttle = 0.0
            control.brake = 1.0
            try:
                self.vehicle.apply_control(control)
            except Exception:
                pass
            
            # Wait before continuing
            time.sleep(1)
            self.collision_occurred = False

    def load_model(self):
        """Load the trained model"""
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Using device: {self.device}")
        
        # Initialize model
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True).to(self.device)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device) 
        
        # Load weights
        if os.path.exists(self.model_path):
            checkpoint = torch.load(self.model_path, map_location=self.device)
            # handle cases where checkpoint is state_dict or a dict with 'state_dict'
            if isinstance(checkpoint, dict) and 'state_dict' in checkpoint:
                state = checkpoint['state_dict']
            else:
                state = checkpoint
            try:
                self.model.load_state_dict(state)
            except RuntimeError:
                # handle possible 'module.' prefix
                new_state = {}
                for k, v in state.items():
                    nk = k[7:] if k.startswith("module.") else k
                    new_state[nk] = v
                self.model.load_state_dict(new_state)
            print(f"Model loaded successfully from {self.model_path}")
        else:
            raise FileNotFoundError(f"Model checkpoint not found at {self.model_path}")
            
        self.model.eval()
        
    # ------------- lane/traffic helpers -------------
    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        """Calculate lateral error from lane center using CARLA waypoints"""
        try:
            # Get the nearest waypoint
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            
            if waypoint is None:
                return 0.0
            
            # Calculate lateral offset
            lane_center = waypoint.transform.location
            
            # Vector from lane center to vehicle
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            
            # Get lane direction (perpendicular to road direction)
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            
            # Calculate cross-track error (positive = right of center, negative = left)
            cross_track_error = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            
            # Calculate heading error (difference between vehicle and lane heading)
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            
            # Normalize heading error to [-pi, pi]
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            
            # Combine cross-track and heading errors
            # Weight heading error more heavily for better stability
            total_error = cross_track_error + 0.5 * heading_error
            
            return total_error
            
        except Exception as e:
            print(f"Error calculating lane error: {e}")
            return 0.0

    def get_forward_vector(self, transform):
        f = transform.get_forward_vector()
        return np.array([f.x, f.y, f.z], dtype=np.float32)

    def nearest_traffic_light_ahead(self):
        """
        Find a traffic light ahead of the vehicle within TRAFFIC_LIGHT_DIST and within forward cone.
        Returns (traffic_light_actor, distance) or (None, None).
        """
        if self.vehicle is None or self.world is None:
            return None, None
        try:
            veh_tf = self.vehicle.get_transform()
            veh_loc = veh_tf.location
            veh_fwd = self.get_forward_vector(veh_tf)
            lights = self.world.get_actors().filter("traffic.traffic_light*")
            best = None
            best_dist = None
            for tl in lights:
                tl_loc = tl.get_transform().location
                vec = np.array([tl_loc.x - veh_loc.x, tl_loc.y - veh_loc.y, tl_loc.z - veh_loc.z], dtype=np.float32)
                dist = float(np.linalg.norm(vec))
                if dist > self.red_stop_distance * 2.0:  # be permissive farther away
                    continue
                if dist < 1e-3:
                    continue
                vec_norm = vec / dist
                dot = float(np.dot(vec_norm, veh_fwd))
                # ahead if dot ~ 0.7..1.0 (stricter to avoid side lights)
                if dot < 0.7:
                    continue
                # prefer closest ahead
                if best is None or dist < best_dist:
                    best = tl
                    best_dist = dist
            return best, best_dist
        except Exception:
            return None, None

    def can_initiate_lane_change(self, waypoint, direction):
        try:
            if direction == "left":
                adj = waypoint.get_left_lane()
            else:
                adj = waypoint.get_right_lane()
            if adj is None:
                return False
            return adj.lane_type == carla.LaneType.Driving
        except Exception:
            return False

    def initiate_lane_change_path(self, direction_prefer=None):
        """
        Build a waypoint path in an adjacent lane (left/right). Sets lane change active if successful.
        direction_prefer: "left" / "right" / None (random)
        """
        try:
            loc = self.vehicle.get_location()
            wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
            if wp is None:
                return False
            directions = ["left", "right"] if direction_prefer is None else [direction_prefer]
            # if preferred fails, try the other
            for dirc in directions + (["left", "right"] if direction_prefer else []):
                if not self.can_initiate_lane_change(wp, dirc):
                    continue
                target = wp.get_left_lane() if dirc == "left" else wp.get_right_lane()
                if target is None:
                    continue
                # step forward a little to avoid sharp cross
                for _ in range(3):
                    nxts = target.next(2.0)
                    if nxts:
                        target = nxts[0]
                waypoints = []
                cur = target
                waypoints.append(cur)
                for _ in range(20):
                    nxts = cur.next(2.0)
                    if not nxts:
                        break
                    cur = nxts[0]
                    waypoints.append(cur)
                if len(waypoints) * 2.0 < 8.0:
                    continue
                # activate lane change
                self.lane_change_active = True
                self.lane_change_waypoints = waypoints
                self.lane_change_start_time = time.time()
                print(f"Initiated lane change to {dirc} lane (path length ~{len(waypoints)*2:.1f}m).")
                return True
            return False
        except Exception:
            traceback.print_exc()
            return False

    def execute_lane_change(self, vehicle_loc, vehicle_rot, current_steer, current_throttle):
        """
        Blend steering/throttle with lane-change target.
        Return (blended_steer, blended_throttle, finished_bool)
        """
        try:
            if not self.lane_change_active or not self.lane_change_waypoints:
                return current_steer, current_throttle, True
            # pick a look-ahead waypoint index
            idx = min(6, len(self.lane_change_waypoints)-1)
            target_wp = self.lane_change_waypoints[idx]
            tloc = target_wp.transform.location
            target_yaw = math.radians(target_wp.transform.rotation.yaw)
            vehicle_yaw = math.radians(vehicle_rot.yaw)
            yaw_diff = target_yaw - vehicle_yaw
            while yaw_diff > math.pi:
                yaw_diff -= 2*math.pi
            while yaw_diff < -math.pi:
                yaw_diff += 2*math.pi
            yaw_error = math.sin(yaw_diff)
            steer_cmd = float(np.clip(1.0 * yaw_error, -0.6, 0.6))
            # lateral error (approx) using target location
            vx = tloc.x - vehicle_loc.x
            vy = tloc.y - vehicle_loc.y
            fwd = vehicle_rot.get_forward_vector()
            lateral_error = vx * fwd.y - vy * fwd.x
            error_weight = float(np.clip(abs(lateral_error) / 1.0, 0.0, 0.9))
            blended = (1 - error_weight) * current_steer + error_weight * steer_cmd
            blended = 0.7 * blended + 0.3 * current_steer  # smoothing
            blended_throttle = min(current_throttle + 0.05, 0.7)
            lane_center_error = abs(self.calculate_lane_error(vehicle_loc, vehicle_rot))
            if lane_center_error < self.lane_change_finish_threshold and abs(yaw_diff) < math.radians(8):
                self.lane_change_active = False
                self.lane_change_waypoints = []
                self.lane_change_start_time = None
                print("✅ Lane change complete (centered & aligned).")
                return blended, blended_throttle, True
            # timeout condition
            if time.time() - (self.lane_change_start_time or time.time()) > (self.lane_change_duration + 4.0):
                print("⚠️ Lane change timeout; aborting.")
                self.lane_change_active = False
                self.lane_change_waypoints = []
                self.lane_change_start_time = None
                return blended, blended_throttle, True
            return blended, blended_throttle, False
        except Exception:
            traceback.print_exc()
            self.lane_change_active = False
            self.lane_change_waypoints = []
            self.lane_change_start_time = None
            return current_steer, current_throttle, True

    # ------------- main loop -------------
    def adaptive_speed_control(self, current_speed_mps, lane_offset, steering_magnitude):
        """Adaptive speed control based on lane position and steering input"""
        target_speed_mps = self.target_speed_kmh / 3.6
        
        # Reduce speed when far from lane center or steering heavily
        offset_factor = max(0.75, 1.0 - abs(lane_offset) / 3.0)  # a bit more conservative
        steering_factor = max(0.75, 1.0 - abs(steering_magnitude) / 0.6)  # reduce speed during turns
        
        adjusted_target = target_speed_mps * offset_factor * steering_factor
        
        # Simple speed controller
        speed_error = adjusted_target - current_speed_mps
        
        if speed_error > 0:
            throttle = min(0.6, 0.36 + speed_error * 0.28)
        else:
            throttle = max(0.12, 0.36 + speed_error * 0.18)
            
        return float(np.clip(throttle, 0.0, 0.9))

    def run_evaluation(self):
        """Main evaluation loop with enhanced lane keeping and safe stopping for red lights"""
        print("\n" + "="*60)
        print("ENHANCED CLOSED-LOOP EVALUATION STARTED (stable variant)")
        print("="*60)
        print("Press Ctrl+C to stop evaluation gracefully\n")
        
        # Initialize
        self.connect_carla()
        start_location = self.spawn_vehicle()
        self.setup_sensors()
        self.load_model()
        
        # Setup spectator
        spectator = self.world.get_spectator()
        
        # Initialize control
        control = carla.VehicleControl()
        control.manual_gear_shift = False
        
        # Wait for initial frames
        print("Warming up sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)
        
        # Initialize variables
        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log_time = time.time()
        
        # Open log file
        log_filename = f"enhanced_evaluation_log_{int(self.start_time)}.csv"
        log_file = open(log_filename, "w")
        log_file.write("timestamp,frame,x,y,z,yaw,speed_mps,speed_kmh,model_steer,final_steer,throttle,lane_error,distance_traveled,state\n")
        
        # Image directory for video frames (using CNN-style naming)
        image_dir = f"enhanced_eval_images_{int(self.start_time)}"
        os.makedirs(image_dir, exist_ok=True)
        
        print("Starting enhanced autonomous driving...\n")
        
        try:
            while self.running and not self.collision_occurred:
                # Update frame stack
                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False
                
                # detect traffic light ahead
                tl_actor, tl_dist = self.nearest_traffic_light_ahead()
                tl_state = None
                if tl_actor is not None:
                    try:
                        tl_state = tl_actor.get_state()
                    except Exception:
                        tl_state = None

                # State transitions: RED -> STOPPING
                if tl_actor is not None and tl_state is not None and tl_state == carla.TrafficLightState.Red:
                    if tl_dist is not None and tl_dist <= self.red_stop_distance:
                        if self.state == "DRIVING":
                            print(f"Red light detected at {tl_dist:.1f}m -> STOPPING")
                            self.state = "STOPPING"
                            self.stop_start_time = None
                # If waiting and green -> plan lane change
                if self.state == "WAITING_GREEN" and tl_actor is not None and tl_state == carla.TrafficLightState.Green:
                    print("Green detected while waiting -> initiating lane-change path")
                    ok = self.initiate_lane_change_path(direction_prefer=None)
                    if ok:
                        self.state = "LANE_CHANGE"
                    else:
                        print("No adjacent lane available; resuming DRIVING")
                        self.state = "DRIVING"

                # Process when we have frames
                if len(frame_stack) == 3:
                    # Get vehicle state
                    transform = self.vehicle.get_transform()
                    location = transform.location
                    rotation = transform.rotation
                    
                    # Calculate speed
                    velocity = self.vehicle.get_velocity()
                    speed_mps = np.sqrt(velocity.x**2 + velocity.y**2 + velocity.z**2)
                    speed_kmh = speed_mps * 3.6
                    
                    # Calculate lane error
                    lane_error = self.calculate_lane_error(location, rotation)
                    
                    # Preprocess frames for model
                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    
                    if input_tensor is None:
                        continue
                    
                    # Prepare speed input
                    speed_normalized = min(speed_mps / 30.0, 1.0)
                    speed_tensor = torch.tensor([[speed_normalized]], dtype=torch.float32, device=self.device)
                    
                    # Model inference
                    with torch.no_grad():
                        output = self.model(input_tensor, aux=speed_tensor)
                    
                    # Extract model outputs
                    model_steer, model_throttle, model_brake = output[0].cpu().numpy()
                    
                    # Enhanced steering control combining model output and lane keeping
                    current_time = time.time()
                    
                    # Use PID controller to correct lane error
                    pid_correction = self.pid_controller.update(lane_error, current_time)
                    
                    # Combine model steering with PID correction
                    # Give more weight to PID when lane error is large (but limited)
                    error_weight = min(abs(lane_error) / 1.0, 0.65)  # max 65% PID weight for stability
                    combined_steer = (1 - error_weight) * model_steer + error_weight * pid_correction
                    
                    # Apply steering smoothing (low-pass)
                    final_steer_candidate = (self.steer_alpha * combined_steer +
                                             (1 - self.steer_alpha) * self.steer_smooth)
                    
                    # limit steering rate to avoid oscillation
                    delta = float(final_steer_candidate - self.steer_smooth)
                    delta = np.clip(delta, -self.max_steer_delta, self.max_steer_delta)
                    final_steer = float(self.steer_smooth + delta)
                    
                    # Clamp steering to safe limits
                    final_steer = float(np.clip(final_steer, -0.6, 0.6))
                    # update smooth state
                    self.steer_smooth = final_steer
                    
                    # Adaptive throttle control
                    adaptive_throttle = self.adaptive_speed_control(speed_mps, lane_error, abs(final_steer))
                    
                    # Apply throttle smoothing
                    final_throttle = (self.throttle_alpha * adaptive_throttle +
                                      (1 - self.throttle_alpha) * self.throttle_smooth)
                    self.throttle_smooth = final_throttle
                    
                    # Ensure minimum throttle to prevent stalling (but keep small)
                    final_throttle = max(final_throttle, 0.12)
                    final_throttle = min(final_throttle, 0.8)
                    
                    # Default brake
                    final_brake = 0.0

                    # State machine actions
                    if self.state == "STOPPING":
                        # initiate stopping behavior; more aggressive braking as we approach
                        if self.stop_start_time is None:
                            self.stop_start_time = time.time()
                            self.pid_controller.reset()
                        # compute brake proportional to distance & speed
                        if tl_dist is None:
                            brake_val = min(1.0, 0.6 + (speed_mps / 10.0))
                        else:
                            approach = max(0.0, (self.red_stop_distance - tl_dist) / max(1.0, self.red_stop_distance))
                            brake_val = np.clip(0.3 + approach * 0.9 + (speed_mps / 20.0), 0.0, 1.0)
                        final_throttle = 0.0
                        final_brake = float(brake_val)
                        # keep steering small while stopping
                        final_steer = float(np.clip(final_steer * 0.5, -0.35, 0.35))
                        # if stopped -> WAITING_GREEN
                        if speed_mps < self.stop_speed_threshold:
                            print("Vehicle stopped at red; waiting for green.")
                            self.state = "WAITING_GREEN"
                    elif self.state == "WAITING_GREEN":
                        # hold stopped
                        final_throttle = 0.0
                        final_brake = 1.0
                        final_steer = 0.0
                    elif self.state == "LANE_CHANGE":
                        # perform lane change blending
                        final_brake = 0.0
                        final_steer, final_throttle, done = self.execute_lane_change(location, rotation, final_steer, final_throttle)
                        if done:
                            self.state = "DRIVING"
                    else:  # DRIVING normal
                        # keep driving with model+pid blended control
                        final_brake = 0.0

                    # Apply controls
                    control.steer = float(final_steer)
                    control.throttle = float(final_throttle)
                    control.brake = float(final_brake)
                    control.hand_brake = False
                    try:
                        self.vehicle.apply_control(control)
                    except Exception:
                        pass
                    
                    # Save stabilized frame (use EMA if available)
                    try:
                        if self._saved_frame_ema is not None:
                            save_img = np.clip(self._saved_frame_ema, 0, 255).astype(np.uint8)
                            frame_filename = os.path.join(image_dir, f"{frame_count:06d}.png")
                            cv2.imwrite(frame_filename, save_img)
                        else:
                            frame_filename = os.path.join(image_dir, f"{frame_count:06d}.png")
                            cv2.imwrite(frame_filename, cv2.cvtColor(self.latest_image, cv2.COLOR_RGB2BGR))
                    except Exception as e:
                        print(f"Warning: failed to save frame {frame_count}: {e}")
                    
                    # Calculate metrics
                    distance_traveled = location.distance(start_location)
                    
                    # Store data
                    self.deviations.append(abs(lane_error))
                    self.controls_history.append({
                        'frame': frame_count,
                        'model_steer': model_steer,
                        'final_steer': final_steer,
                        'throttle': final_throttle,
                        'speed': speed_mps,
                        'lane_error': lane_error,
                        'state': self.state
                    })
                    self.trajectory.append({
                        'x': location.x,
                        'y': location.y,
                        'z': location.z,
                        'yaw': rotation.yaw
                    })
                    
                    # Log to file
                    timestamp = time.time() - self.start_time
                    log_file.write(
                        f"{timestamp:.3f},{frame_count},{location.x:.2f},{location.y:.2f},{location.z:.2f},"
                        f"{rotation.yaw:.2f},{speed_mps:.3f},{speed_kmh:.1f},{model_steer:.4f},"
                        f"{final_steer:.4f},{final_throttle:.4f},{lane_error:.4f},{distance_traveled:.2f},{self.state}\n"
                    )
                    log_file.flush()
                    
                    # Update spectator view (smoothed)
                    try:
                        transform = self.vehicle.get_transform()
                        loc_s = transform.location
                        rot_s = transform.rotation
                        spectator.set_transform(carla.Transform(loc_s + transform.get_forward_vector()*-6 + carla.Location(z=3),
                                                                carla.Rotation(pitch=-15, yaw=rot_s.yaw)))
                    except Exception:
                        pass
                    
                    # Print status every second
                    current_time = time.time()
                    if current_time - last_log_time >= 1.0:
                        print(f"[{timestamp:6.1f}s] Frame {frame_count:5d} | "
                              f"Speed: {speed_kmh:5.1f} km/h | "
                              f"Distance: {distance_traveled:6.1f}m | "
                              f"Lane error: {lane_error:+.3f}m | "
                              f"Steering (M/F): {model_steer:+.3f}/{final_steer:+.3f} | STATE: {self.state}")
                        last_log_time = current_time
                    
                    frame_count += 1
                
                # Tick the simulation
                self.world.tick()

        except Exception as e:
            print(f"\nError during evaluation: {e}")
            traceback.print_exc()
            
        finally:
            # Close log file
            try:
                log_file.close()
            except Exception:
                pass
            
            # Create video from saved frames (exact CNN-style logic)
            try:
                images = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
                if images:
                    out_video = f"enhanced_evaluation_video_{int(self.start_time)}.mp4"
                    with imageio.get_writer(out_video, fps=30) as writer:
                        for img in images:
                            writer.append_data(imageio.imread(img))
                    print(f"Enhanced evaluation video saved as {out_video}")
            except Exception as e:
                print(f"Warning: failed to create video: {e}")
            finally:
                try:
                    gc.collect()
                except:
                    pass
            
            # Print final statistics
            self.print_statistics(frame_count)
            
            # Cleanup
            self.cleanup()

    def print_statistics(self, frame_count):
        """Print enhanced evaluation statistics"""
        elapsed_time = time.time() - self.start_time
        
        print("\n" + "="*60)
        print("ENHANCED EVALUATION STATISTICS")
        print("="*60)
        
        print(f"Total frames processed: {frame_count}")
        print(f"Total time: {elapsed_time:.2f} seconds")
        print(f"Average FPS: {frame_count/elapsed_time:.2f}")
        print(f"Total collisions: {self.collision_count}")
        
        if len(self.deviations) > 0:
            print(f"\nLane keeping performance:")
            print(f"  Average offset: {np.mean(self.deviations):.3f} meters")
            print(f"  Std deviation: {np.std(self.deviations):.3f} meters")
            print(f"  Max offset: {np.max(self.deviations):.3f} meters")
            print(f"  95th percentile: {np.percentile(self.deviations, 95):.3f} meters")
            
            # Calculate percentage of time within lane bounds
            within_lane = np.sum(np.array(self.deviations) <= 1.5) / len(self.deviations) * 100
            print(f"  Time within 1.5m of lane center: {within_lane:.1f}%")
            
        if len(self.trajectory) > 0:
            first_pos = self.trajectory[0]
            last_pos = self.trajectory[-1]
            total_distance = np.sqrt(
                (last_pos['x'] - first_pos['x'])**2 + 
                (last_pos['y'] - first_pos['y'])**2
            )
            print(f"\nTotal distance traveled: {total_distance:.2f} meters")
            
        if len(self.controls_history) > 0:
            all_speeds = [c['speed'] for c in self.controls_history]
            all_lane_errors = [c['lane_error'] for c in self.controls_history]
            
            print(f"\nSpeed statistics:")
            print(f"  Average speed: {np.mean(all_speeds)*3.6:.2f} km/h")
            print(f"  Max speed: {np.max(all_speeds)*3.6:.2f} km/h")
            
            print(f"\nLane error statistics:")
            print(f"  Mean absolute error: {np.mean(np.abs(all_lane_errors)):.3f} meters")
            print(f"  RMS error: {np.sqrt(np.mean(np.array(all_lane_errors)**2)):.3f} meters")
                
        print("="*60)
        
    def cleanup(self):
        """Clean up CARLA actors and reset settings"""
        print("Cleaning up...")
        
        # Reset to asynchronous mode
        if hasattr(self, 'world') and self.original_settings is not None:
            try:
                self.world.apply_settings(self.original_settings)
            except Exception:
                pass
        
        # Destroy actors
        actors_to_destroy = []
        if hasattr(self, 'camera'):
            try:
                self.camera.stop()
            except:
                pass
            actors_to_destroy.append(self.camera)
        if hasattr(self, 'collision_sensor'):
            try:
                self.collision_sensor.stop()
            except:
                pass
            actors_to_destroy.append(self.collision_sensor)
        if hasattr(self, 'vehicle'):
            actors_to_destroy.append(self.vehicle)
            
        for actor in actors_to_destroy:
            if actor is not None:
                try:
                    actor.destroy()
                except:
                    pass
                
        try:
            gc.collect()
        except Exception:
            pass
                
        print("Cleanup complete.")


def main():
    # Path to your trained model
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    
    # Create evaluator and run
    evaluator = ClosedLoopEvaluator(model_path=MODEL_PATH)
    evaluator.run_evaluation()


if __name__ == "__main__":
    main()



ENHANCED CLOSED-LOOP EVALUATION STARTED (stable variant)
Press Ctrl+C to stop evaluation gracefully

Spawned vehicle at: (-103.18, -14.43)
Using device: cuda


/tmp/ipykernel_2106093/2195924447.py:369: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(self.model_path, map_location=self.device)
[ERROR:0@1.470] gl

Model loaded successfully from /home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth
Warming up sensors...
Starting enhanced autonomous driving...

[   1.0s] Frame    49 | Speed:  12.4 km/h | Distance:    2.0m | Lane error: +0.090m | Steering (M/F): +0.006/+0.015 | STATE: DRIVING
[   2.0s] Frame   106 | Speed:  12.7 km/h | Distance:   12.0m | Lane error: -0.076m | Steering (M/F): +0.004/+0.004 | STATE: DRIVING
[   3.0s] Frame   165 | Speed:   7.7 km/h | Distance:   20.9m | Lane error: +0.655m | Steering (M/F): -0.002/+0.196 | STATE: DRIVING
[   4.0s] Frame   223 | Speed:  13.1 km/h | Distance:   28.5m | Lane error: -0.105m | Steering (M/F): +0.011/+0.005 | STATE: DRIVING
[   5.0s] Frame   282 | Speed:   9.5 km/h | Distance:   38.1m | Lane error: +0.647m | Steering (M/F): +0.015/+0.169 | STATE: DRIVING
[   6.1s] Frame   339 | Speed:  10.1 km/h | Distance:   43.4m | Lane error: +0.273m | Steering (M/F): +0.006/+0.055 | STATE: DRIVING
[   7.1s] Frame   395 | Speed

In [ ]:
import carla
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import cv2
import gc
import imageio.v2 as imageio
import signal
import math
import traceback

# Environment setup (limits threads for reproducibility)
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


# ----------------- Utilities -----------------
def preprocess_image_stack(frame_stack, device):
    """Preprocess 3 RGB frames -> (1, 9, 120, 160) tensor on device."""
    if len(frame_stack) != 3:
        return None
    processed_frames = []
    for frame in frame_stack:
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        frame = frame.astype(np.float32) / 255.0
        frame = (frame - 0.5) / 0.5
        processed_frames.append(frame)
    stacked = np.concatenate(processed_frames, axis=2)  # (120,160,9)
    stacked = np.transpose(stacked, (2, 0, 1))  # (9,120,160)
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)
    return tensor.to(device)


class ImprovedPIDController:
    """Stronger PID for better lane-keeping."""
    def __init__(self):
        # INCREASED gains for better lane tracking
        self.Kp = 0.85  # Increased from 0.45
        self.Ki = 0.02  # Increased from 0.005
        self.Kd = 0.15  # Increased from 0.04
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        self.integral_limit = 0.8  # Increased from 0.5
        self.output_limit = 0.7  # Increased from 0.4

    def update(self, error, current_time):
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05
        else:
            dt = max(current_time - self.prev_time, 1e-3)
            self.prev_time = current_time
        P = self.Kp * error
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        D = self.Kd * derivative
        self.prev_error = error
        output = P + I + D
        return float(np.clip(output, -self.output_limit, self.output_limit))

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


# ----------------- Main evaluator -----------------
class ClosedLoopEvaluator:
    def __init__(self, model_path, host="localhost", port=2000, detect_yellow=False, traffic_light_distance=30.0):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        # controllers & smoothing
        self.pid_controller = ImprovedPIDController()
        self.steer_smooth = 0.0
        self.throttle_smooth = 0.3
        # ADJUSTED smoothing for better responsiveness
        self.steer_alpha = 0.25  # Increased from 0.12
        self.throttle_alpha = 0.20  # Increased from 0.15

        # steering rate limit
        self.max_steer_delta = 0.08  # Increased from 0.04

        # speed / thresholds
        self.target_speed_kmh = 18.0  # Increased from 15.0
        self.max_lane_offset = 2.0

        # lane-change state machine
        self.lane_change_active = False
        self.lane_change_waypoints = []
        self.lane_change_start_time = None
        self.lane_change_duration = 3.5  # Increased from 2.8
        self.lane_change_finish_threshold = 0.35  # Relaxed from 0.22
        self.lane_change_cooldown = 0.0  # Track when last lane change finished
        self.min_lane_change_interval = 3.0  # Minimum seconds between lane changes

        # traffic-light settings
        self.DETECT_YELLOW = detect_yellow
        self.TRAFFIC_LIGHT_DIST = traffic_light_distance

        # red/green behavior states
        self.state = "DRIVING"  # other: 'STOPPING', 'WAITING_GREEN', 'LANE_CHANGE'
        self.stop_start_time = None
        self.stop_speed_threshold = 0.5
        self.red_stop_distance = 18.0  # Increased from 15.0
        self.green_resume_delay = 0.3

        # stats & actors
        self.collision_count = 0
        self.deviations = []
        self.controls_history = []
        self.trajectory = []
        self.lane_changes_completed = 0

        # placeholders
        self.client = None
        self.world = None
        self.map = None
        self.vehicle = None
        self.camera = None
        self.collision_sensor = None
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        self.start_time = None
        self.original_settings = None

        # image stabilization
        self._saved_frame_ema = None
        self._ema_alpha = 0.25

        # graceful shutdown
        signal.signal(signal.SIGINT, self.signal_handler)

    def signal_handler(self, sig, frame):
        print("\nGraceful shutdown requested.")
        self.running = False

    def connect_carla(self):
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        self.original_settings = self.world.get_settings()
        self.map = self.world.get_map()

        # enable synchronous mode
        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)

    def spawn_vehicle(self):
        bp_lib = self.world.get_blueprint_library()
        candidates = bp_lib.filter("vehicle.*")
        if not candidates:
            raise RuntimeError("No vehicle blueprints found.")
        vehicle_bp = None
        for name in ("vehicle.tesla.model3", "vehicle.mercedes.coupe_2020", "vehicle.audi.tt"):
            items = bp_lib.filter(name)
            if items:
                vehicle_bp = items[0]
                break
        if vehicle_bp is None:
            vehicle_bp = random.choice(candidates)
        spawn_points = self.map.get_spawn_points()
        if not spawn_points:
            raise RuntimeError("No spawn points available.")
        sp = random.choice(spawn_points)
        sp.location.z += 0.5
        self.vehicle = self.world.spawn_actor(vehicle_bp, sp)
        print(f"Spawned vehicle at ({sp.location.x:.2f}, {sp.location.y:.2f})")
        return sp.location

    def setup_sensors(self):
        bp_lib = self.world.get_blueprint_library()
        cam_bp = bp_lib.find("sensor.camera.rgb")
        cam_bp.set_attribute("image_size_x", "800")
        cam_bp.set_attribute("image_size_y", "600")
        cam_bp.set_attribute("fov", "100")
        cam_transform = carla.Transform(carla.Location(x=1.3, y=0.0, z=1.8), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)

        col_bp = bp_lib.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(col_bp, carla.Transform(), attach_to=self.vehicle)

        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False

        self.camera.listen(self.camera_callback)
        self.collision_sensor.listen(self.collision_callback)

    def camera_callback(self, image):
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]  # BGR -> RGB
            small = cv2.resize(arr, (160, 120))
            self.latest_image = small
            self.new_image_available = True

            save_frame = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
            if self._saved_frame_ema is None:
                self._saved_frame_ema = save_frame.astype(np.float32)
            else:
                self._saved_frame_ema = (self._ema_alpha * save_frame.astype(np.float32) +
                                         (1 - self._ema_alpha) * self._saved_frame_ema)
        except Exception:
            pass

    def collision_callback(self, event):
        self.collision_count += 1
        t = time.time() - (self.start_time or time.time())
        print(f"\n*** Collision #{self.collision_count} at {t:.2f}s ***")
        try:
            print("    Impact with:", event.other_actor.type_id)
        except Exception:
            pass
        self.collision_occurred = True
        try:
            self.vehicle.set_velocity(carla.Vector3D(0, 0, 0))
        except Exception:
            pass
        self.pid_controller.reset()
        try:
            ctrl = carla.VehicleControl()
            ctrl.steer = 0.0
            ctrl.throttle = 0.0
            ctrl.brake = 1.0
            self.vehicle.apply_control(ctrl)
        except Exception:
            pass
        time.sleep(1.0)
        self.collision_occurred = False

    def load_model(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Using device:", self.device)
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device)
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model not found: {self.model_path}")
        loaded = torch.load(self.model_path, map_location=self.device)
        if isinstance(loaded, dict) and 'state_dict' in loaded:
            state = loaded['state_dict']
        else:
            state = loaded
        try:
            try:
                self.model.load_state_dict(state)
            except RuntimeError:
                new_state = {}
                for k, v in state.items():
                    nk = k[7:] if k.startswith("module.") else k
                    new_state[nk] = v
                self.model.load_state_dict(new_state)
            print("Model loaded:", self.model_path)
        except Exception as e:
            raise RuntimeError("Failed to load model state dict: " + str(e))
        self.model.eval()

    # ---------------- helpers ----------------
    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        """Cross-track + heading error to lane center (meters / radians weighted)"""
        try:
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            if waypoint is None:
                return 0.0
            lane_center = waypoint.transform.location
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            cross_track_error = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            total_error = cross_track_error + 0.6 * heading_error  # Increased heading weight
            return total_error
        except Exception:
            return 0.0

    def get_forward_vector(self, transform):
        f = transform.get_forward_vector()
        return np.array([f.x, f.y, f.z], dtype=np.float32)

    def nearest_traffic_light_ahead(self):
        """Return (traffic_light_actor, distance) if one exists ahead."""
        if self.vehicle is None or self.world is None:
            return None, None
        try:
            veh_tf = self.vehicle.get_transform()
            veh_loc = veh_tf.location
            veh_fwd = self.get_forward_vector(veh_tf)
            lights = self.world.get_actors().filter("traffic.traffic_light*")
            best = None
            best_dist = None
            for tl in lights:
                tl_loc = tl.get_transform().location
                vec = np.array([tl_loc.x - veh_loc.x, tl_loc.y - veh_loc.y, tl_loc.z - veh_loc.z], dtype=np.float32)
                dist = float(np.linalg.norm(vec))
                if dist > self.TRAFFIC_LIGHT_DIST:
                    continue
                if dist < 1e-3:
                    continue
                vec_norm = vec / dist
                dot = float(np.dot(vec_norm, veh_fwd))
                if dot < 0.7:
                    continue
                if best is None or dist < best_dist:
                    best = tl
                    best_dist = dist
            return best, best_dist
        except Exception:
            return None, None

    def can_initiate_lane_change(self, waypoint, direction):
        """Check if lane change is possible in given direction."""
        try:
            if direction == "left":
                adj = waypoint.get_left_lane()
            else:
                adj = waypoint.get_right_lane()
            
            if adj is None:
                return False
            
            # Check if it's a driving lane
            if adj.lane_type != carla.LaneType.Driving:
                return False
            
            # CRITICAL FIX: Check if lanes are in same direction (not opposite traffic)
            # Compare lane IDs - opposite direction lanes typically have opposite signs
            if (waypoint.lane_id > 0) != (adj.lane_id > 0):
                return False
                
            return True
        except Exception:
            return False

    def initiate_lane_change_path(self, direction_prefer=None):
        """Build a waypoint path for lane change. Returns True if successful."""
        try:
            # Check cooldown
            if time.time() - self.lane_change_cooldown < self.min_lane_change_interval:
                print(f"Lane change on cooldown (wait {self.min_lane_change_interval}s between changes)")
                return False
            
            loc = self.vehicle.get_location()
            wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
            if wp is None:
                print("Cannot get current waypoint for lane change")
                return False
            
            # Try preferred direction first, then alternative
            if direction_prefer:
                directions = [direction_prefer, "right" if direction_prefer == "left" else "left"]
            else:
                directions = ["left", "right"]
            
            for dirc in directions:
                print(f"Attempting lane change to {dirc}...")
                
                if not self.can_initiate_lane_change(wp, dirc):
                    print(f"  ✗ {dirc.capitalize()} lane not available or unsafe")
                    continue
                
                # Get target lane
                target = wp.get_left_lane() if dirc == "left" else wp.get_right_lane()
                if target is None:
                    continue
                
                # Build longer path for smoother transition
                waypoints = []
                cur = target
                for _ in range(30):  # Increased from 20
                    waypoints.append(cur)
                    nxts = cur.next(2.5)  # Increased spacing from 2.0
                    if not nxts:
                        break
                    cur = nxts[0]
                
                if len(waypoints) < 10:
                    print(f"  ✗ Path too short ({len(waypoints)} waypoints)")
                    continue
                
                # Success! Activate lane change
                self.lane_change_active = True
                self.lane_change_waypoints = waypoints
                self.lane_change_start_time = time.time()
                print(f"  ✓ Lane change to {dirc} initiated! Path: {len(waypoints)} waypoints (~{len(waypoints)*2.5:.1f}m)")
                return True
            
            print("  ✗ No valid lane change path found")
            return False
            
        except Exception as e:
            print(f"Lane change initiation error: {e}")
            traceback.print_exc()
            return False

    def execute_lane_change(self, vehicle_loc, vehicle_rot, current_steer, current_throttle):
        """Execute lane change maneuver. Returns (steer, throttle, finished)."""
        try:
            if not self.lane_change_active or not self.lane_change_waypoints:
                return current_steer, current_throttle, True
            
            # Look ahead more aggressively
            idx = min(10, len(self.lane_change_waypoints)-1)  # Increased from 6
            target_wp = self.lane_change_waypoints[idx]
            tloc = target_wp.transform.location
            target_yaw = math.radians(target_wp.transform.rotation.yaw)
            
            # Calculate heading error
            vehicle_yaw = math.radians(vehicle_rot.yaw)
            yaw_diff = target_yaw - vehicle_yaw
            while yaw_diff > math.pi:
                yaw_diff -= 2*math.pi
            while yaw_diff < -math.pi:
                yaw_diff += 2*math.pi
            
            # Calculate lateral error
            dx = tloc.x - vehicle_loc.x
            dy = tloc.y - vehicle_loc.y
            dist_to_target = math.sqrt(dx**2 + dy**2)
            
            # Compute steering command
            fwd = vehicle_rot.get_forward_vector()
            lateral_error = dx * fwd.y - dy * fwd.x
            
            # STRONGER lane change steering
            steer_cmd = float(np.clip(
                1.2 * math.sin(yaw_diff) + 0.5 * np.tanh(lateral_error / 2.0),  # Increased gains
                -0.8, 0.8
            ))
            
            # Blend with current steering (more aggressive)
            error_weight = 0.75  # Increased from variable weight
            blended = error_weight * steer_cmd + (1 - error_weight) * current_steer
            
            # Maintain speed during lane change
            blended_throttle = max(current_throttle, 0.45)  # Ensure minimum speed
            
            # Check completion with relaxed thresholds
            current_lane_error = abs(self.calculate_lane_error(vehicle_loc, vehicle_rot))
            
            # Finish conditions: close to lane center OR aligned with lane OR timeout
            elapsed = time.time() - (self.lane_change_start_time or time.time())
            
            if current_lane_error < self.lane_change_finish_threshold and abs(yaw_diff) < math.radians(10):
                self.lane_change_active = False
                self.lane_change_waypoints = []
                self.lane_change_start_time = None
                self.lane_change_cooldown = time.time()
                self.lane_changes_completed += 1
                print(f"✅ Lane change #{self.lane_changes_completed} complete! (centered & aligned)")
                return blended, blended_throttle, True
            
            if elapsed > self.lane_change_duration + 5.0:  # Extended timeout
                self.lane_change_active = False
                self.lane_change_waypoints = []
                self.lane_change_start_time = None
                self.lane_change_cooldown = time.time()
                print("⚠️ Lane change timeout; completing anyway")
                return blended, blended_throttle, True
            
            return blended, blended_throttle, False
            
        except Exception as e:
            print(f"Lane change execution error: {e}")
            traceback.print_exc()
            self.lane_change_active = False
            self.lane_change_waypoints = []
            self.lane_change_start_time = None
            return current_steer, current_throttle, True

    def adaptive_speed_control(self, current_speed_mps, lane_offset, steering_magnitude):
        """Adaptive throttle during normal driving."""
        target_speed_mps = self.target_speed_kmh / 3.6
        offset_factor = max(0.75, 1.0 - abs(lane_offset) / 3.0)
        steering_factor = max(0.80, 1.0 - abs(steering_magnitude) / 0.6)
        adjusted_target = target_speed_mps * offset_factor * steering_factor
        speed_error = adjusted_target - current_speed_mps
        if speed_error > 0:
            throttle = min(0.7, 0.42 + speed_error * 0.30)
        else:
            throttle = max(0.0, 0.38 + speed_error * 0.25)
        return float(np.clip(throttle, 0.0, 0.9))

    # ---------------- main loop ----------------
    def run_evaluation(self):
        print("\n" + "="*60)
        print("STARTING: CNN + Traffic Light Detection + Smart Lane Changes")
        print("="*60)
        try:
            self.connect_carla()
            start_location = self.spawn_vehicle()
            self.setup_sensors()
            self.load_model()
        except Exception as e:
            print("Init failed:", e)
            traceback.print_exc()
            return

        spectator = self.world.get_spectator()
        ctrl = carla.VehicleControl()
        ctrl.manual_gear_shift = False

        print("Warming sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)

        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log = time.time()
        last_lane_change_attempt = 0.0

        log_filename = f"eval_log_{int(self.start_time)}.csv"
        log_file = open(log_filename, "w")
        log_file.write("timestamp,frame,x,y,z,yaw,speed_mps,speed_kmh,model_steer,final_steer,throttle,lane_error,state\n")

        image_dir = f"eval_images_{int(self.start_time)}"
        os.makedirs(image_dir, exist_ok=True)

        try:
            while self.running:
                self.world.tick()

                # Spectator follow
                try:
                    tf = self.vehicle.get_transform()
                    sp_loc = tf.location + tf.get_forward_vector() * -6 + carla.Location(z=3.0)
                    sp_rot = carla.Rotation(pitch=-12, yaw=tf.rotation.yaw)
                    spectator.set_transform(carla.Transform(sp_loc, sp_rot))
                except Exception:
                    pass

                # Collect frames
                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False

                # Traffic light detection
                tl_actor, tl_dist = self.nearest_traffic_light_ahead()
                tl_state = None
                if tl_actor is not None:
                    try:
                        tl_state = tl_actor.get_state()
                    except Exception:
                        tl_state = None

                # STATE TRANSITIONS
                # Red light -> STOPPING
                if (tl_actor is not None and tl_state == carla.TrafficLightState.Red and
                    tl_dist is not None and tl_dist <= self.red_stop_distance and 
                    self.state == "DRIVING"):
                    print(f"🔴 Red light at {tl_dist:.1f}m -> STOPPING")
                    self.state = "STOPPING"
                    self.stop_start_time = None

                # Green light after waiting -> LANE_CHANGE
                if (self.state == "WAITING_GREEN" and tl_actor is not None and 
                    tl_state == carla.TrafficLightState.Green):
                    print("🟢 Green light -> Attempting lane change...")
                    time.sleep(self.green_resume_delay)
                    ok = self.initiate_lane_change_path(direction_prefer=None)
                    if ok:
                        self.state = "LANE_CHANGE"
                    else:
                        print("   No lane change possible, resuming driving")
                        self.state = "DRIVING"

                # RANDOM LANE CHANGES during normal driving (every 8-15 seconds)
                if (self.state == "DRIVING" and not self.lane_change_active):
                    elapsed_since_last = time.time() - last_lane_change_attempt
                    # Random interval between attempts
                    if elapsed_since_last > random.uniform(8.0, 15.0):
                        print("🔀 Random lane change attempt...")
                        ok = self.initiate_lane_change_path(direction_prefer=None)
                        if ok:
                            self.state = "LANE_CHANGE"
                        last_lane_change_attempt = time.time()

                # Process with 3 frames
                if len(frame_stack) == 3:
                    tf = self.vehicle.get_transform()
                    loc = tf.location
                    rot = tf.rotation
                    vel = self.vehicle.get_velocity()
                    speed_mps = math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
                    speed_kmh = speed_mps * 3.6

                    lane_error = self.calculate_lane_error(loc, rot)

                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    if input_tensor is None:
                        continue

                    # Speed input for model
                    speed_norm = min(speed_mps / 30.0, 1.0)
                    speed_tensor = torch.tensor([[speed_norm]], dtype=torch.float32, device=self.device)

                    # Model inference
                    with torch.no_grad():
                        output = self.model(input_tensor, aux=speed_tensor)
                    model_steer, model_throttle, model_brake = output[0].cpu().numpy()

                    now = time.time()
                    pid_corr = self.pid_controller.update(lane_error, now)

                    # IMPROVED BLENDING: Give more weight to PID for better lane-keeping
                    err_w = min(abs(lane_error) / 0.8, 0.85)  # Increased max PID weight to 0.85
                    combined = (1 - err_w) * model_steer + err_w * pid_corr

                    # Steering smoothing
                    final_steer_candidate = (self.steer_alpha * combined + (1 - self.steer_alpha) * self.steer_smooth)

                    # Apply rate limiting
                    delta = float(final_steer_candidate - self.steer_smooth)
                    delta = np.clip(delta, -self.max_steer_delta, self.max_steer_delta)
                    final_steer = float(self.steer_smooth + delta)
                    final_steer = float(np.clip(final_steer, -0.7, 0.7))
                    self.steer_smooth = final_steer

                    # Throttle smoothing
                    adaptive_throttle = self.adaptive_speed_control(speed_mps, lane_error, abs(final_steer))
                    final_throttle_candidate = (self.throttle_alpha * adaptive_throttle + (1 - self.throttle_alpha) * self.throttle_smooth)
                    final_throttle = float(np.clip(final_throttle_candidate, 0.0, 0.9))
                    self.throttle_smooth = final_throttle

                    # Default brake off
                    final_brake = 0.0

                    # STATE MACHINE EXECUTION
                    if self.state == "STOPPING":
                        # Smooth braking to stop at red light
                        if self.stop_start_time is None:
                            self.stop_start_time = time.time()
                            self.pid_controller.reset()
                        
                        if tl_dist is None:
                            brake_val = min(1.0, 0.6 + (speed_mps / 10.0))
                        else:
                            approach = max(0.05, (self.red_stop_distance - tl_dist) / max(1.0, self.red_stop_distance))
                            brake_val = np.clip(0.3 + approach * 0.9 + (speed_mps / 20.0), 0.0, 1.0)
                        
                        final_throttle = 0.0
                        final_brake = float(brake_val)
                        final_steer = float(np.clip(final_steer * 0.5, -0.3, 0.3))
                        
                        if speed_mps < self.stop_speed_threshold:
                            print("🛑 Stopped at red light, waiting for green...")
                            self.state = "WAITING_GREEN"
                    
                    elif self.state == "WAITING_GREEN":
                        # Hold stopped
                        final_throttle = 0.0
                        final_brake = 1.0
                        final_steer = 0.0
                    
                    elif self.state == "LANE_CHANGE":
                        # Execute lane change maneuver
                        final_brake = 0.0
                        final_steer, final_throttle, done = self.execute_lane_change(loc, rot, final_steer, final_throttle)
                        if done:
                            print("   Resuming normal driving")
                            self.state = "DRIVING"
                    
                    else:  # DRIVING
                        # Normal driving - already computed above
                        final_brake = 0.0

                    # Apply control to vehicle
                    ctrl.steer = float(final_steer)
                    ctrl.throttle = float(final_throttle)
                    ctrl.brake = float(final_brake)
                    ctrl.hand_brake = False
                    try:
                        self.vehicle.apply_control(ctrl)
                    except Exception:
                        pass

                    # Save frame
                    try:
                        if self._saved_frame_ema is not None:
                            save_img = np.clip(self._saved_frame_ema, 0, 255).astype(np.uint8)
                            save_path = os.path.join(image_dir, f"{frame_count:06d}.png")
                            cv2.imwrite(save_path, save_img)
                    except Exception:
                        pass

                    # Statistics tracking
                    self.deviations.append(abs(lane_error))
                    self.controls_history.append({
                        'frame': frame_count,
                        'model_steer': float(model_steer),
                        'final_steer': float(final_steer),
                        'throttle': float(final_throttle),
                        'speed': float(speed_mps),
                        'lane_error': float(lane_error),
                        'state': self.state
                    })
                    self.trajectory.append({
                        'x': loc.x,
                        'y': loc.y,
                        'z': loc.z,
                        'yaw': rot.yaw
                    })

                    # Logging
                    tstamp = time.time() - self.start_time
                    log_file.write(
                        f"{tstamp:.3f},{frame_count},{loc.x:.2f},{loc.y:.2f},{loc.z:.2f},{rot.yaw:.2f},"
                        f"{speed_mps:.3f},{speed_kmh:.1f},{model_steer:.4f},{final_steer:.4f},"
                        f"{final_throttle:.4f},{lane_error:.4f},{self.state}\n"
                    )
                    log_file.flush()

                    # Console output
                    if time.time() - last_log >= 1.0:
                        tl_msg = "None"
                        if tl_actor is not None and tl_dist is not None:
                            try:
                                st = tl_actor.get_state()
                                tl_msg = f"{st.name} @ {tl_dist:.1f}m"
                            except Exception:
                                tl_msg = f"@{tl_dist:.1f}m"
                        
                        lc_status = f"[LC #{self.lane_changes_completed}]" if self.lane_changes_completed > 0 else ""
                        
                        print(
                            f"[{tstamp:6.1f}s] Frame {frame_count:5d} | "
                            f"Speed: {speed_kmh:5.1f} km/h | "
                            f"Lane: {lane_error:+.3f}m | "
                            f"Steer: {final_steer:+.3f} | "
                            f"TL: {tl_msg:20s} | "
                            f"STATE: {self.state:15s} {lc_status}"
                        )
                        last_log = time.time()

                    frame_count += 1

        except Exception as e:
            print("Evaluation error:", e)
            traceback.print_exc()
        finally:
            try:
                log_file.close()
            except Exception:
                pass
            
            # Create video
            try:
                imgs = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
                if imgs:
                    out_vid = f"eval_video_{int(self.start_time)}.mp4"
                    print(f"Creating video from {len(imgs)} frames...")
                    with imageio.get_writer(out_vid, fps=30) as writer:
                        for im in imgs:
                            writer.append_data(imageio.imread(im))
                    print("✓ Saved video:", out_vid)
            except Exception as e:
                print("Video creation warning:", e)

            self.print_statistics(frame_count)
            self.cleanup()

    def print_statistics(self, frame_count):
        elapsed = time.time() - (self.start_time or time.time())
        print("\n" + "="*60)
        print("EVALUATION STATISTICS")
        print("="*60)
        print(f"Total frames: {frame_count}")
        print(f"Total time: {elapsed:.2f} s")
        if elapsed > 0:
            print(f"Average FPS: {frame_count / elapsed:.2f}")
        print(f"Collisions: {self.collision_count}")
        print(f"Lane changes completed: {self.lane_changes_completed}")
        if self.deviations:
            print(f"Lane keeping:")
            print(f"  Average offset: {np.mean(self.deviations):.3f} m")
            print(f"  Max offset: {np.max(self.deviations):.3f} m")
            print(f"  Std deviation: {np.std(self.deviations):.3f} m")
            within_threshold = sum(1 for d in self.deviations if d < 1.0) / len(self.deviations) * 100
            print(f"  Time within 1.0m: {within_threshold:.1f}%")
        if self.trajectory:
            distances = []
            for i in range(1, len(self.trajectory)):
                dx = self.trajectory[i]['x'] - self.trajectory[i-1]['x']
                dy = self.trajectory[i]['y'] - self.trajectory[i-1]['y']
                distances.append(math.sqrt(dx**2 + dy**2))
            total_dist = sum(distances)
            print(f"Total distance: {total_dist:.2f} m")
        print("="*60)

    def cleanup(self):
        print("Cleaning up actors and restoring world settings...")
        try:
            if self.world is not None and self.original_settings is not None:
                self.world.apply_settings(self.original_settings)
        except Exception:
            pass
        actors = []
        for a in (self.camera, self.collision_sensor, self.vehicle):
            try:
                if a is not None:
                    try:
                        a.stop()
                    except Exception:
                        pass
                    actors.append(a)
            except Exception:
                pass
        for a in actors:
            try:
                a.destroy()
            except Exception:
                pass
        try:
            gc.collect()
        except Exception:
            pass
        print("✓ Cleanup complete")


# ----------------- Entrypoint -----------------
def main():
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    evaluator = ClosedLoopEvaluator(
        model_path=MODEL_PATH,
        detect_yellow=False,
        traffic_light_distance=30.0
    )
    evaluator.run_evaluation()


if __name__ == "__main__":
    main()


STARTING: CNN + Traffic Light Detection + Smart Lane Changes
Spawned vehicle at (106.00, 92.81)
Using device: cuda
Model loaded: /home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth
Warming sensors...


/tmp/ipykernel_2572241/3781641641.py:303: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded = torch.load(self.model_path, map_location=self.device)


🔀 Random lane change attempt...
Attempting lane change to left...
  ✗ Left lane not available or unsafe
Attempting lane change to right...
  ✓ Lane change to right initiated! Path: 30 waypoints (~75.0m)
✅ Lane change #1 complete! (centered & aligned)
   Resuming normal driving
[   1.0s] Frame    47 | Speed:  12.1 km/h | Lane: -0.213m | Steer: -0.058 | TL: None                 | STATE: DRIVING         [LC #1]
[   2.0s] Frame    97 | Speed:  16.6 km/h | Lane: +0.101m | Steer: +0.017 | TL: None                 | STATE: DRIVING         [LC #1]
[   3.0s] Frame   153 | Speed:  18.0 km/h | Lane: -0.005m | Steer: +0.013 | TL: None                 | STATE: DRIVING         [LC #1]
[   4.0s] Frame   211 | Speed:  17.8 km/h | Lane: -0.117m | Steer: -0.009 | TL: Yellow @ 19.3m       | STATE: DRIVING         [LC #1]
[   5.0s] Frame   268 | Speed:  18.0 km/h | Lane: -0.107m | Steer: +0.007 | TL: Red @ 18.6m          | STATE: DRIVING         [LC #1]
🔴 Red light at 17.9m -> STOPPING
🛑 Stopped at red li

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Graceful shutdown requested.
Creating video from 1976 frames...

*** Collision #14 at 36.43s ***
    Impact with: static.building

*** Collision #15 at 37.43s ***
    Impact with: static.building

*** Collision #16 at 38.43s ***
    Impact with: static.building

*** Collision #17 at 39.43s ***
    Impact with: static.building

*** Collision #18 at 40.43s ***
    Impact with: static.building

*** Collision #19 at 41.44s ***
    Impact with: static.building

*** Collision #20 at 42.44s ***
    Impact with: static.building

*** Collision #21 at 43.44s ***
    Impact with: static.building

*** Collision #22 at 44.44s ***
    Impact with: static.building

*** Collision #23 at 45.44s ***
    Impact with: static.building

*** Collision #24 at 46.44s ***
    Impact with: static.building


In [1]:
import carla
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import cv2
import gc
import imageio.v2 as imageio
import signal
import math
import traceback

# Environment setup (limits threads for reproducibility)
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


# ----------------- Utilities -----------------
def preprocess_image_stack(frame_stack, device):
    """Preprocess 3 RGB frames -> (1, 9, 120, 160) tensor on device."""
    if len(frame_stack) != 3:
        return None
    processed_frames = []
    for frame in frame_stack:
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        frame = frame.astype(np.float32) / 255.0
        frame = (frame - 0.5) / 0.5
        processed_frames.append(frame)
    stacked = np.concatenate(processed_frames, axis=2)
    stacked = np.transpose(stacked, (2, 0, 1))
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)
    return tensor.to(device)


class ImprovedPIDController:
    """Stronger PID for better lane-keeping."""
    def __init__(self):
        self.Kp = 0.85
        self.Ki = 0.02
        self.Kd = 0.15
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        self.integral_limit = 0.8
        self.output_limit = 0.7

    def update(self, error, current_time):
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05
        else:
            dt = max(current_time - self.prev_time, 1e-3)
            self.prev_time = current_time
        P = self.Kp * error
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        D = self.Kd * derivative
        self.prev_error = error
        output = P + I + D
        return float(np.clip(output, -self.output_limit, self.output_limit))

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


# ----------------- Main evaluator -----------------
class ClosedLoopEvaluator:
    def __init__(self, model_path, host="localhost", port=2000, detect_yellow=False, traffic_light_distance=30.0):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        self.pid_controller = ImprovedPIDController()
        self.steer_smooth = 0.0
        self.throttle_smooth = 0.3
        self.steer_alpha = 0.25
        self.throttle_alpha = 0.20
        self.max_steer_delta = 0.08

        self.target_speed_kmh = 18.0
        self.max_lane_offset = 2.0

        self.lane_change_active = False
        self.lane_change_waypoints = []
        self.lane_change_start_time = None
        self.lane_change_duration = 3.5
        self.lane_change_finish_threshold = 0.35
        self.lane_change_cooldown = 0.0
        self.min_lane_change_interval = 5.0
        self.max_lane_change_interval = 12.0
        self.next_lane_change_time = 0.0

        self.DETECT_YELLOW = detect_yellow
        self.TRAFFIC_LIGHT_DIST = traffic_light_distance

        self.state = "DRIVING"
        self.stop_start_time = None
        self.stop_speed_threshold = 0.5
        self.red_stop_distance = 18.0
        self.green_resume_delay = 0.3

        self.collision_count = 0
        self.deviations = []
        self.controls_history = []
        self.trajectory = []
        self.lane_changes_completed = 0
        self.lane_changes_attempted = 0
        self.lane_changes_by_direction = {"left": 0, "right": 0}

        self.client = None
        self.world = None
        self.map = None
        self.vehicle = None
        self.camera = None
        self.collision_sensor = None
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        self.start_time = None
        self.original_settings = None

        self._saved_frame_ema = None
        self._ema_alpha = 0.25

        signal.signal(signal.SIGINT, self.signal_handler)

    def signal_handler(self, sig, frame):
        print("\nGraceful shutdown requested.")
        self.running = False

    def connect_carla(self):
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        self.original_settings = self.world.get_settings()
        self.map = self.world.get_map()

        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)

    def spawn_vehicle(self):
        bp_lib = self.world.get_blueprint_library()
        candidates = bp_lib.filter("vehicle.*")
        if not candidates:
            raise RuntimeError("No vehicle blueprints found.")
        vehicle_bp = None
        for name in ("vehicle.tesla.model3", "vehicle.mercedes.coupe_2020", "vehicle.audi.tt"):
            items = bp_lib.filter(name)
            if items:
                vehicle_bp = items[0]
                break
        if vehicle_bp is None:
            vehicle_bp = random.choice(candidates)
        spawn_points = self.map.get_spawn_points()
        if not spawn_points:
            raise RuntimeError("No spawn points available.")
        sp = random.choice(spawn_points)
        sp.location.z += 0.5
        self.vehicle = self.world.spawn_actor(vehicle_bp, sp)
        print(f"Spawned vehicle at ({sp.location.x:.2f}, {sp.location.y:.2f})")
        return sp.location

    def setup_sensors(self):
        bp_lib = self.world.get_blueprint_library()
        cam_bp = bp_lib.find("sensor.camera.rgb")
        cam_bp.set_attribute("image_size_x", "800")
        cam_bp.set_attribute("image_size_y", "600")
        cam_bp.set_attribute("fov", "100")
        cam_transform = carla.Transform(carla.Location(x=1.3, y=0.0, z=1.8), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)

        col_bp = bp_lib.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(col_bp, carla.Transform(), attach_to=self.vehicle)

        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False

        self.camera.listen(self.camera_callback)
        self.collision_sensor.listen(self.collision_callback)

    def camera_callback(self, image):
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]
            small = cv2.resize(arr, (160, 120))
            self.latest_image = small
            self.new_image_available = True

            save_frame = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
            if self._saved_frame_ema is None:
                self._saved_frame_ema = save_frame.astype(np.float32)
            else:
                self._saved_frame_ema = (self._ema_alpha * save_frame.astype(np.float32) +
                                         (1 - self._ema_alpha) * self._saved_frame_ema)
        except Exception:
            pass

    def collision_callback(self, event):
        self.collision_count += 1
        t = time.time() - (self.start_time or time.time())
        print(f"\n*** Collision #{self.collision_count} at {t:.2f}s ***")
        try:
            print("    Impact with:", event.other_actor.type_id)
        except Exception:
            pass
        self.collision_occurred = True
        try:
            self.vehicle.set_velocity(carla.Vector3D(0, 0, 0))
        except Exception:
            pass
        self.pid_controller.reset()
        try:
            ctrl = carla.VehicleControl()
            ctrl.steer = 0.0
            ctrl.throttle = 0.0
            ctrl.brake = 1.0
            self.vehicle.apply_control(ctrl)
        except Exception:
            pass
        time.sleep(1.0)
        self.collision_occurred = False

    def load_model(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Using device:", self.device)
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device)
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model not found: {self.model_path}")
        loaded = torch.load(self.model_path, map_location=self.device)
        if isinstance(loaded, dict) and 'state_dict' in loaded:
            state = loaded['state_dict']
        else:
            state = loaded
        try:
            try:
                self.model.load_state_dict(state)
            except RuntimeError:
                new_state = {}
                for k, v in state.items():
                    nk = k[7:] if k.startswith("module.") else k
                    new_state[nk] = v
                self.model.load_state_dict(new_state)
            print("Model loaded:", self.model_path)
        except Exception as e:
            raise RuntimeError("Failed to load model state dict: " + str(e))
        self.model.eval()

    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        try:
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            if waypoint is None:
                return 0.0
            lane_center = waypoint.transform.location
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            cross_track_error = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            total_error = cross_track_error + 0.6 * heading_error
            return total_error
        except Exception:
            return 0.0

    def get_forward_vector(self, transform):
        f = transform.get_forward_vector()
        return np.array([f.x, f.y, f.z], dtype=np.float32)

    def nearest_traffic_light_ahead(self):
        if self.vehicle is None or self.world is None:
            return None, None
        try:
            veh_tf = self.vehicle.get_transform()
            veh_loc = veh_tf.location
            veh_fwd = self.get_forward_vector(veh_tf)
            lights = self.world.get_actors().filter("traffic.traffic_light*")
            best = None
            best_dist = None
            for tl in lights:
                tl_loc = tl.get_transform().location
                vec = np.array([tl_loc.x - veh_loc.x, tl_loc.y - veh_loc.y, tl_loc.z - veh_loc.z], dtype=np.float32)
                dist = float(np.linalg.norm(vec))
                if dist > self.TRAFFIC_LIGHT_DIST:
                    continue
                if dist < 1e-3:
                    continue
                vec_norm = vec / dist
                dot = float(np.dot(vec_norm, veh_fwd))
                if dot < 0.7:
                    continue
                if best is None or dist < best_dist:
                    best = tl
                    best_dist = dist
            return best, best_dist
        except Exception:
            return None, None

    def is_lane_clear_for_change(self, target_waypoint, direction):
        try:
            veh_loc = self.vehicle.get_location()
            veh_fwd = self.get_forward_vector(self.vehicle.get_transform())
            
            vehicles = self.world.get_actors().filter("vehicle.*")
            
            check_distance_ahead = 25.0
            check_distance_behind = 10.0
            safe_lateral_distance = 2.5
            
            for other_vehicle in vehicles:
                if other_vehicle.id == self.vehicle.id:
                    continue
                
                other_loc = other_vehicle.get_location()
                
                dx = other_loc.x - veh_loc.x
                dy = other_loc.y - veh_loc.y
                distance_2d = math.sqrt(dx**2 + dy**2)
                
                if distance_2d > max(check_distance_ahead, check_distance_behind):
                    continue
                
                vec_to_other = np.array([dx, dy, 0], dtype=np.float32)
                vec_to_other_norm = vec_to_other / (np.linalg.norm(vec_to_other) + 1e-6)
                dot = np.dot(vec_to_other_norm[:2], veh_fwd[:2])
                
                is_ahead = dot > 0.3
                is_behind = dot < -0.3
                
                if is_ahead and distance_2d > check_distance_ahead:
                    continue
                if is_behind and distance_2d > check_distance_behind:
                    continue
                
                try:
                    other_wp = self.map.get_waypoint(other_loc, project_to_road=True, 
                                                     lane_type=carla.LaneType.Driving)
                    if other_wp is None:
                        continue
                    
                    if other_wp.lane_id == target_waypoint.lane_id:
                        target_center = target_waypoint.transform.location
                        dist_to_lane_center = math.sqrt(
                            (other_loc.x - target_center.x)**2 + 
                            (other_loc.y - target_center.y)**2
                        )
                        
                        if dist_to_lane_center < safe_lateral_distance:
                            print(f"  ⚠️ Vehicle detected in target {direction} lane @ {distance_2d:.1f}m")
                            return False
                except Exception:
                    continue
            
            return True
            
        except Exception as e:
            print(f"Lane clearance check error: {e}")
            return False

    def can_initiate_lane_change(self, waypoint, direction):
        try:
            if direction == "left":
                adj = waypoint.get_left_lane()
            else:
                adj = waypoint.get_right_lane()
            
            if adj is None:
                return False
            
            if adj.lane_type != carla.LaneType.Driving:
                return False
            
            if (waypoint.lane_id > 0) != (adj.lane_id > 0):
                return False
            
            if not self.is_lane_clear_for_change(adj, direction):
                return False
                
            return True
        except Exception:
            return False

    def initiate_lane_change_path(self, direction_prefer=None):
        try:
            if time.time() - self.lane_change_cooldown < self.min_lane_change_interval:
                remaining = self.min_lane_change_interval - (time.time() - self.lane_change_cooldown)
                print(f"  ⏳ Lane change on cooldown ({remaining:.1f}s remaining)")
                return False
            
            loc = self.vehicle.get_location()
            wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
            if wp is None:
                print("  ✗ Cannot get current waypoint")
                return False
            
            if direction_prefer:
                directions = [direction_prefer]
            else:
                directions = ["left", "right"] if random.random() < 0.5 else ["right", "left"]
            
            self.lane_changes_attempted += 1
            
            for dirc in directions:
                print(f"  Checking {dirc} lane...")
                
                if not self.can_initiate_lane_change(wp, dirc):
                    print(f"    ✗ {dirc.capitalize()} lane not available or unsafe")
                    continue
                
                target = wp.get_left_lane() if dirc == "left" else wp.get_right_lane()
                if target is None:
                    continue
                
                waypoints = []
                cur = target
                for _ in range(35):
                    waypoints.append(cur)
                    nxts = cur.next(2.5)
                    if not nxts:
                        break
                    cur = nxts[0]
                
                if len(waypoints) < 12:
                    print(f"    ✗ Path too short ({len(waypoints)} waypoints)")
                    continue
                
                self.lane_change_active = True
                self.lane_change_waypoints = waypoints
                self.lane_change_start_time = time.time()
                self.lane_changes_by_direction[dirc] += 1
                print(f"    ✓ Lane change to {dirc.upper()} initiated! ({len(waypoints)} waypoints, ~{len(waypoints)*2.5:.1f}m)")
                return True
            
            print("    ✗ No valid safe lane available")
            return False
            
        except Exception as e:
            print(f"Lane change initiation error: {e}")
            traceback.print_exc()
            return False

    def execute_lane_change(self, vehicle_loc, vehicle_rot, current_steer, current_throttle):
        try:
            if not self.lane_change_active or not self.lane_change_waypoints:
                return current_steer, current_throttle, True
            
            elapsed = time.time() - (self.lane_change_start_time or time.time())
            progress = min(elapsed / self.lane_change_duration, 1.0)
            
            base_idx = int(5 + progress * 10)
            idx = min(base_idx, len(self.lane_change_waypoints)-1)
            
            target_wp = self.lane_change_waypoints[idx]
            tloc = target_wp.transform.location
            target_yaw = math.radians(target_wp.transform.rotation.yaw)
            
            vehicle_yaw = math.radians(vehicle_rot.yaw)
            yaw_diff = target_yaw - vehicle_yaw
            while yaw_diff > math.pi:
                yaw_diff -= 2*math.pi
            while yaw_diff < -math.pi:
                yaw_diff += 2*math.pi
            
            dx = tloc.x - vehicle_loc.x
            dy = tloc.y - vehicle_loc.y
            dist_to_target = math.sqrt(dx**2 + dy**2)
            
            fwd = vehicle_rot.get_forward_vector()
            lateral_error = dx * fwd.y - dy * fwd.x
            
            early_weight = max(0.3, 1.0 - progress)
            
            steer_cmd = float(np.clip(
                early_weight * 1.4 * math.sin(yaw_diff) + 0.6 * np.tanh(lateral_error / 2.5),
                -0.8, 0.8
            ))
            
            blend_factor = 0.7 + 0.2 * progress
            blended = blend_factor * steer_cmd + (1 - blend_factor) * current_steer
            
            blended_throttle = max(current_throttle * 0.9, 0.40)
            
            current_lane_error = abs(self.calculate_lane_error(vehicle_loc, vehicle_rot))
            
            min_elapsed = 2.0
            
            if (elapsed > min_elapsed and 
                current_lane_error < self.lane_change_finish_threshold and 
                abs(yaw_diff) < math.radians(8)):
                
                self.lane_change_active = False
                self.lane_change_waypoints = []
                self.lane_change_start_time = None
                self.lane_change_cooldown = time.time()
                self.lane_changes_completed += 1
                
                self.next_lane_change_time = time.time() + random.uniform(
                    self.min_lane_change_interval, 
                    self.max_lane_change_interval
                )
                
                print(f"  ✅ Lane change #{self.lane_changes_completed} COMPLETE! (centered & aligned in {elapsed:.1f}s)")
                return blended, blended_throttle, True
            
            if elapsed > self.lane_change_duration + 6.0:
                self.lane_change_active = False
                self.lane_change_waypoints = []
                self.lane_change_start_time = None
                self.lane_change_cooldown = time.time()
                
                self.next_lane_change_time = time.time() + random.uniform(
                    self.min_lane_change_interval, 
                    self.max_lane_change_interval
                )
                
                print(f"  ⚠️ Lane change timeout after {elapsed:.1f}s; completing anyway")
                return blended, blended_throttle, True
            
            return blended, blended_throttle, False
            
        except Exception as e:
            print(f"Lane change execution error: {e}")
            traceback.print_exc()
            self.lane_change_active = False
            self.lane_change_waypoints = []
            self.lane_change_start_time = None
            return current_steer, current_throttle, True

    def adaptive_speed_control(self, current_speed_mps, lane_offset, steering_magnitude):
        target_speed_mps = self.target_speed_kmh / 3.6
        offset_factor = max(0.75, 1.0 - abs(lane_offset) / 3.0)
        steering_factor = max(0.80, 1.0 - abs(steering_magnitude) / 0.6)
        adjusted_target = target_speed_mps * offset_factor * steering_factor
        speed_error = adjusted_target - current_speed_mps
        if speed_error > 0:
            throttle = min(0.7, 0.42 + speed_error * 0.30)
        else:
            throttle = max(0.0, 0.38 + speed_error * 0.25)
        return float(np.clip(throttle, 0.0, 0.9))

    def run_evaluation(self):
        print("\n" + "="*60)
        print("STARTING: CNN + Traffic Light Detection + Smart Lane Changes")
        print("="*60)
        try:
            self.connect_carla()
            start_location = self.spawn_vehicle()
            self.setup_sensors()
            self.load_model()
        except Exception as e:
            print("Init failed:", e)
            traceback.print_exc()
            return

        spectator = self.world.get_spectator()
        ctrl = carla.VehicleControl()
        ctrl.manual_gear_shift = False

        print("Warming sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)

        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log = time.time()
        
        self.next_lane_change_time = time.time() + random.uniform(
            self.min_lane_change_interval, 
            self.max_lane_change_interval
        )

        log_filename = f"eval_log_{int(self.start_time)}.csv"
        log_file = open(log_filename, "w")
        log_file.write("timestamp,frame,x,y,z,yaw,speed_mps,speed_kmh,model_steer,final_steer,throttle,lane_error,state\n")

        image_dir = f"eval_images_{int(self.start_time)}"
        os.makedirs(image_dir, exist_ok=True)

        try:
            while self.running:
                self.world.tick()

                try:
                    tf = self.vehicle.get_transform()
                    sp_loc = tf.location + tf.get_forward_vector() * -6 + carla.Location(z=3.0)
                    sp_rot = carla.Rotation(pitch=-12, yaw=tf.rotation.yaw)
                    spectator.set_transform(carla.Transform(sp_loc, sp_rot))
                except Exception:
                    pass

                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False

                tl_actor, tl_dist = self.nearest_traffic_light_ahead()
                tl_state = None
                if tl_actor is not None:
                    try:
                        tl_state = tl_actor.get_state()
                    except Exception:
                        tl_state = None

                if (tl_actor is not None and tl_state == carla.TrafficLightState.Red and
                    tl_dist is not None and tl_dist <= self.red_stop_distance and 
                    self.state == "DRIVING"):
                    print(f"🔴 Red light at {tl_dist:.1f}m -> STOPPING")
                    self.state = "STOPPING"
                    self.stop_start_time = None

                if (self.state == "WAITING_GREEN" and tl_actor is not None and 
                    tl_state == carla.TrafficLightState.Green):
                    print("🟢 Green light detected -> Attempting lane change...")
                    time.sleep(self.green_resume_delay)
                    ok = self.initiate_lane_change_path(direction_prefer=None)
                    if ok:
                        self.state = "LANE_CHANGE"
                    else:
                        print("   No safe lane change available, resuming driving")
                        self.state = "DRIVING"

                if (self.state == "DRIVING" and 
                    not self.lane_change_active and 
                    time.time() >= self.next_lane_change_time):
                    
                    print("🔀 Scheduled lane change attempt...")
                    ok = self.initiate_lane_change_path(direction_prefer=None)
                    if ok:
                        self.state = "LANE_CHANGE"
                    else:
                        self.next_lane_change_time = time.time() + random.uniform(3.0, 6.0)
                        print(f"   No safe lane available, retry in {self.next_lane_change_time - time.time():.1f}s")

                if len(frame_stack) == 3:
                    tf = self.vehicle.get_transform()
                    loc = tf.location
                    rot = tf.rotation
                    vel = self.vehicle.get_velocity()
                    speed_mps = math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
                    speed_kmh = speed_mps * 3.6

                    lane_error = self.calculate_lane_error(loc, rot)

                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    if input_tensor is None:
                        continue

                    speed_norm = min(speed_mps / 30.0, 1.0)
                    speed_tensor = torch.tensor([[speed_norm]], dtype=torch.float32, device=self.device)

                    with torch.no_grad():
                        output = self.model(input_tensor, aux=speed_tensor)
                    model_steer, model_throttle, model_brake = output[0].cpu().numpy()

                    now = time.time()
                    pid_corr = self.pid_controller.update(lane_error, now)

                    err_w = min(abs(lane_error) / 0.8, 0.85)
                    combined = (1 - err_w) * model_steer + err_w * pid_corr

                    final_steer_candidate = (self.steer_alpha * combined + (1 - self.steer_alpha) * self.steer_smooth)

                    delta = float(final_steer_candidate - self.steer_smooth)
                    delta = np.clip(delta, -self.max_steer_delta, self.max_steer_delta)
                    final_steer = float(self.steer_smooth + delta)
                    final_steer = float(np.clip(final_steer, -0.7, 0.7))
                    self.steer_smooth = final_steer

                    adaptive_throttle = self.adaptive_speed_control(speed_mps, lane_error, abs(final_steer))
                    final_throttle_candidate = (self.throttle_alpha * adaptive_throttle + (1 - self.throttle_alpha) * self.throttle_smooth)
                    final_throttle = float(np.clip(final_throttle_candidate, 0.0, 0.9))
                    self.throttle_smooth = final_throttle

                    final_brake = 0.0

                    if self.state == "STOPPING":
                        if self.stop_start_time is None:
                            self.stop_start_time = time.time()
                            self.pid_controller.reset()
                        
                        if tl_dist is None:
                            brake_val = min(1.0, 0.6 + (speed_mps / 10.0))
                        else:
                            approach = max(0.05, (self.red_stop_distance - tl_dist) / max(1.0, self.red_stop_distance))
                            brake_val = np.clip(0.3 + approach * 0.9 + (speed_mps / 20.0), 0.0, 1.0)
                        
                        final_throttle = 0.0
                        final_brake = float(brake_val)
                        final_steer = float(np.clip(final_steer * 0.5, -0.3, 0.3))
                        
                        if speed_mps < self.stop_speed_threshold:
                            print("🛑 Stopped at red light, waiting for green...")
                            self.state = "WAITING_GREEN"
                    
                    elif self.state == "WAITING_GREEN":
                        final_throttle = 0.0
                        final_brake = 1.0
                        final_steer = 0.0
                    
                    elif self.state == "LANE_CHANGE":
                        final_brake = 0.0
                        final_steer, final_throttle, done = self.execute_lane_change(loc, rot, final_steer, final_throttle)
                        if done:
                            print("   Resuming normal driving")
                            self.state = "DRIVING"
                    
                    else:
                        final_brake = 0.0

                    ctrl.steer = float(final_steer)
                    ctrl.throttle = float(final_throttle)
                    ctrl.brake = float(final_brake)
                    ctrl.hand_brake = False
                    try:
                        self.vehicle.apply_control(ctrl)
                    except Exception:
                        pass

                    try:
                        if self._saved_frame_ema is not None:
                            save_img = np.clip(self._saved_frame_ema, 0, 255).astype(np.uint8)
                            save_path = os.path.join(image_dir, f"{frame_count:06d}.png")
                            cv2.imwrite(save_path, save_img)
                    except Exception:
                        pass

                    self.deviations.append(abs(lane_error))
                    self.controls_history.append({
                        'frame': frame_count,
                        'model_steer': float(model_steer),
                        'final_steer': float(final_steer),
                        'throttle': float(final_throttle),
                        'speed': float(speed_mps),
                        'lane_error': float(lane_error),
                        'state': self.state
                    })
                    self.trajectory.append({
                        'x': loc.x,
                        'y': loc.y,
                        'z': loc.z,
                        'yaw': rot.yaw
                    })

                    tstamp = time.time() - self.start_time
                    log_file.write(
                        f"{tstamp:.3f},{frame_count},{loc.x:.2f},{loc.y:.2f},{loc.z:.2f},{rot.yaw:.2f},"
                        f"{speed_mps:.3f},{speed_kmh:.1f},{model_steer:.4f},{final_steer:.4f},"
                        f"{final_throttle:.4f},{lane_error:.4f},{self.state}\n"
                    )
                    log_file.flush()

                    if time.time() - last_log >= 1.0:
                        tl_msg = "None"
                        if tl_actor is not None and tl_dist is not None:
                            try:
                                st = tl_actor.get_state()
                                tl_msg = f"{st.name} @ {tl_dist:.1f}m"
                            except Exception:
                                tl_msg = f"@{tl_dist:.1f}m"
                        
                        lc_status = ""
                        if self.lane_changes_completed > 0:
                            lc_status = f"[LC: {self.lane_changes_completed}/{self.lane_changes_attempted}]"
                        if self.state == "DRIVING" and not self.lane_change_active:
                            next_lc = self.next_lane_change_time - time.time()
                            if next_lc > 0:
                                lc_status += f" Next:{next_lc:.0f}s"
                        
                        print(
                            f"[{tstamp:6.1f}s] Frame {frame_count:5d} | "
                            f"Speed: {speed_kmh:5.1f} km/h | "
                            f"Lane: {lane_error:+.3f}m | "
                            f"Steer: {final_steer:+.3f} | "
                            f"TL: {tl_msg:20s} | "
                            f"STATE: {self.state:15s} {lc_status}"
                        )
                        last_log = time.time()

                    frame_count += 1

        except Exception as e:
            print("Evaluation error:", e)
            traceback.print_exc()
        finally:
            try:
                log_file.close()
            except Exception:
                pass
            
            try:
                imgs = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
                if imgs:
                    out_vid = f"eval_video_{int(self.start_time)}.mp4"
                    print(f"Creating video from {len(imgs)} frames...")
                    with imageio.get_writer(out_vid, fps=30) as writer:
                        for im in imgs:
                            writer.append_data(imageio.imread(im))
                    print("✓ Saved video:", out_vid)
            except Exception as e:
                print("Video creation warning:", e)

            self.print_statistics(frame_count)
            self.cleanup()

    def print_statistics(self, frame_count):
        elapsed = time.time() - (self.start_time or time.time())
        print("\n" + "="*60)
        print("EVALUATION STATISTICS")
        print("="*60)
        print(f"Total frames: {frame_count}")
        print(f"Total time: {elapsed:.2f} s")
        if elapsed > 0:
            print(f"Average FPS: {frame_count / elapsed:.2f}")
        print(f"Collisions: {self.collision_count}")
        print(f"\nLane Change Performance:")
        print(f"  Attempted: {self.lane_changes_attempted}")
        print(f"  Completed: {self.lane_changes_completed}")
        if self.lane_changes_attempted > 0:
            success_rate = (self.lane_changes_completed / self.lane_changes_attempted) * 100
            print(f"  Success rate: {success_rate:.1f}%")
        print(f"  Left lane changes: {self.lane_changes_by_direction['left']}")
        print(f"  Right lane changes: {self.lane_changes_by_direction['right']}")
        if self.deviations:
            print(f"\nLane Keeping Performance:")
            print(f"  Average offset: {np.mean(self.deviations):.3f} m")
            print(f"  Max offset: {np.max(self.deviations):.3f} m")
            print(f"  Std deviation: {np.std(self.deviations):.3f} m")
            within_threshold = sum(1 for d in self.deviations if d < 1.0) / len(self.deviations) * 100
            print(f"  Time within 1.0m: {within_threshold:.1f}%")
            within_threshold_tight = sum(1 for d in self.deviations if d < 0.5) / len(self.deviations) * 100
            print(f"  Time within 0.5m: {within_threshold_tight:.1f}%")
        if self.trajectory:
            distances = []
            for i in range(1, len(self.trajectory)):
                dx = self.trajectory[i]['x'] - self.trajectory[i-1]['x']
                dy = self.trajectory[i]['y'] - self.trajectory[i-1]['y']
                distances.append(math.sqrt(dx**2 + dy**2))
            total_dist = sum(distances)
            print(f"\nTotal distance: {total_dist:.2f} m")
            if elapsed > 0:
                avg_speed_mps = total_dist / elapsed
                print(f"Average speed: {avg_speed_mps * 3.6:.2f} km/h")
        print("="*60)

    def cleanup(self):
        print("Cleaning up actors and restoring world settings...")
        try:
            if self.world is not None and self.original_settings is not None:
                self.world.apply_settings(self.original_settings)
        except Exception:
            pass
        actors = []
        for a in (self.camera, self.collision_sensor, self.vehicle):
            try:
                if a is not None:
                    try:
                        a.stop()
                    except Exception:
                        pass
                    actors.append(a)
            except Exception:
                pass
        for a in actors:
            try:
                a.destroy()
            except Exception:
                pass
        try:
            gc.collect()
        except Exception:
            pass
        print("✓ Cleanup complete")


def main():
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    evaluator = ClosedLoopEvaluator(
        model_path=MODEL_PATH,
        detect_yellow=False,
        traffic_light_distance=30.0
    )
    evaluator.run_evaluation()


if __name__ == "__main__":
    main()


STARTING: CNN + Traffic Light Detection + Smart Lane Changes
Spawned vehicle at (-114.43, 56.85)
Using device: cuda


/tmp/ipykernel_2582996/3191549787.py:293: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  loaded = torch.load(self.model_path, map_location=self.device)


Model loaded: /home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth
Warming sensors...
[   1.0s] Frame    24 | Speed:   0.0 km/h | Lane: -0.053m | Steer: -0.010 | TL: None                 | STATE: DRIVING          Next:6s
[   2.0s] Frame    54 | Speed:  15.0 km/h | Lane: -0.034m | Steer: +0.007 | TL: None                 | STATE: DRIVING          Next:5s
[   3.1s] Frame    85 | Speed:  17.9 km/h | Lane: -0.073m | Steer: -0.003 | TL: None                 | STATE: DRIVING          Next:4s
[   4.1s] Frame   117 | Speed:  18.0 km/h | Lane: -0.099m | Steer: -0.007 | TL: None                 | STATE: DRIVING          Next:3s
[   5.1s] Frame   149 | Speed:  17.5 km/h | Lane: -0.258m | Steer: -0.065 | TL: None                 | STATE: DRIVING          Next:2s
[   6.1s] Frame   180 | Speed:  16.2 km/h | Lane: -0.148m | Steer: -0.020 | TL: None                 | STATE: DRIVING          Next:1s
🔀 Scheduled lane change attempt...
  Checking right lane...
    ✗ Right lane n

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Graceful shutdown requested.
Creating video from 318 frames...

*** Collision #3 at 10.78s ***
    Impact with: static.vegetation

*** Collision #4 at 11.78s ***
    Impact with: static.vegetation

*** Collision #5 at 12.78s ***
    Impact with: static.vegetation
✓ Saved video: eval_video_1760296156.mp4

EVALUATION STATISTICS
Total frames: 318
Total time: 13.24 s
Average FPS: 24.02
Collisions: 5

Lane Change Performance:
  Attempted: 1
  Completed: 0
  Success rate: 0.0%
  Left lane changes: 1
  Right lane changes: 0

Lane Keeping Performance:
  Average offset: 0.936 m
  Max offset: 3.694 m
  Std deviation: 1.344 m
  Time within 1.0m: 71.7%
  Time within 0.5m: 68.6%

Total distance: 54.73 m
Average speed: 14.88 km/h
Cleaning up actors and restoring world settings...

*** Collision #6 at 13.78s ***
    Impact with: static.vegetation
✓ Cleanup complete


In [4]:
import carla
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import cv2
import gc
import imageio.v2 as imageio
import signal
import math
import traceback

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


def preprocess_image_stack(frame_stack, device):
    if len(frame_stack) != 3:
        return None
    processed_frames = []
    for frame in frame_stack:
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        frame = frame.astype(np.float32) / 255.0
        frame = (frame - 0.5) / 0.5
        processed_frames.append(frame)
    stacked = np.concatenate(processed_frames, axis=2)
    stacked = np.transpose(stacked, (2, 0, 1))
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)
    return tensor.to(device)


class ImprovedPIDController:
    """Very strong PID for excellent lane-keeping."""
    def __init__(self):
        # MUCH STRONGER gains to eliminate drift
        self.Kp = 1.2  # Increased from 0.85
        self.Ki = 0.05  # Increased from 0.02
        self.Kd = 0.25  # Increased from 0.15
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        self.integral_limit = 1.0
        self.output_limit = 0.85

    def update(self, error, current_time):
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05
        else:
            dt = max(current_time - self.prev_time, 1e-3)
            self.prev_time = current_time
        P = self.Kp * error
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        D = self.Kd * derivative
        self.prev_error = error
        output = P + I + D
        return float(np.clip(output, -self.output_limit, self.output_limit))

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


class ClosedLoopEvaluator:
    def __init__(self, model_path, host="localhost", port=2000, traffic_light_distance=30.0):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        self.pid_controller = ImprovedPIDController()
        self.steer_smooth = 0.0
        self.throttle_smooth = 0.3
        self.steer_alpha = 0.35  # More responsive
        self.throttle_alpha = 0.25
        self.max_steer_delta = 0.10

        self.target_speed_kmh = 20.0
        self.max_lane_offset = 2.0

        # Lane change settings - ONLY at traffic lights
        self.lane_change_active = False
        self.lane_change_target_wp = None
        self.lane_change_start_time = None
        self.lane_change_duration = 4.0  # Longer for smoother transition
        self.lane_change_cooldown = 0.0
        self.min_lane_change_interval = 2.0  # Cooldown after completion
        
        self.TRAFFIC_LIGHT_DIST = traffic_light_distance

        # Traffic light states
        self.state = "DRIVING"
        self.stop_start_time = None
        self.stop_speed_threshold = 0.5
        self.red_stop_distance = 20.0
        self.green_resume_delay = 0.5
        self.at_traffic_light = False  # Track if we're at a signal
        self.last_green_light_time = 0.0

        self.collision_count = 0
        self.deviations = []
        self.controls_history = []
        self.trajectory = []
        self.lane_changes_completed = 0
        self.lane_changes_attempted = 0
        self.lane_changes_by_direction = {"left": 0, "right": 0}

        self.client = None
        self.world = None
        self.map = None
        self.vehicle = None
        self.camera = None
        self.collision_sensor = None
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        self.start_time = None
        self.original_settings = None

        self._saved_frame_ema = None
        self._ema_alpha = 0.25

        signal.signal(signal.SIGINT, self.signal_handler)

    def signal_handler(self, sig, frame):
        print("\nGraceful shutdown requested.")
        self.running = False

    def connect_carla(self):
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        self.original_settings = self.world.get_settings()
        self.map = self.world.get_map()

        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)

    def spawn_vehicle(self):
        bp_lib = self.world.get_blueprint_library()
        candidates = bp_lib.filter("vehicle.*")
        if not candidates:
            raise RuntimeError("No vehicle blueprints found.")
        vehicle_bp = None
        for name in ("vehicle.tesla.model3", "vehicle.mercedes.coupe_2020", "vehicle.audi.tt"):
            items = bp_lib.filter(name)
            if items:
                vehicle_bp = items[0]
                break
        if vehicle_bp is None:
            vehicle_bp = random.choice(candidates)
        spawn_points = self.map.get_spawn_points()
        if not spawn_points:
            raise RuntimeError("No spawn points available.")
        sp = random.choice(spawn_points)
        sp.location.z += 0.5
        self.vehicle = self.world.spawn_actor(vehicle_bp, sp)
        print(f"Spawned vehicle at ({sp.location.x:.2f}, {sp.location.y:.2f})")
        return sp.location

    def setup_sensors(self):
        bp_lib = self.world.get_blueprint_library()
        cam_bp = bp_lib.find("sensor.camera.rgb")
        cam_bp.set_attribute("image_size_x", "800")
        cam_bp.set_attribute("image_size_y", "600")
        cam_bp.set_attribute("fov", "100")
        cam_transform = carla.Transform(carla.Location(x=1.3, y=0.0, z=1.8), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)

        col_bp = bp_lib.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(col_bp, carla.Transform(), attach_to=self.vehicle)

        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False

        self.camera.listen(self.camera_callback)
        self.collision_sensor.listen(self.collision_callback)

    def camera_callback(self, image):
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]
            small = cv2.resize(arr, (160, 120))
            self.latest_image = small
            self.new_image_available = True

            save_frame = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
            if self._saved_frame_ema is None:
                self._saved_frame_ema = save_frame.astype(np.float32)
            else:
                self._saved_frame_ema = (self._ema_alpha * save_frame.astype(np.float32) +
                                         (1 - self._ema_alpha) * self._saved_frame_ema)
        except Exception:
            pass

    def collision_callback(self, event):
        self.collision_count += 1
        t = time.time() - (self.start_time or time.time())
        print(f"\n*** Collision #{self.collision_count} at {t:.2f}s ***")
        try:
            print("    Impact with:", event.other_actor.type_id)
        except Exception:
            pass
        self.collision_occurred = True
        # Cancel lane change on collision
        self.lane_change_active = False
        self.lane_change_target_wp = None
        try:
            self.vehicle.set_velocity(carla.Vector3D(0, 0, 0))
        except Exception:
            pass
        self.pid_controller.reset()
        try:
            ctrl = carla.VehicleControl()
            ctrl.steer = 0.0
            ctrl.throttle = 0.0
            ctrl.brake = 1.0
            self.vehicle.apply_control(ctrl)
        except Exception:
            pass
        time.sleep(1.0)
        self.collision_occurred = False

    def load_model(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Using device:", self.device)
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device)
        if not os.path.exists(self.model_path):
            raise FileNotFoundError(f"Model not found: {self.model_path}")
        loaded = torch.load(self.model_path, map_location=self.device, weights_only=False)
        if isinstance(loaded, dict) and 'state_dict' in loaded:
            state = loaded['state_dict']
        else:
            state = loaded
        try:
            try:
                self.model.load_state_dict(state)
            except RuntimeError:
                new_state = {}
                for k, v in state.items():
                    nk = k[7:] if k.startswith("module.") else k
                    new_state[nk] = v
                self.model.load_state_dict(new_state)
            print("Model loaded:", self.model_path)
        except Exception as e:
            raise RuntimeError("Failed to load model state dict: " + str(e))
        self.model.eval()

    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        try:
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            if waypoint is None:
                return 0.0
            lane_center = waypoint.transform.location
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            cross_track_error = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            # Stronger heading weight for better alignment
            total_error = cross_track_error + 0.8 * heading_error
            return total_error
        except Exception:
            return 0.0

    def get_forward_vector(self, transform):
        f = transform.get_forward_vector()
        return np.array([f.x, f.y, f.z], dtype=np.float32)

    def nearest_traffic_light_ahead(self):
        if self.vehicle is None or self.world is None:
            return None, None
        try:
            veh_tf = self.vehicle.get_transform()
            veh_loc = veh_tf.location
            veh_fwd = self.get_forward_vector(veh_tf)
            lights = self.world.get_actors().filter("traffic.traffic_light*")
            best = None
            best_dist = None
            for tl in lights:
                tl_loc = tl.get_transform().location
                vec = np.array([tl_loc.x - veh_loc.x, tl_loc.y - veh_loc.y, tl_loc.z - veh_loc.z], dtype=np.float32)
                dist = float(np.linalg.norm(vec))
                if dist > self.TRAFFIC_LIGHT_DIST:
                    continue
                if dist < 1e-3:
                    continue
                vec_norm = vec / dist
                dot = float(np.dot(vec_norm, veh_fwd))
                if dot < 0.7:
                    continue
                if best is None or dist < best_dist:
                    best = tl
                    best_dist = dist
            return best, best_dist
        except Exception:
            return None, None

    def can_change_lane_safely(self, current_wp, direction):
        """Check if lane change is safe - stricter validation."""
        try:
            if direction == "left":
                target_wp = current_wp.get_left_lane()
            else:
                target_wp = current_wp.get_right_lane()
            
            if target_wp is None:
                return False, None
            
            # Must be driving lane
            if target_wp.lane_type != carla.LaneType.Driving:
                return False, None
            
            # Must be same direction (not opposite traffic)
            if (current_wp.lane_id > 0) != (target_wp.lane_id > 0):
                return False, None
            
            # Check if there's a long enough path ahead (prevent going off-road)
            test_wp = target_wp
            valid_path_length = 0
            for _ in range(25):  # Check 50m ahead
                nxts = test_wp.next(2.0)
                if not nxts:
                    break
                test_wp = nxts[0]
                if test_wp.lane_type != carla.LaneType.Driving:
                    break
                valid_path_length += 2.0
            
            if valid_path_length < 30.0:  # Need at least 30m of valid path
                print(f"    ✗ {direction.capitalize()} lane path too short ({valid_path_length:.1f}m)")
                return False, None
            
            # Check for vehicles in target lane
            veh_loc = self.vehicle.get_location()
            vehicles = self.world.get_actors().filter("vehicle.*")
            
            for other in vehicles:
                if other.id == self.vehicle.id:
                    continue
                
                other_loc = other.get_location()
                dx = other_loc.x - veh_loc.x
                dy = other_loc.y - veh_loc.y
                dist = math.sqrt(dx**2 + dy**2)
                
                if dist > 30.0:
                    continue
                
                try:
                    other_wp = self.map.get_waypoint(other_loc, project_to_road=True)
                    if other_wp and other_wp.lane_id == target_wp.lane_id:
                        if dist < 20.0:  # Too close
                            print(f"    ✗ Vehicle in {direction} lane @ {dist:.1f}m")
                            return False, None
                except Exception:
                    continue
            
            return True, target_wp
            
        except Exception as e:
            print(f"Lane check error: {e}")
            return False, None

    def attempt_lane_change_at_signal(self):
        """Only called when at traffic light - try to change lanes."""
        try:
            # Check cooldown
            if time.time() - self.lane_change_cooldown < self.min_lane_change_interval:
                print("  ⏳ Lane change on cooldown")
                return False
            
            loc = self.vehicle.get_location()
            current_wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
            if current_wp is None:
                print("  ✗ Cannot get current waypoint")
                return False
            
            self.lane_changes_attempted += 1
            
            # Try both directions
            directions = ["left", "right"] if random.random() < 0.5 else ["right", "left"]
            
            for direction in directions:
                print(f"  Checking {direction} lane...")
                safe, target_wp = self.can_change_lane_safely(current_wp, direction)
                
                if safe and target_wp:
                    # Initiate lane change
                    self.lane_change_active = True
                    self.lane_change_target_wp = target_wp
                    self.lane_change_start_time = time.time()
                    self.lane_changes_by_direction[direction] += 1
                    print(f"    ✓ Lane change to {direction.upper()} initiated!")
                    return True
            
            print("    ✗ No safe lane available")
            return False
            
        except Exception as e:
            print(f"Lane change attempt error: {e}")
            traceback.print_exc()
            return False

    def execute_lane_change(self, vehicle_loc, vehicle_rot, base_steer, base_throttle):
        """Execute smooth lane change using target waypoint."""
        try:
            if not self.lane_change_active or self.lane_change_target_wp is None:
                return base_steer, base_throttle, True
            
            elapsed = time.time() - (self.lane_change_start_time or time.time())
            
            # Get target waypoint position ahead
            target_wp = self.lane_change_target_wp
            for _ in range(15):  # Look 30m ahead
                nxts = target_wp.next(2.0)
                if nxts:
                    target_wp = nxts[0]
                else:
                    break
            
            target_loc = target_wp.transform.location
            target_yaw = math.radians(target_wp.transform.rotation.yaw)
            
            # Calculate errors
            vehicle_yaw = math.radians(vehicle_rot.yaw)
            yaw_diff = target_yaw - vehicle_yaw
            while yaw_diff > math.pi:
                yaw_diff -= 2*math.pi
            while yaw_diff < -math.pi:
                yaw_diff += 2*math.pi
            
            # Lateral error
            dx = target_loc.x - vehicle_loc.x
            dy = target_loc.y - vehicle_loc.y
            fwd = vehicle_rot.get_forward_vector()
            lateral_error = dx * fwd.y - dy * fwd.x
            
            # Smooth steering command
            progress = min(elapsed / self.lane_change_duration, 1.0)
            aggression = 0.8 * (1.0 - progress) + 0.3 * progress  # Start aggressive, end gentle
            
            steer_cmd = aggression * (0.9 * math.sin(yaw_diff) + 0.4 * np.tanh(lateral_error / 3.0))
            steer_cmd = float(np.clip(steer_cmd, -0.7, 0.7))
            
            # Blend with base steering
            blend_weight = 0.85
            final_steer = blend_weight * steer_cmd + (1 - blend_weight) * base_steer
            
            # Reduce speed slightly during lane change
            final_throttle = base_throttle * 0.85
            
            # Check completion - must be well-centered in new lane
            current_lane_error = abs(self.calculate_lane_error(vehicle_loc, vehicle_rot))
            
            if elapsed > 2.5 and current_lane_error < 0.25 and abs(yaw_diff) < math.radians(5):
                self.lane_change_active = False
                self.lane_change_target_wp = None
                self.lane_change_cooldown = time.time()
                self.lane_changes_completed += 1
                print(f"  ✅ Lane change #{self.lane_changes_completed} COMPLETE! (error: {current_lane_error:.2f}m, {elapsed:.1f}s)")
                # Reset PID to stabilize in new lane
                self.pid_controller.reset()
                return final_steer, final_throttle, True
            
            # Timeout
            if elapsed > self.lane_change_duration + 3.0:
                print(f"  ⚠️ Lane change timeout (error: {current_lane_error:.2f}m)")
                self.lane_change_active = False
                self.lane_change_target_wp = None
                self.lane_change_cooldown = time.time()
                self.pid_controller.reset()
                return final_steer, final_throttle, True
            
            return final_steer, final_throttle, False
            
        except Exception as e:
            print(f"Lane change execution error: {e}")
            self.lane_change_active = False
            self.lane_change_target_wp = None
            return base_steer, base_throttle, True

    def adaptive_speed_control(self, current_speed_mps, lane_offset, steering_magnitude):
        target_speed_mps = self.target_speed_kmh / 3.6
        offset_factor = max(0.8, 1.0 - abs(lane_offset) / 2.5)
        steering_factor = max(0.85, 1.0 - abs(steering_magnitude) / 0.5)
        adjusted_target = target_speed_mps * offset_factor * steering_factor
        speed_error = adjusted_target - current_speed_mps
        if speed_error > 0:
            throttle = min(0.75, 0.45 + speed_error * 0.32)
        else:
            throttle = max(0.0, 0.40 + speed_error * 0.28)
        return float(np.clip(throttle, 0.0, 0.9))

    def run_evaluation(self):
        print("\n" + "="*60)
        print("STARTING: Signal-Based Lane Changes + Strong Lane Keeping")
        print("="*60)
        try:
            self.connect_carla()
            start_location = self.spawn_vehicle()
            self.setup_sensors()
            self.load_model()
        except Exception as e:
            print("Init failed:", e)
            traceback.print_exc()
            return

        spectator = self.world.get_spectator()
        ctrl = carla.VehicleControl()
        ctrl.manual_gear_shift = False

        print("Warming sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)

        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log = time.time()

        log_filename = f"eval_log_{int(self.start_time)}.csv"
        log_file = open(log_filename, "w")
        log_file.write("timestamp,frame,x,y,z,yaw,speed_mps,speed_kmh,model_steer,final_steer,throttle,lane_error,state\n")

        image_dir = f"eval_images_{int(self.start_time)}"
        os.makedirs(image_dir, exist_ok=True)

        try:
            while self.running:
                self.world.tick()

                try:
                    tf = self.vehicle.get_transform()
                    sp_loc = tf.location + tf.get_forward_vector() * -6 + carla.Location(z=3.0)
                    sp_rot = carla.Rotation(pitch=-12, yaw=tf.rotation.yaw)
                    spectator.set_transform(carla.Transform(sp_loc, sp_rot))
                except Exception:
                    pass

                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False

                tl_actor, tl_dist = self.nearest_traffic_light_ahead()
                tl_state = None
                if tl_actor is not None:
                    try:
                        tl_state = tl_actor.get_state()
                    except Exception:
                        tl_state = None

                # Track if we're approaching a traffic light
                self.at_traffic_light = (tl_actor is not None and tl_dist is not None and tl_dist < 25.0)

                # STATE TRANSITIONS
                if (tl_actor is not None and tl_state == carla.TrafficLightState.Red and
                    tl_dist is not None and tl_dist <= self.red_stop_distance and 
                    self.state == "DRIVING"):
                    print(f"🔴 Red light at {tl_dist:.1f}m -> STOPPING")
                    self.state = "STOPPING"
                    self.stop_start_time = None

                # Green light -> attempt lane change ONLY if at signal
                if (self.state == "WAITING_GREEN" and tl_actor is not None and 
                    tl_state == carla.TrafficLightState.Green):
                    print("🟢 Green light detected -> Attempting lane change...")
                    time.sleep(self.green_resume_delay)
                    self.last_green_light_time = time.time()
                    
                    ok = self.attempt_lane_change_at_signal()
                    if ok:
                        self.state = "LANE_CHANGE"
                    else:
                        print("   No lane change possible, resuming driving")
                        self.state = "DRIVING"

                if len(frame_stack) == 3:
                    tf = self.vehicle.get_transform()
                    loc = tf.location
                    rot = tf.rotation
                    vel = self.vehicle.get_velocity()
                    speed_mps = math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
                    speed_kmh = speed_mps * 3.6

                    lane_error = self.calculate_lane_error(loc, rot)

                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    if input_tensor is None:
                        continue

                    speed_norm = min(speed_mps / 30.0, 1.0)
                    speed_tensor = torch.tensor([[speed_norm]], dtype=torch.float32, device=self.device)

                    with torch.no_grad():
                        output = self.model(input_tensor, aux=speed_tensor)
                    model_steer, model_throttle, model_brake = output[0].cpu().numpy()

                    now = time.time()
                    pid_corr = self.pid_controller.update(lane_error, now)

                    # VERY STRONG PID blending - prioritize lane keeping
                    err_w = min(abs(lane_error) / 0.6, 0.92)  # Up to 92% PID influence
                    combined = (1 - err_w) * model_steer + err_w * pid_corr

                    # Steering smoothing
                    final_steer_candidate = (self.steer_alpha * combined + (1 - self.steer_alpha) * self.steer_smooth)

                    # Apply rate limiting
                    delta = float(final_steer_candidate - self.steer_smooth)
                    delta = np.clip(delta, -self.max_steer_delta, self.max_steer_delta)
                    final_steer = float(self.steer_smooth + delta)
                    final_steer = float(np.clip(final_steer, -0.8, 0.8))
                    self.steer_smooth = final_steer

                    # Throttle control
                    adaptive_throttle = self.adaptive_speed_control(speed_mps, lane_error, abs(final_steer))
                    final_throttle_candidate = (self.throttle_alpha * adaptive_throttle + (1 - self.throttle_alpha) * self.throttle_smooth)
                    final_throttle = float(np.clip(final_throttle_candidate, 0.0, 0.9))
                    self.throttle_smooth = final_throttle

                    final_brake = 0.0

                    # STATE MACHINE EXECUTION
                    if self.state == "STOPPING":
                        if self.stop_start_time is None:
                            self.stop_start_time = time.time()
                            self.pid_controller.reset()
                        
                        if tl_dist is None:
                            brake_val = min(1.0, 0.6 + (speed_mps / 10.0))
                        else:
                            approach = max(0.05, (self.red_stop_distance - tl_dist) / max(1.0, self.red_stop_distance))
                            brake_val = np.clip(0.3 + approach * 0.9 + (speed_mps / 20.0), 0.0, 1.0)
                        
                        final_throttle = 0.0
                        final_brake = float(brake_val)
                        final_steer = float(np.clip(final_steer * 0.5, -0.3, 0.3))
                        
                        if speed_mps < self.stop_speed_threshold:
                            print("🛑 Stopped at red light, waiting for green...")
                            self.state = "WAITING_GREEN"
                    
                    elif self.state == "WAITING_GREEN":
                        final_throttle = 0.0
                        final_brake = 1.0
                        final_steer = 0.0
                    
                    elif self.state == "LANE_CHANGE":
                        final_brake = 0.0
                        final_steer, final_throttle, done = self.execute_lane_change(loc, rot, final_steer, final_throttle)
                        if done:
                            print("   Resuming normal driving with strong lane centering")
                            self.state = "DRIVING"
                    
                    else:  # DRIVING
                        final_brake = 0.0

                    # Apply control to vehicle
                    ctrl.steer = float(final_steer)
                    ctrl.throttle = float(final_throttle)
                    ctrl.brake = float(final_brake)
                    ctrl.hand_brake = False
                    try:
                        self.vehicle.apply_control(ctrl)
                    except Exception:
                        pass

                    # Save frame
                    try:
                        if self._saved_frame_ema is not None:
                            save_img = np.clip(self._saved_frame_ema, 0, 255).astype(np.uint8)
                            save_path = os.path.join(image_dir, f"{frame_count:06d}.png")
                            cv2.imwrite(save_path, save_img)
                    except Exception:
                        pass

                    # Statistics tracking
                    self.deviations.append(abs(lane_error))
                    self.controls_history.append({
                        'frame': frame_count,
                        'model_steer': float(model_steer),
                        'final_steer': float(final_steer),
                        'throttle': float(final_throttle),
                        'speed': float(speed_mps),
                        'lane_error': float(lane_error),
                        'state': self.state
                    })
                    self.trajectory.append({
                        'x': loc.x,
                        'y': loc.y,
                        'z': loc.z,
                        'yaw': rot.yaw
                    })

                    # Logging
                    tstamp = time.time() - self.start_time
                    log_file.write(
                        f"{tstamp:.3f},{frame_count},{loc.x:.2f},{loc.y:.2f},{loc.z:.2f},{rot.yaw:.2f},"
                        f"{speed_mps:.3f},{speed_kmh:.1f},{model_steer:.4f},{final_steer:.4f},"
                        f"{final_throttle:.4f},{lane_error:.4f},{self.state}\n"
                    )
                    log_file.flush()

                    # Console output
                    if time.time() - last_log >= 1.0:
                        tl_msg = "None"
                        if tl_actor is not None and tl_dist is not None:
                            try:
                                st = tl_actor.get_state()
                                tl_msg = f"{st.name} @ {tl_dist:.1f}m"
                            except Exception:
                                tl_msg = f"@{tl_dist:.1f}m"
                        
                        lc_status = ""
                        if self.lane_changes_completed > 0:
                            lc_status = f"[LC: {self.lane_changes_completed}/{self.lane_changes_attempted}]"
                        if self.at_traffic_light and self.state == "DRIVING":
                            lc_status += " @Signal"
                        
                        print(
                            f"[{tstamp:6.1f}s] F{frame_count:4d} | "
                            f"Spd:{speed_kmh:5.1f} | "
                            f"Lane:{lane_error:+.2f}m | "
                            f"Steer:{final_steer:+.2f} | "
                            f"TL:{tl_msg:20s} | "
                            f"{self.state:12s} {lc_status}"
                        )
                        last_log = time.time()

                    frame_count += 1

        except Exception as e:
            print("Evaluation error:", e)
            traceback.print_exc()
        finally:
            try:
                log_file.close()
            except Exception:
                pass
            
            try:
                imgs = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
                if imgs:
                    out_vid = f"eval_video_{int(self.start_time)}.mp4"
                    print(f"\nCreating video from {len(imgs)} frames...")
                    with imageio.get_writer(out_vid, fps=30) as writer:
                        for im in imgs:
                            writer.append_data(imageio.imread(im))
                    print("✓ Saved video:", out_vid)
            except Exception as e:
                print("Video creation warning:", e)

            self.print_statistics(frame_count)
            self.cleanup()

    def print_statistics(self, frame_count):
        elapsed = time.time() - (self.start_time or time.time())
        print("\n" + "="*60)
        print("EVALUATION STATISTICS")
        print("="*60)
        print(f"Total frames: {frame_count}")
        print(f"Total time: {elapsed:.2f} s")
        if elapsed > 0:
            print(f"Average FPS: {frame_count / elapsed:.2f}")
        print(f"Collisions: {self.collision_count}")
        print(f"\nLane Change Performance:")
        print(f"  Attempted: {self.lane_changes_attempted}")
        print(f"  Completed: {self.lane_changes_completed}")
        if self.lane_changes_attempted > 0:
            success_rate = (self.lane_changes_completed / self.lane_changes_attempted) * 100
            print(f"  Success rate: {success_rate:.1f}%")
        print(f"  Left: {self.lane_changes_by_direction['left']}")
        print(f"  Right: {self.lane_changes_by_direction['right']}")
        if self.deviations:
            print(f"\nLane Keeping Performance:")
            print(f"  Average offset: {np.mean(self.deviations):.3f} m")
            print(f"  Max offset: {np.max(self.deviations):.3f} m")
            print(f"  Std deviation: {np.std(self.deviations):.3f} m")
            within_1m = sum(1 for d in self.deviations if d < 1.0) / len(self.deviations) * 100
            within_half = sum(1 for d in self.deviations if d < 0.5) / len(self.deviations) * 100
            within_quarter = sum(1 for d in self.deviations if d < 0.25) / len(self.deviations) * 100
            print(f"  Within 1.0m: {within_1m:.1f}%")
            print(f"  Within 0.5m: {within_half:.1f}%")
            print(f"  Within 0.25m: {within_quarter:.1f}%")
        if self.trajectory:
            distances = []
            for i in range(1, len(self.trajectory)):
                dx = self.trajectory[i]['x'] - self.trajectory[i-1]['x']
                dy = self.trajectory[i]['y'] - self.trajectory[i-1]['y']
                distances.append(math.sqrt(dx**2 + dy**2))
            total_dist = sum(distances)
            print(f"\nTotal distance: {total_dist:.2f} m")
            if elapsed > 0:
                avg_speed_mps = total_dist / elapsed
                print(f"Average speed: {avg_speed_mps * 3.6:.2f} km/h")
        print("="*60)

    def cleanup(self):
        print("\nCleaning up actors and restoring world settings...")
        try:
            if self.world is not None and self.original_settings is not None:
                self.world.apply_settings(self.original_settings)
        except Exception:
            pass
        actors = []
        for a in (self.camera, self.collision_sensor, self.vehicle):
            try:
                if a is not None:
                    try:
                        a.stop()
                    except Exception:
                        pass
                    actors.append(a)
            except Exception:
                pass
        for a in actors:
            try:
                a.destroy()
            except Exception:
                pass
        try:
            gc.collect()
        except Exception:
            pass
        print("✓ Cleanup complete")


def main():
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    evaluator = ClosedLoopEvaluator(
        model_path=MODEL_PATH,
        traffic_light_distance=30.0
    )
    evaluator.run_evaluation()


if __name__ == "__main__":
    main()


STARTING: Signal-Based Lane Changes + Strong Lane Keeping
Spawned vehicle at (-45.16, 92.46)
Using device: cuda
Model loaded: /home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth
Warming sensors...
[   1.0s] F  17 | Spd:  0.0 | Lane:+0.11m | Steer:+0.02 | TL:None                 | DRIVING      
[   2.1s] F  39 | Spd:  7.4 | Lane:+0.11m | Steer:+0.03 | TL:None                 | DRIVING      
[   3.1s] F  60 | Spd: 18.3 | Lane:-0.01m | Steer:+0.02 | TL:None                 | DRIVING      
[   4.1s] F  82 | Spd: 17.9 | Lane:-0.19m | Steer:-0.07 | TL:None                 | DRIVING      
[   5.1s] F 104 | Spd: 18.1 | Lane:+0.11m | Steer:+0.06 | TL:None                 | DRIVING      
[   6.2s] F 126 | Spd: 18.4 | Lane:+0.01m | Steer:+0.01 | TL:None                 | DRIVING      
[   7.2s] F 148 | Spd: 19.0 | Lane:-0.20m | Steer:-0.05 | TL:None                 | DRIVING      
[   8.2s] F 170 | Spd: 18.4 | Lane:+0.01m | Steer:+0.01 | TL:Red @ 29.7m          | DRIVI

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Graceful shutdown requested.

Creating video from 1037 frames...
✓ Saved video: eval_video_1760299963.mp4

EVALUATION STATISTICS
Total frames: 1037
Total time: 53.10 s
Average FPS: 19.53
Collisions: 0

Lane Change Performance:
  Attempted: 2
  Completed: 1
  Success rate: 50.0%
  Left: 0
  Right: 2

Lane Keeping Performance:
  Average offset: 1.569 m
  Max offset: 4.091 m
  Std deviation: 1.090 m
  Within 1.0m: 34.2%
  Within 0.5m: 28.4%
  Within 0.25m: 26.1%

Total distance: 162.06 m
Average speed: 10.99 km/h

Cleaning up actors and restoring world settings...
✓ Cleanup complete


# perfect one with changes  needed

In [3]:
import carla
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import cv2
import gc
import imageio.v2 as imageio
import signal
import math
import traceback

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model (unchanged) -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


def preprocess_image_stack(frame_stack, device):
    if len(frame_stack) != 3:
        return None
    processed_frames = []
    for frame in frame_stack:
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        frame = frame.astype(np.float32) / 255.0
        frame = (frame - 0.5) / 0.5
        processed_frames.append(frame)
    stacked = np.concatenate(processed_frames, axis=2)
    stacked = np.transpose(stacked, (2, 0, 1))
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)
    return tensor.to(device)


# ----------------- Enhanced PID Controller -----------------
class StrongPIDController:
    """Ultra-precise lane keeping PID controller"""
    def __init__(self):
        self.Kp = 1.8  # Very strong proportional
        self.Ki = 0.08  # Moderate integral
        self.Kd = 0.45  # Strong derivative
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        self.integral_limit = 0.8
        self.output_limit = 1.0

    def update(self, error, current_time):
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05
        else:
            dt = max(current_time - self.prev_time, 1e-3)
            self.prev_time = current_time
        
        P = self.Kp * error
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        D = self.Kd * derivative
        self.prev_error = error
        output = P + I + D
        return float(np.clip(output, -self.output_limit, self.output_limit))

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


# ----------------- Waypoint Path Follower -----------------
class WaypointFollower:
    """Follow a pre-computed waypoint path smoothly"""
    def __init__(self):
        self.path = []
        self.current_idx = 0
        self.lookahead_distance = 8.0  # meters
        
    def set_path(self, waypoints):
        """Set new waypoint path"""
        self.path = waypoints
        self.current_idx = 0
        
    def clear_path(self):
        """Clear current path"""
        self.path = []
        self.current_idx = 0
        
    def get_target_point(self, vehicle_location):
        """Get lookahead target point on path"""
        if not self.path:
            return None
            
        # Find closest waypoint
        min_dist = float('inf')
        closest_idx = self.current_idx
        
        for i in range(self.current_idx, min(len(self.path), self.current_idx + 10)):
            wp_loc = self.path[i].transform.location
            dist = math.sqrt(
                (wp_loc.x - vehicle_location.x)**2 + 
                (wp_loc.y - vehicle_location.y)**2
            )
            if dist < min_dist:
                min_dist = dist
                closest_idx = i
        
        self.current_idx = closest_idx
        
        # Find lookahead point
        target_idx = closest_idx
        cumulative_dist = 0.0
        
        for i in range(closest_idx, len(self.path) - 1):
            wp1 = self.path[i].transform.location
            wp2 = self.path[i + 1].transform.location
            segment_dist = math.sqrt(
                (wp2.x - wp1.x)**2 + (wp2.y - wp1.y)**2
            )
            cumulative_dist += segment_dist
            
            if cumulative_dist >= self.lookahead_distance:
                target_idx = i + 1
                break
        else:
            target_idx = len(self.path) - 1
            
        if target_idx >= len(self.path):
            return None
            
        return self.path[target_idx].transform.location
    
    def is_path_complete(self, vehicle_location):
        """Check if we've reached the end of path"""
        if not self.path:
            return True
        
        end_loc = self.path[-1].transform.location
        dist = math.sqrt(
            (end_loc.x - vehicle_location.x)**2 + 
            (end_loc.y - vehicle_location.y)**2
        )
        return dist < 3.0


# ----------------- Main Controller -----------------
class ImprovedLaneChangeController:
    def __init__(self, model_path, host="localhost", port=2000):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        # Controllers
        self.lane_pid = StrongPIDController()
        self.path_follower = WaypointFollower()
        
        # Smoothing
        self.steer_smooth = 0.0
        self.throttle_smooth = 0.3
        self.steer_alpha = 0.25  # Smoother steering
        self.throttle_alpha = 0.20

        # Speed control
        self.target_speed_kmh = 20.0
        self.stop_distance_from_line = 3.5  # Stop 3.5m before traffic light

        # State machine
        self.state = "DRIVING"
        self.stop_start_time = None
        self.green_wait_time = 0.0
        self.lane_change_decision = None  # "left" or "right"
        self.lane_change_path_created = False

        # Stats
        self.collision_count = 0
        self.deviations = []
        self.controls_history = []
        self.trajectory = []
        self.lane_changes_completed = 0

        # CARLA objects
        self.client = None
        self.world = None
        self.map = None
        self.vehicle = None
        self.camera = None
        self.collision_sensor = None
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        self.start_time = None
        self.original_settings = None
        self._saved_frame_ema = None
        self._ema_alpha = 0.25

        signal.signal(signal.SIGINT, self.signal_handler)

    def signal_handler(self, sig, frame):
        print("\nShutdown requested.")
        self.running = False

    def connect_carla(self):
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        self.original_settings = self.world.get_settings()
        self.map = self.world.get_map()

        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)

    def spawn_vehicle(self):
        bp_lib = self.world.get_blueprint_library()
        vehicle_bp = bp_lib.filter("vehicle.tesla.model3")[0] if bp_lib.filter("vehicle.tesla.model3") else random.choice(bp_lib.filter("vehicle.*"))
        spawn_points = self.map.get_spawn_points()
        sp = random.choice(spawn_points)
        sp.location.z += 0.5
        self.vehicle = self.world.spawn_actor(vehicle_bp, sp)
        print(f"Spawned at ({sp.location.x:.2f}, {sp.location.y:.2f})")
        return sp.location

    def setup_sensors(self):
        bp_lib = self.world.get_blueprint_library()
        
        # Camera
        cam_bp = bp_lib.find("sensor.camera.rgb")
        cam_bp.set_attribute("image_size_x", "800")
        cam_bp.set_attribute("image_size_y", "600")
        cam_bp.set_attribute("fov", "100")
        cam_transform = carla.Transform(carla.Location(x=1.3, z=1.8), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)
        self.camera.listen(self.camera_callback)

        # Collision sensor
        col_bp = bp_lib.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(col_bp, carla.Transform(), attach_to=self.vehicle)
        self.collision_sensor.listen(self.collision_callback)

    def camera_callback(self, image):
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]
            small = cv2.resize(arr, (160, 120))
            self.latest_image = small
            self.new_image_available = True

            save_frame = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
            if self._saved_frame_ema is None:
                self._saved_frame_ema = save_frame.astype(np.float32)
            else:
                self._saved_frame_ema = (self._ema_alpha * save_frame.astype(np.float32) +
                                         (1 - self._ema_alpha) * self._saved_frame_ema)
        except Exception:
            pass

    def collision_callback(self, event):
        self.collision_count += 1
        print(f"\n*** COLLISION #{self.collision_count} ***")
        self.collision_occurred = True
        
        # Emergency stop and reset
        self.path_follower.clear_path()
        self.lane_pid.reset()
        self.state = "COLLISION_RECOVERY"
        
        try:
            ctrl = carla.VehicleControl()
            ctrl.steer = 0.0
            ctrl.throttle = 0.0
            ctrl.brake = 1.0
            self.vehicle.apply_control(ctrl)
        except Exception:
            pass
        
        time.sleep(1.5)
        self.collision_occurred = False
        self.state = "DRIVING"

    def load_model(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Device:", self.device)
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device)
        
        loaded = torch.load(self.model_path, map_location=self.device, weights_only=False)
        state = loaded['state_dict'] if isinstance(loaded, dict) and 'state_dict' in loaded else loaded
        
        try:
            self.model.load_state_dict(state)
        except RuntimeError:
            new_state = {k[7:] if k.startswith("module.") else k: v for k, v in state.items()}
            self.model.load_state_dict(new_state)
        
        print("Model loaded")
        self.model.eval()

    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        """Calculate cross-track and heading error"""
        try:
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            if waypoint is None:
                return 0.0
            
            # Cross-track error
            lane_center = waypoint.transform.location
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            cross_track = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            
            # Heading error
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            
            # Combined error with strong heading weight
            total_error = cross_track + 1.2 * heading_error
            return total_error
        except Exception:
            return 0.0

    def find_traffic_light_ahead(self):
        """Find nearest traffic light ahead of vehicle"""
        if self.vehicle is None:
            return None, None
        
        try:
            veh_tf = self.vehicle.get_transform()
            veh_loc = veh_tf.location
            veh_fwd = veh_tf.get_forward_vector()
            
            lights = self.world.get_actors().filter("traffic.traffic_light*")
            best_light = None
            best_dist = None
            
            for tl in lights:
                tl_loc = tl.get_transform().location
                vec = np.array([tl_loc.x - veh_loc.x, tl_loc.y - veh_loc.y, tl_loc.z - veh_loc.z])
                dist = np.linalg.norm(vec)
                
                if dist > 40.0 or dist < 0.5:
                    continue
                
                # Check if ahead
                vec_norm = vec / (dist + 1e-6)
                dot = vec_norm[0] * veh_fwd.x + vec_norm[1] * veh_fwd.y + vec_norm[2] * veh_fwd.z
                
                if dot > 0.7:  # Ahead of us
                    if best_light is None or dist < best_dist:
                        best_light = tl
                        best_dist = dist
            
            return best_light, best_dist
        except Exception:
            return None, None

    def check_lane_availability(self, current_wp, direction):
        """Check if lane change is safe and valid"""
        try:
            # Get target lane
            if direction == "left":
                target_wp = current_wp.get_left_lane()
            else:
                target_wp = current_wp.get_right_lane()
            
            if target_wp is None:
                return False
            
            # Must be driving lane
            if target_wp.lane_type != carla.LaneType.Driving:
                return False
            
            # Must be same direction (check lane_id sign)
            if (current_wp.lane_id > 0) != (target_wp.lane_id > 0):
                return False
            
            # Check path ahead is valid
            test_wp = target_wp
            valid_distance = 0.0
            for _ in range(30):
                nxts = test_wp.next(2.0)
                if not nxts:
                    break
                test_wp = nxts[0]
                if test_wp.lane_type != carla.LaneType.Driving:
                    break
                valid_distance += 2.0
            
            if valid_distance < 40.0:  # Need good path ahead
                return False
            
            # Check for vehicles in target lane
            veh_loc = self.vehicle.get_location()
            for other in self.world.get_actors().filter("vehicle.*"):
                if other.id == self.vehicle.id:
                    continue
                
                other_loc = other.get_location()
                dist = math.sqrt(
                    (other_loc.x - veh_loc.x)**2 + 
                    (other_loc.y - veh_loc.y)**2
                )
                
                if dist < 25.0:  # Check nearby vehicles
                    try:
                        other_wp = self.map.get_waypoint(other_loc, project_to_road=True)
                        if other_wp and other_wp.lane_id == target_wp.lane_id:
                            return False  # Vehicle in target lane
                    except Exception:
                        pass
            
            return True
            
        except Exception as e:
            print(f"Lane check error: {e}")
            return False

    def create_lane_change_path(self, direction):
        """Create smooth waypoint path for lane change"""
        try:
            loc = self.vehicle.get_location()
            current_wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
            
            if current_wp is None:
                return []
            
            # Get target lane
            if direction == "left":
                target_wp = current_wp.get_left_lane()
            else:
                target_wp = current_wp.get_right_lane()
            
            if target_wp is None:
                return []
            
            # Build path: current -> transition -> target
            path = []
            
            # Phase 1: Move forward in current lane (10m)
            wp = current_wp
            for _ in range(5):
                nxts = wp.next(2.0)
                if nxts:
                    wp = nxts[0]
                    path.append(wp)
            
            # Phase 2: Transition to target lane (20m smooth curve)
            start_wp = wp
            
            # Get corresponding point in target lane ahead
            end_wp = target_wp
            for _ in range(15):
                nxts = end_wp.next(2.0)
                if nxts:
                    end_wp = nxts[0]
            
            # Create interpolated waypoints
            for i in range(1, 11):
                t = i / 10.0
                # Smooth interpolation
                interp_loc = carla.Location(
                    x=start_wp.transform.location.x * (1 - t) + end_wp.transform.location.x * t,
                    y=start_wp.transform.location.y * (1 - t) + end_wp.transform.location.y * t,
                    z=start_wp.transform.location.z * (1 - t) + end_wp.transform.location.z * t
                )
                interp_wp = self.map.get_waypoint(interp_loc, project_to_road=True)
                if interp_wp:
                    path.append(interp_wp)
            
            # Phase 3: Continue in target lane (20m)
            wp = end_wp
            for _ in range(10):
                nxts = wp.next(2.0)
                if nxts:
                    wp = nxts[0]
                    path.append(wp)
            
            return path
            
        except Exception as e:
            print(f"Path creation error: {e}")
            return []

    def calculate_steering_to_point(self, vehicle_loc, vehicle_rot, target_point):
        """Pure pursuit steering to target point"""
        # Vector to target
        dx = target_point.x - vehicle_loc.x
        dy = target_point.y - vehicle_loc.y
        
        # Transform to vehicle frame
        yaw = math.radians(vehicle_rot.yaw)
        local_x = dx * math.cos(yaw) + dy * math.sin(yaw)
        local_y = -dx * math.sin(yaw) + dy * math.cos(yaw)
        
        # Steering angle
        if abs(local_x) < 0.1:
            return 0.0
        
        steering_angle = math.atan2(local_y, local_x)
        
        # Normalize to [-1, 1]
        steer_cmd = steering_angle / math.radians(30)  # Max 30 degrees
        return float(np.clip(steer_cmd, -1.0, 1.0))

    def adaptive_speed_control(self, current_speed_mps, lane_error, steering_mag):
        """Adaptive throttle based on conditions"""
        target_speed_mps = self.target_speed_kmh / 3.6
        
        # Reduce speed with lane error
        lane_factor = max(0.7, 1.0 - abs(lane_error) / 3.0)
        
        # Reduce speed with steering
        steer_factor = max(0.75, 1.0 - abs(steering_mag) / 0.6)
        
        adjusted_target = target_speed_mps * lane_factor * steer_factor
        speed_error = adjusted_target - current_speed_mps
        
        if speed_error > 0:
            throttle = min(0.7, 0.4 + speed_error * 0.3)
        else:
            throttle = max(0.0, 0.35 + speed_error * 0.25)
        
        return float(np.clip(throttle, 0.0, 0.8))

    def run_evaluation(self):
        print("\n" + "="*70)
        print("SMOOTH LANE CHANGE WITH PRECISE STOPPING")
        print("="*70)
        
        try:
            self.connect_carla()
            self.spawn_vehicle()
            self.setup_sensors()
            self.load_model()
        except Exception as e:
            print("Init failed:", e)
            traceback.print_exc()
            return

        spectator = self.world.get_spectator()
        ctrl = carla.VehicleControl()

        print("Warming up sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)

        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log = time.time()

        # Logging
        log_filename = f"eval_log_{int(self.start_time)}.csv"
        log_file = open(log_filename, "w")
        log_file.write("timestamp,frame,x,y,z,yaw,speed_kmh,final_steer,throttle,lane_error,state\n")

        image_dir = f"eval_images_{int(self.start_time)}"
        os.makedirs(image_dir, exist_ok=True)

        try:
            while self.running:
                self.world.tick()

                # Update spectator
                try:
                    tf = self.vehicle.get_transform()
                    spec_loc = tf.location + tf.get_forward_vector() * -8 + carla.Location(z=4.0)
                    spec_rot = carla.Rotation(pitch=-15, yaw=tf.rotation.yaw)
                    spectator.set_transform(carla.Transform(spec_loc, spec_rot))
                except Exception:
                    pass

                # Collect frames
                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False

                # Get traffic light info
                tl_actor, tl_dist = self.find_traffic_light_ahead()
                tl_state = None
                if tl_actor is not None:
                    try:
                        tl_state = tl_actor.get_state()
                    except Exception:
                        pass

                # Main control loop
                if len(frame_stack) == 3:
                    # Get vehicle state
                    tf = self.vehicle.get_transform()
                    loc = tf.location
                    rot = tf.rotation
                    vel = self.vehicle.get_velocity()
                    speed_mps = math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
                    speed_kmh = speed_mps * 3.6

                    # Lane error
                    lane_error = self.calculate_lane_error(loc, rot)

                    # Get model prediction
                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    if input_tensor is None:
                        continue

                    speed_norm = min(speed_mps / 30.0, 1.0)
                    speed_tensor = torch.tensor([[speed_norm]], dtype=torch.float32, device=self.device)

                    with torch.no_grad():
                        output = self.model(input_tensor, aux=speed_tensor)
                    model_steer, model_throttle, model_brake = output[0].cpu().numpy()

                    # Initialize control
                    final_steer = 0.0
                    final_throttle = 0.0
                    final_brake = 0.0

                    # ============ STATE MACHINE ============
                    
                    if self.state == "DRIVING":
                        # Check for red light
                        if (tl_actor is not None and tl_state == carla.TrafficLightState.Red and 
                            tl_dist is not None and tl_dist <= 25.0):
                            print(f"🔴 Red light @ {tl_dist:.1f}m -> STOPPING")
                            self.state = "APPROACHING_STOP"
                            self.stop_start_time = time.time()
                        else:
                            # Normal lane keeping with strong PID
                            now = time.time()
                            pid_correction = self.lane_pid.update(lane_error, now)
                            
                            # Blend model and PID (heavy PID weight)
                            error_weight = min(abs(lane_error) / 0.5, 0.90)
                            combined_steer = (1 - error_weight) * model_steer + error_weight * pid_correction
                            
                            # Smooth steering
                            final_steer = self.steer_alpha * combined_steer + (1 - self.steer_alpha) * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.7, 0.7))
                            
                            # Adaptive throttle
                            final_throttle = self.adaptive_speed_control(speed_mps, lane_error, abs(final_steer))
                            final_brake = 0.0
                    
                    elif self.state == "APPROACHING_STOP":
                        # Smooth braking to stop at correct position
                        if tl_dist is not None:
                            # Calculate brake based on distance and speed
                            stop_dist = tl_dist - self.stop_distance_from_line
                            
                            if stop_dist <= 0.5 or speed_mps < 0.3:
                                # Arrived at stop line
                                print("🛑 Stopped at line, waiting for green...")
                                self.state = "WAITING_GREEN"
                                self.green_wait_time = 0.0
                                self.lane_pid.reset()
                                final_steer = 0.0
                                final_throttle = 0.0
                                final_brake = 1.0
                            else:
                                # Proportional braking
                                brake_strength = min(1.0, 0.4 + (speed_mps / 15.0) + (1.0 / max(stop_dist, 1.0)))
                                final_brake = float(np.clip(brake_strength, 0.3, 1.0))
                                final_throttle = 0.0
                                
                                # Gentle steering to maintain lane while braking
                                now = time.time()
                                pid_correction = self.lane_pid.update(lane_error, now)
                                final_steer = 0.5 * pid_correction  # Gentle correction
                                final_steer = float(np.clip(final_steer, -0.3, 0.3))
                        else:
                            # Lost traffic light, stop anyway
                            final_brake = 0.8
                            final_throttle = 0.0
                            final_steer = 0.0
                    
                    elif self.state == "WAITING_GREEN":
                        # Wait at stop line
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                        
                        # Check for green light
                        if tl_actor is not None and tl_state == carla.TrafficLightState.Green:
                            self.green_wait_time += 0.05  # One tick
                            
                            if self.green_wait_time > 0.5:  # Wait 0.5s after green
                                print("🟢 Green light -> Deciding lane change...")
                                
                                # Decide lane change direction
                                loc = self.vehicle.get_location()
                                current_wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
                                
                                if current_wp:
                                    # Try random direction first
                                    directions = ["left", "right"] if random.random() < 0.5 else ["right", "left"]
                                    
                                    for direction in directions:
                                        if self.check_lane_availability(current_wp, direction):
                                            print(f"  ✓ {direction.upper()} lane is available")
                                            self.lane_change_decision = direction
                                            self.state = "INITIATING_LANE_CHANGE"
                                            break
                                    else:
                                        print("  ✗ No safe lane, going straight")
                                        self.state = "DRIVING"
                                        self.lane_pid.reset()
                                else:
                                    print("  ✗ Cannot get waypoint, going straight")
                                    self.state = "DRIVING"
                                    self.lane_pid.reset()
                    
                    elif self.state == "INITIATING_LANE_CHANGE":
                        # Create lane change path
                        if not self.lane_change_path_created:
                            path = self.create_lane_change_path(self.lane_change_decision)
                            
                            if path:
                                self.path_follower.set_path(path)
                                self.lane_change_path_created = True
                                print(f"  🔄 Lane change path created ({len(path)} waypoints)")
                                self.state = "EXECUTING_LANE_CHANGE"
                            else:
                                print("  ✗ Failed to create path, aborting")
                                self.state = "DRIVING"
                                self.lane_pid.reset()
                        
                        # While creating, stay stopped
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                    
                    elif self.state == "EXECUTING_LANE_CHANGE":
                        # Follow the waypoint path
                        target_point = self.path_follower.get_target_point(loc)
                        
                        if target_point is not None:
                            # Calculate steering to target
                            final_steer = self.calculate_steering_to_point(loc, rot, target_point)
                            
                            # Smooth steering
                            final_steer = self.steer_alpha * final_steer + (1 - self.steer_alpha) * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.6, 0.6))
                            
                            # Moderate speed during lane change
                            target_lc_speed = 15.0  # km/h
                            lc_speed_mps = target_lc_speed / 3.6
                            speed_error = lc_speed_mps - speed_mps
                            
                            if speed_error > 0:
                                final_throttle = min(0.5, 0.3 + speed_error * 0.2)
                            else:
                                final_throttle = 0.2
                            
                            final_brake = 0.0
                            
                            # Check if path complete
                            if self.path_follower.is_path_complete(loc):
                                print(f"  ✅ Lane change to {self.lane_change_decision.upper()} COMPLETE!")
                                self.lane_changes_completed += 1
                                self.path_follower.clear_path()
                                self.lane_change_path_created = False
                                self.lane_change_decision = None
                                self.state = "STABILIZING"
                                self.stop_start_time = time.time()
                        else:
                            # Lost path, abort
                            print("  ⚠️ Lost path target, aborting")
                            self.path_follower.clear_path()
                            self.lane_change_path_created = False
                            self.lane_change_decision = None
                            self.state = "DRIVING"
                            self.lane_pid.reset()
                            final_steer = 0.0
                            final_throttle = 0.0
                            final_brake = 0.3
                    
                    elif self.state == "STABILIZING":
                        # After lane change, stabilize in new lane
                        elapsed = time.time() - self.stop_start_time
                        
                        if elapsed < 2.0:
                            # Strong PID to center in lane
                            now = time.time()
                            pid_correction = self.lane_pid.update(lane_error, now)
                            
                            final_steer = pid_correction
                            final_steer = float(np.clip(final_steer, -0.5, 0.5))
                            
                            # Gentle acceleration
                            final_throttle = min(0.4, 0.15 + elapsed * 0.1)
                            final_brake = 0.0
                        else:
                            print("  📍 Stabilized in new lane, resuming normal driving")
                            self.state = "DRIVING"
                            self.lane_pid.reset()
                    
                    elif self.state == "COLLISION_RECOVERY":
                        # Recovery from collision
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0

                    # ============ APPLY CONTROL ============
                    
                    # Apply smoothing to steering
                    self.steer_smooth = final_steer
                    
                    # Apply smoothing to throttle
                    self.throttle_smooth = final_throttle
                    
                    # Apply to vehicle
                    ctrl.steer = float(final_steer)
                    ctrl.throttle = float(final_throttle)
                    ctrl.brake = float(final_brake)
                    ctrl.hand_brake = False
                    
                    try:
                        self.vehicle.apply_control(ctrl)
                    except Exception:
                        pass

                    # ============ LOGGING & RECORDING ============
                    
                    # Save frame
                    try:
                        if self._saved_frame_ema is not None:
                            save_img = np.clip(self._saved_frame_ema, 0, 255).astype(np.uint8)
                            save_path = os.path.join(image_dir, f"{frame_count:06d}.png")
                            cv2.imwrite(save_path, save_img)
                    except Exception:
                        pass

                    # Statistics
                    self.deviations.append(abs(lane_error))
                    self.controls_history.append({
                        'frame': frame_count,
                        'final_steer': float(final_steer),
                        'throttle': float(final_throttle),
                        'speed': float(speed_mps),
                        'lane_error': float(lane_error),
                        'state': self.state
                    })
                    self.trajectory.append({
                        'x': loc.x,
                        'y': loc.y,
                        'z': loc.z,
                        'yaw': rot.yaw
                    })

                    # Log to CSV
                    tstamp = time.time() - self.start_time
                    log_file.write(
                        f"{tstamp:.3f},{frame_count},{loc.x:.2f},{loc.y:.2f},{loc.z:.2f},"
                        f"{rot.yaw:.2f},{speed_kmh:.1f},{final_steer:.4f},{final_throttle:.4f},"
                        f"{lane_error:.4f},{self.state}\n"
                    )
                    log_file.flush()

                    # Console output
                    if time.time() - last_log >= 1.0:
                        tl_msg = "None"
                        if tl_actor is not None and tl_dist is not None:
                            try:
                                st = tl_actor.get_state()
                                tl_msg = f"{st.name[:3]} @ {tl_dist:.1f}m"
                            except Exception:
                                tl_msg = f"@ {tl_dist:.1f}m"
                        
                        lc_info = f"[LC: {self.lane_changes_completed}]" if self.lane_changes_completed > 0 else ""
                        
                        print(
                            f"[{tstamp:6.1f}s] F{frame_count:4d} | "
                            f"Speed:{speed_kmh:5.1f} | "
                            f"Lane:{lane_error:+.3f}m | "
                            f"Steer:{final_steer:+.3f} | "
                            f"TL:{tl_msg:15s} | "
                            f"{self.state:22s} {lc_info}"
                        )
                        last_log = time.time()

                    frame_count += 1

        except Exception as e:
            print("Evaluation error:", e)
            traceback.print_exc()
        finally:
            try:
                log_file.close()
            except Exception:
                pass
            
            # Create video
            try:
                imgs = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
                if imgs:
                    out_vid = f"eval_video_{int(self.start_time)}.mp4"
                    print(f"\nCreating video from {len(imgs)} frames...")
                    with imageio.get_writer(out_vid, fps=20) as writer:
                        for im in imgs:
                            writer.append_data(imageio.imread(im))
                    print("✓ Video saved:", out_vid)
            except Exception as e:
                print("Video creation warning:", e)

            self.print_statistics(frame_count)
            self.cleanup()

    def print_statistics(self, frame_count):
        elapsed = time.time() - (self.start_time or time.time())
        print("\n" + "="*70)
        print("EVALUATION STATISTICS")
        print("="*70)
        print(f"Duration: {elapsed:.2f} s")
        print(f"Frames: {frame_count}")
        if elapsed > 0:
            print(f"FPS: {frame_count / elapsed:.2f}")
        print(f"Collisions: {self.collision_count}")
        print(f"\nLane Changes:")
        print(f"  Completed: {self.lane_changes_completed}")
        
        if self.deviations:
            print(f"\nLane Keeping:")
            print(f"  Mean error: {np.mean(self.deviations):.3f} m")
            print(f"  Max error: {np.max(self.deviations):.3f} m")
            print(f"  Std dev: {np.std(self.deviations):.3f} m")
            within_half = sum(1 for d in self.deviations if d < 0.5) / len(self.deviations) * 100
            within_quarter = sum(1 for d in self.deviations if d < 0.25) / len(self.deviations) * 100
            print(f"  Within 0.5m: {within_half:.1f}%")
            print(f"  Within 0.25m: {within_quarter:.1f}%")
        
        if self.trajectory:
            distances = []
            for i in range(1, len(self.trajectory)):
                dx = self.trajectory[i]['x'] - self.trajectory[i-1]['x']
                dy = self.trajectory[i]['y'] - self.trajectory[i-1]['y']
                distances.append(math.sqrt(dx**2 + dy**2))
            total_dist = sum(distances)
            print(f"\nDistance traveled: {total_dist:.2f} m")
            if elapsed > 0:
                avg_speed = (total_dist / elapsed) * 3.6
                print(f"Average speed: {avg_speed:.2f} km/h")
        print("="*70)

    def cleanup(self):
        print("\nCleaning up...")
        try:
            if self.world is not None and self.original_settings is not None:
                self.world.apply_settings(self.original_settings)
        except Exception:
            pass
        
        actors = []
        for a in (self.camera, self.collision_sensor, self.vehicle):
            if a is not None:
                try:
                    a.stop()
                except Exception:
                    pass
                actors.append(a)
        
        for a in actors:
            try:
                a.destroy()
            except Exception:
                pass
        
        gc.collect()
        print("✓ Cleanup complete")


def main():
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    
    controller = ImprovedLaneChangeController(
        model_path=MODEL_PATH,
        host="localhost",
        port=2000
    )
    
    controller.run_evaluation()


if __name__ == "__main__":
    main()


SMOOTH LANE CHANGE WITH PRECISE STOPPING
Spawned at (-110.76, 46.66)
Device: cuda
Model loaded
Warming up sensors...
[   1.0s] F  32 | Speed:  0.0 | Lane:+0.000m | Steer:+0.016 | TL:None            | DRIVING                
[   2.0s] F  67 | Speed: 18.4 | Lane:-0.115m | Steer:-0.025 | TL:None            | DRIVING                
[   3.0s] F 102 | Speed: 18.0 | Lane:+0.112m | Steer:+0.057 | TL:None            | DRIVING                
[   4.0s] F 137 | Speed: 18.1 | Lane:-0.176m | Steer:-0.117 | TL:None            | DRIVING                
[   5.1s] F 172 | Speed: 17.5 | Lane:+0.006m | Steer:+0.050 | TL:None            | DRIVING                
[   6.1s] F 206 | Speed: 16.8 | Lane:+0.246m | Steer:+0.270 | TL:None            | DRIVING                
[   7.1s] F 240 | Speed: 15.5 | Lane:-0.093m | Steer:-0.007 | TL:None            | DRIVING                
[   8.1s] F 275 | Speed: 16.1 | Lane:-0.345m | Steer:-0.500 | TL:None            | DRIVING                
[   9.1s] F 309 | Speed: 1

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Shutdown requested.

Creating video from 593 frames...
✓ Video saved: eval_video_1768080980.mp4

EVALUATION STATISTICS
Duration: 21.66 s
Frames: 593
FPS: 27.38
Collisions: 0

Lane Changes:
  Completed: 0

Lane Keeping:
  Mean error: 0.220 m
  Max error: 0.559 m
  Std dev: 0.161 m
  Within 0.5m: 98.8%
  Within 0.25m: 56.8%

Distance traveled: 101.05 m
Average speed: 16.80 km/h

Cleaning up...
✓ Cleanup complete


In [2]:
import carla
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import cv2
import gc
import imageio.v2 as imageio
import signal
import math
import traceback

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model (unchanged) -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


def preprocess_image_stack(frame_stack, device):
    if len(frame_stack) != 3:
        return None
    processed_frames = []
    for frame in frame_stack:
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        frame = frame.astype(np.float32) / 255.0
        frame = (frame - 0.5) / 0.5
        processed_frames.append(frame)
    stacked = np.concatenate(processed_frames, axis=2)
    stacked = np.transpose(stacked, (2, 0, 1))
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)
    return tensor.to(device)


# ----------------- Enhanced PID Controller -----------------
class StrongPIDController:
    """Smooth and stable PID controller"""
    def __init__(self):
        # MUCH GENTLER gains - eliminates oscillations
        self.Kp = 0.55  # Reduced from 1.8
        self.Ki = 0.015  # Reduced from 0.08
        self.Kd = 0.12  # Reduced from 0.45
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        self.integral_limit = 1.5  # Increased from 0.8
        self.output_limit = 0.8  # Reduced from 1.0

    def update(self, error, current_time):
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05
        else:
            dt = max(current_time - self.prev_time, 1e-3)
            self.prev_time = current_time
        
        P = self.Kp * error
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        D = self.Kd * derivative
        self.prev_error = error
        output = P + I + D
        return float(np.clip(output, -self.output_limit, self.output_limit))

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


# ----------------- Waypoint Path Follower -----------------
class WaypointFollower:
    """Follow a pre-computed waypoint path smoothly"""
    def __init__(self):
        self.path = []
        self.current_idx = 0
        self.lookahead_distance = 8.0  # meters
        
    def set_path(self, waypoints):
        """Set new waypoint path"""
        self.path = waypoints
        self.current_idx = 0
        
    def clear_path(self):
        """Clear current path"""
        self.path = []
        self.current_idx = 0
        
    def get_target_point(self, vehicle_location):
        """Get lookahead target point on path"""
        if not self.path:
            return None
            
        # Find closest waypoint
        min_dist = float('inf')
        closest_idx = self.current_idx
        
        for i in range(self.current_idx, min(len(self.path), self.current_idx + 10)):
            wp_loc = self.path[i].transform.location
            dist = math.sqrt(
                (wp_loc.x - vehicle_location.x)**2 + 
                (wp_loc.y - vehicle_location.y)**2
            )
            if dist < min_dist:
                min_dist = dist
                closest_idx = i
        
        self.current_idx = closest_idx
        
        # Find lookahead point
        target_idx = closest_idx
        cumulative_dist = 0.0
        
        for i in range(closest_idx, len(self.path) - 1):
            wp1 = self.path[i].transform.location
            wp2 = self.path[i + 1].transform.location
            segment_dist = math.sqrt(
                (wp2.x - wp1.x)**2 + (wp2.y - wp1.y)**2
            )
            cumulative_dist += segment_dist
            
            if cumulative_dist >= self.lookahead_distance:
                target_idx = i + 1
                break
        else:
            target_idx = len(self.path) - 1
            
        if target_idx >= len(self.path):
            return None
            
        return self.path[target_idx].transform.location
    
    def is_path_complete(self, vehicle_location):
        """Check if we've reached the end of path"""
        if not self.path:
            return True
        
        end_loc = self.path[-1].transform.location
        dist = math.sqrt(
            (end_loc.x - vehicle_location.x)**2 + 
            (end_loc.y - vehicle_location.y)**2
        )
        return dist < 3.0


# ----------------- Main Controller -----------------
class ImprovedLaneChangeController:
    def __init__(self, model_path, host="localhost", port=2000):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        # Controllers
        self.lane_pid = StrongPIDController()
        self.path_follower = WaypointFollower()
        
        # Smoothing - MUCH MORE AGGRESSIVE
        self.steer_smooth = 0.0
        self.throttle_smooth = 0.3
        self.steer_alpha = 0.12  # Changed from 0.25 - MUCH smoother
        self.throttle_alpha = 0.20

        # Speed control
        self.target_speed_kmh = 20.0
        self.stop_distance_from_line = 2.8  # Changed from 3.5 - closer to line

        # State machine
        self.state = "DRIVING"
        self.stop_start_time = None
        self.green_wait_time = 0.0
        self.lane_change_decision = None  # "left" or "right"
        self.lane_change_path_created = False

        # Stats
        self.collision_count = 0
        self.deviations = []
        self.controls_history = []
        self.trajectory = []
        self.lane_changes_completed = 0

        # CARLA objects
        self.client = None
        self.world = None
        self.map = None
        self.vehicle = None
        self.camera = None
        self.collision_sensor = None
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        self.start_time = None
        self.original_settings = None
        self._saved_frame_ema = None
        self._ema_alpha = 0.25

        signal.signal(signal.SIGINT, self.signal_handler)

    def signal_handler(self, sig, frame):
        print("\nShutdown requested.")
        self.running = False

    def connect_carla(self):
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        self.original_settings = self.world.get_settings()
        self.map = self.world.get_map()

        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)

    def spawn_vehicle(self):
        bp_lib = self.world.get_blueprint_library()
        vehicle_bp = bp_lib.filter("vehicle.tesla.model3")[0] if bp_lib.filter("vehicle.tesla.model3") else random.choice(bp_lib.filter("vehicle.*"))
        spawn_points = self.map.get_spawn_points()
        sp = random.choice(spawn_points)
        sp.location.z += 0.5
        self.vehicle = self.world.spawn_actor(vehicle_bp, sp)
        print(f"Spawned at ({sp.location.x:.2f}, {sp.location.y:.2f})")
        return sp.location

    def setup_sensors(self):
        bp_lib = self.world.get_blueprint_library()
        
        # Camera
        cam_bp = bp_lib.find("sensor.camera.rgb")
        cam_bp.set_attribute("image_size_x", "800")
        cam_bp.set_attribute("image_size_y", "600")
        cam_bp.set_attribute("fov", "100")
        cam_transform = carla.Transform(carla.Location(x=1.3, z=1.8), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)
        self.camera.listen(self.camera_callback)

        # Collision sensor
        col_bp = bp_lib.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(col_bp, carla.Transform(), attach_to=self.vehicle)
        self.collision_sensor.listen(self.collision_callback)

    def camera_callback(self, image):
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]
            small = cv2.resize(arr, (160, 120))
            self.latest_image = small
            self.new_image_available = True

            save_frame = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
            if self._saved_frame_ema is None:
                self._saved_frame_ema = save_frame.astype(np.float32)
            else:
                self._saved_frame_ema = (self._ema_alpha * save_frame.astype(np.float32) +
                                         (1 - self._ema_alpha) * self._saved_frame_ema)
        except Exception:
            pass

    def collision_callback(self, event):
        self.collision_count += 1
        print(f"\n*** COLLISION #{self.collision_count} ***")
        self.collision_occurred = True
        
        # Emergency stop and reset
        self.path_follower.clear_path()
        self.lane_pid.reset()
        self.state = "COLLISION_RECOVERY"
        
        try:
            ctrl = carla.VehicleControl()
            ctrl.steer = 0.0
            ctrl.throttle = 0.0
            ctrl.brake = 1.0
            self.vehicle.apply_control(ctrl)
        except Exception:
            pass
        
        time.sleep(1.5)
        self.collision_occurred = False
        self.state = "DRIVING"

    def load_model(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Device:", self.device)
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device)
        
        loaded = torch.load(self.model_path, map_location=self.device, weights_only=False)
        state = loaded['state_dict'] if isinstance(loaded, dict) and 'state_dict' in loaded else loaded
        
        try:
            self.model.load_state_dict(state)
        except RuntimeError:
            new_state = {k[7:] if k.startswith("module.") else k: v for k, v in state.items()}
            self.model.load_state_dict(new_state)
        
        print("Model loaded")
        self.model.eval()

    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        """Calculate cross-track and heading error"""
        try:
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            if waypoint is None:
                return 0.0
            
            # Cross-track error
            lane_center = waypoint.transform.location
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            cross_track = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            
            # Heading error
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            
            # Combined error with GENTLE heading weight (changed from 1.2)
            total_error = cross_track + 0.5 * heading_error
            return total_error
        except Exception:
            return 0.0

    def find_traffic_light_ahead(self):
        """Find nearest traffic light ahead of vehicle"""
        if self.vehicle is None:
            return None, None
        
        try:
            veh_tf = self.vehicle.get_transform()
            veh_loc = veh_tf.location
            veh_fwd = veh_tf.get_forward_vector()
            
            lights = self.world.get_actors().filter("traffic.traffic_light*")
            best_light = None
            best_dist = None
            
            for tl in lights:
                tl_loc = tl.get_transform().location
                vec = np.array([tl_loc.x - veh_loc.x, tl_loc.y - veh_loc.y, tl_loc.z - veh_loc.z])
                dist = np.linalg.norm(vec)
                
                if dist > 40.0 or dist < 0.5:
                    continue
                
                # Check if ahead
                vec_norm = vec / (dist + 1e-6)
                dot = vec_norm[0] * veh_fwd.x + vec_norm[1] * veh_fwd.y + vec_norm[2] * veh_fwd.z
                
                if dot > 0.7:  # Ahead of us
                    if best_light is None or dist < best_dist:
                        best_light = tl
                        best_dist = dist
            
            return best_light, best_dist
        except Exception:
            return None, None

    def check_lane_availability(self, current_wp, direction):
        """Check if lane change is safe and valid"""
        try:
            # Get target lane
            if direction == "left":
                target_wp = current_wp.get_left_lane()
            else:
                target_wp = current_wp.get_right_lane()
            
            if target_wp is None:
                return False
            
            # Must be driving lane
            if target_wp.lane_type != carla.LaneType.Driving:
                return False
            
            # Must be same direction (check lane_id sign)
            if (current_wp.lane_id > 0) != (target_wp.lane_id > 0):
                return False
            
            # Check path ahead is valid
            test_wp = target_wp
            valid_distance = 0.0
            for _ in range(30):
                nxts = test_wp.next(2.0)
                if not nxts:
                    break
                test_wp = nxts[0]
                if test_wp.lane_type != carla.LaneType.Driving:
                    break
                valid_distance += 2.0
            
            if valid_distance < 40.0:  # Need good path ahead
                return False
            
            # Check for vehicles in target lane
            veh_loc = self.vehicle.get_location()
            for other in self.world.get_actors().filter("vehicle.*"):
                if other.id == self.vehicle.id:
                    continue
                
                other_loc = other.get_location()
                dist = math.sqrt(
                    (other_loc.x - veh_loc.x)**2 + 
                    (other_loc.y - veh_loc.y)**2
                )
                
                if dist < 25.0:  # Check nearby vehicles
                    try:
                        other_wp = self.map.get_waypoint(other_loc, project_to_road=True)
                        if other_wp and other_wp.lane_id == target_wp.lane_id:
                            return False  # Vehicle in target lane
                    except Exception:
                        pass
            
            return True
            
        except Exception as e:
            print(f"Lane check error: {e}")
            return False

    def create_lane_change_path(self, direction):
        """Create smooth waypoint path for lane change"""
        try:
            loc = self.vehicle.get_location()
            current_wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
            
            if current_wp is None:
                return []
            
            # Get target lane
            if direction == "left":
                target_wp = current_wp.get_left_lane()
            else:
                target_wp = current_wp.get_right_lane()
            
            if target_wp is None:
                return []
            
            # Build path: current -> transition -> target
            path = []
            
            # Phase 1: Move forward in current lane (10m)
            wp = current_wp
            for _ in range(5):
                nxts = wp.next(2.0)
                if nxts:
                    wp = nxts[0]
                    path.append(wp)
            
            # Phase 2: Transition to target lane (20m smooth curve)
            start_wp = wp
            
            # Get corresponding point in target lane ahead
            end_wp = target_wp
            for _ in range(15):
                nxts = end_wp.next(2.0)
                if nxts:
                    end_wp = nxts[0]
            
            # Create interpolated waypoints
            for i in range(1, 11):
                t = i / 10.0
                # Smooth interpolation
                interp_loc = carla.Location(
                    x=start_wp.transform.location.x * (1 - t) + end_wp.transform.location.x * t,
                    y=start_wp.transform.location.y * (1 - t) + end_wp.transform.location.y * t,
                    z=start_wp.transform.location.z * (1 - t) + end_wp.transform.location.z * t
                )
                interp_wp = self.map.get_waypoint(interp_loc, project_to_road=True)
                if interp_wp:
                    path.append(interp_wp)
            
            # Phase 3: Continue in target lane (20m)
            wp = end_wp
            for _ in range(10):
                nxts = wp.next(2.0)
                if nxts:
                    wp = nxts[0]
                    path.append(wp)
            
            return path
            
        except Exception as e:
            print(f"Path creation error: {e}")
            return []

    def calculate_steering_to_point(self, vehicle_loc, vehicle_rot, target_point):
        """Pure pursuit steering to target point"""
        # Vector to target
        dx = target_point.x - vehicle_loc.x
        dy = target_point.y - vehicle_loc.y
        
        # Transform to vehicle frame
        yaw = math.radians(vehicle_rot.yaw)
        local_x = dx * math.cos(yaw) + dy * math.sin(yaw)
        local_y = -dx * math.sin(yaw) + dy * math.cos(yaw)
        
        # Steering angle
        if abs(local_x) < 0.1:
            return 0.0
        
        steering_angle = math.atan2(local_y, local_x)
        
        # Normalize to [-1, 1]
        steer_cmd = steering_angle / math.radians(30)  # Max 30 degrees
        return float(np.clip(steer_cmd, -1.0, 1.0))

    def adaptive_speed_control(self, current_speed_mps, lane_error, steering_mag):
        """Adaptive throttle based on conditions"""
        target_speed_mps = self.target_speed_kmh / 3.6
        
        # Reduce speed with lane error
        lane_factor = max(0.7, 1.0 - abs(lane_error) / 3.0)
        
        # Reduce speed with steering
        steer_factor = max(0.75, 1.0 - abs(steering_mag) / 0.6)
        
        adjusted_target = target_speed_mps * lane_factor * steer_factor
        speed_error = adjusted_target - current_speed_mps
        
        if speed_error > 0:
            throttle = min(0.7, 0.4 + speed_error * 0.3)
        else:
            throttle = max(0.0, 0.35 + speed_error * 0.25)
        
        return float(np.clip(throttle, 0.0, 0.8))

    def run_evaluation(self):
        print("\n" + "="*70)
        print("SMOOTH LANE CHANGE WITH PRECISE STOPPING")
        print("="*70)
        
        try:
            self.connect_carla()
            self.spawn_vehicle()
            self.setup_sensors()
            self.load_model()
        except Exception as e:
            print("Init failed:", e)
            traceback.print_exc()
            return

        spectator = self.world.get_spectator()
        ctrl = carla.VehicleControl()

        print("Warming up sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)

        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log = time.time()

        # Logging
        log_filename = f"eval_log_{int(self.start_time)}.csv"
        log_file = open(log_filename, "w")
        log_file.write("timestamp,frame,x,y,z,yaw,speed_kmh,final_steer,throttle,lane_error,state\n")

        image_dir = f"eval_images_{int(self.start_time)}"
        os.makedirs(image_dir, exist_ok=True)

        try:
            while self.running:
                self.world.tick()

                # Update spectator
                try:
                    tf = self.vehicle.get_transform()
                    spec_loc = tf.location + tf.get_forward_vector() * -8 + carla.Location(z=4.0)
                    spec_rot = carla.Rotation(pitch=-15, yaw=tf.rotation.yaw)
                    spectator.set_transform(carla.Transform(spec_loc, spec_rot))
                except Exception:
                    pass

                # Collect frames
                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False

                # Get traffic light info
                tl_actor, tl_dist = self.find_traffic_light_ahead()
                tl_state = None
                if tl_actor is not None:
                    try:
                        tl_state = tl_actor.get_state()
                    except Exception:
                        pass

                # Main control loop
                if len(frame_stack) == 3:
                    # Get vehicle state
                    tf = self.vehicle.get_transform()
                    loc = tf.location
                    rot = tf.rotation
                    vel = self.vehicle.get_velocity()
                    speed_mps = math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
                    speed_kmh = speed_mps * 3.6

                    # Lane error
                    lane_error = self.calculate_lane_error(loc, rot)

                    # Get model prediction
                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    if input_tensor is None:
                        continue

                    speed_norm = min(speed_mps / 30.0, 1.0)
                    speed_tensor = torch.tensor([[speed_norm]], dtype=torch.float32, device=self.device)

                    with torch.no_grad():
                        output = self.model(input_tensor, aux=speed_tensor)
                    model_steer, model_throttle, model_brake = output[0].cpu().numpy()

                    # Initialize control
                    final_steer = 0.0
                    final_throttle = 0.0
                    final_brake = 0.0

                    # ============ STATE MACHINE ============
                    
                    if self.state == "DRIVING":
                        # Check for red light
                        if (tl_actor is not None and tl_state == carla.TrafficLightState.Red and 
                            tl_dist is not None and tl_dist <= 25.0):
                            print(f"🔴 Red light @ {tl_dist:.1f}m -> STOPPING")
                            self.state = "APPROACHING_STOP"
                            self.stop_start_time = time.time()
                        else:
                            # Normal lane keeping with GENTLE, BALANCED control
                            now = time.time()
                            pid_correction = self.lane_pid.update(lane_error, now)
                            
                            # MUCH GENTLER blending - trust model more (changed from 0.5, 0.90)
                            error_weight = min(abs(lane_error) / 1.2, 0.65)  # Max 65% PID
                            combined_steer = (1 - error_weight) * model_steer + error_weight * pid_correction
                            
                            # VERY smooth steering to prevent oscillations
                            final_steer = self.steer_alpha * combined_steer + (1 - self.steer_alpha) * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.55, 0.55))  # Reduced from 0.7
                            
                            # Adaptive throttle
                            final_throttle = self.adaptive_speed_control(speed_mps, lane_error, abs(final_steer))
                            final_brake = 0.0
                    
                    elif self.state == "APPROACHING_STOP":
                        # Smooth braking to stop at correct position
                        if tl_dist is not None:
                            # Calculate brake based on distance and speed
                            stop_dist = tl_dist - self.stop_distance_from_line
                            
                            if stop_dist <= 0.5 or speed_mps < 0.3:
                                # Arrived at stop line
                                print("🛑 Stopped at line, waiting for green...")
                                self.state = "WAITING_GREEN"
                                self.green_wait_time = 0.0
                                self.lane_pid.reset()
                                final_steer = 0.0
                                final_throttle = 0.0
                                final_brake = 1.0
                            else:
                                # Proportional braking
                                brake_strength = min(1.0, 0.4 + (speed_mps / 15.0) + (1.0 / max(stop_dist, 1.0)))
                                final_brake = float(np.clip(brake_strength, 0.3, 1.0))
                                final_throttle = 0.0
                                
                                # Gentle steering to maintain lane while braking
                                now = time.time()
                                pid_correction = self.lane_pid.update(lane_error, now)
                                final_steer = 0.5 * pid_correction  # Gentle correction
                                final_steer = float(np.clip(final_steer, -0.3, 0.3))
                        else:
                            # Lost traffic light, stop anyway
                            final_brake = 0.8
                            final_throttle = 0.0
                            final_steer = 0.0
                    
                    elif self.state == "WAITING_GREEN":
                        # Wait at stop line
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                        
                        # Check for green light
                        if tl_actor is not None and tl_state == carla.TrafficLightState.Green:
                            self.green_wait_time += 0.05  # One tick
                            
                            if self.green_wait_time > 0.5:  # Wait 0.5s after green
                                print("🟢 Green light -> Deciding lane change...")
                                
                                # Decide lane change direction
                                loc = self.vehicle.get_location()
                                current_wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
                                
                                if current_wp:
                                    # Try random direction first
                                    directions = ["left", "right"] if random.random() < 0.5 else ["right", "left"]
                                    
                                    for direction in directions:
                                        if self.check_lane_availability(current_wp, direction):
                                            print(f"  ✓ {direction.upper()} lane is available")
                                            self.lane_change_decision = direction
                                            self.state = "INITIATING_LANE_CHANGE"
                                            break
                                    else:
                                        print("  ✗ No safe lane, going straight")
                                        self.state = "DRIVING"
                                        self.lane_pid.reset()
                                else:
                                    print("  ✗ Cannot get waypoint, going straight")
                                    self.state = "DRIVING"
                                    self.lane_pid.reset()
                    
                    elif self.state == "INITIATING_LANE_CHANGE":
                        # Create lane change path
                        if not self.lane_change_path_created:
                            path = self.create_lane_change_path(self.lane_change_decision)
                            
                            if path:
                                self.path_follower.set_path(path)
                                self.lane_change_path_created = True
                                print(f"  🔄 Lane change path created ({len(path)} waypoints)")
                                self.state = "EXECUTING_LANE_CHANGE"
                            else:
                                print("  ✗ Failed to create path, aborting")
                                self.state = "DRIVING"
                                self.lane_pid.reset()
                        
                        # While creating, stay stopped
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                    
                    elif self.state == "EXECUTING_LANE_CHANGE":
                        # Follow the waypoint path
                        target_point = self.path_follower.get_target_point(loc)
                        
                        if target_point is not None:
                            # Calculate steering to target
                            path_steer = self.calculate_steering_to_point(loc, rot, target_point)
                            
                            # EXTREMELY smooth steering during lane change
                            final_steer = 0.08 * path_steer + 0.92 * self.steer_smooth  # Very gradual
                            final_steer = float(np.clip(final_steer, -0.40, 0.40))  # Reduced limit
                            
                            # SLOW speed during lane change for maximum smoothness
                            target_lc_speed = 13.0  # Reduced from 15 km/h
                            lc_speed_mps = target_lc_speed / 3.6
                            speed_error = lc_speed_mps - speed_mps
                            
                            if speed_error > 0:
                                final_throttle = min(0.42, 0.22 + speed_error * 0.15)
                            else:
                                final_throttle = 0.15
                            
                            final_brake = 0.0
                            
                            # Check if path complete
                            if self.path_follower.is_path_complete(loc):
                                print(f"  ✅ Lane change to {self.lane_change_decision.upper()} COMPLETE!")
                                self.lane_changes_completed += 1
                                self.path_follower.clear_path()
                                self.lane_change_path_created = False
                                self.lane_change_decision = None
                                self.state = "STABILIZING"
                                self.stop_start_time = time.time()
                        else:
                            # Lost path, abort
                            print("  ⚠️ Lost path target, stabilizing")
                            self.path_follower.clear_path()
                            self.lane_change_path_created = False
                            self.lane_change_decision = None
                            self.state = "STABILIZING"
                            self.stop_start_time = time.time()
                            self.lane_pid.reset()
                            final_steer = 0.0
                            final_throttle = 0.0
                            final_brake = 0.4
                    
                    elif self.state == "STABILIZING":
                        # After lane change, stabilize VERY GENTLY
                        elapsed = time.time() - self.stop_start_time
                        
                        if elapsed < 3.5:  # Longer stabilization period
                            # GENTLE PID with model blending
                            now = time.time()
                            pid_correction = self.lane_pid.update(lane_error, now)
                            
                            # Gentle blending (changed)
                            error_weight = min(abs(lane_error) / 1.8, 0.55)  # Max 55% PID
                            combined_steer = (1 - error_weight) * model_steer + error_weight * pid_correction
                            
                            final_steer = self.steer_alpha * combined_steer + (1 - self.steer_alpha) * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.35, 0.35))  # Very limited
                            
                            # Very gentle acceleration
                            final_throttle = min(0.32, 0.08 + elapsed * 0.07)
                            final_brake = 0.0
                            
                            # Early exit if well-centered
                            if elapsed > 2.0 and abs(lane_error) < 0.18:
                                print(f"  📍 Well-centered (error: {abs(lane_error):.3f}m), resuming")
                                self.state = "DRIVING"
                        else:
                            print(f"  📍 Stabilization complete (error: {abs(lane_error):.3f}m)")
                            self.state = "DRIVING"
                            self.lane_pid.reset()
                    
                    elif self.state == "COLLISION_RECOVERY":
                        # Recovery from collision
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0

                    # ============ APPLY CONTROL ============
                    
                    # Apply smoothing to steering
                    self.steer_smooth = final_steer
                    
                    # Apply smoothing to throttle
                    self.throttle_smooth = final_throttle
                    
                    # Apply to vehicle
                    ctrl.steer = float(final_steer)
                    ctrl.throttle = float(final_throttle)
                    ctrl.brake = float(final_brake)
                    ctrl.hand_brake = False
                    
                    try:
                        self.vehicle.apply_control(ctrl)
                    except Exception:
                        pass

                    # ============ LOGGING & RECORDING ============
                    
                    # Save frame
                    try:
                        if self._saved_frame_ema is not None:
                            save_img = np.clip(self._saved_frame_ema, 0, 255).astype(np.uint8)
                            save_path = os.path.join(image_dir, f"{frame_count:06d}.png")
                            cv2.imwrite(save_path, save_img)
                    except Exception:
                        pass

                    # Statistics
                    self.deviations.append(abs(lane_error))
                    self.controls_history.append({
                        'frame': frame_count,
                        'final_steer': float(final_steer),
                        'throttle': float(final_throttle),
                        'speed': float(speed_mps),
                        'lane_error': float(lane_error),
                        'state': self.state
                    })
                    self.trajectory.append({
                        'x': loc.x,
                        'y': loc.y,
                        'z': loc.z,
                        'yaw': rot.yaw
                    })

                    # Log to CSV
                    tstamp = time.time() - self.start_time
                    log_file.write(
                        f"{tstamp:.3f},{frame_count},{loc.x:.2f},{loc.y:.2f},{loc.z:.2f},"
                        f"{rot.yaw:.2f},{speed_kmh:.1f},{final_steer:.4f},{final_throttle:.4f},"
                        f"{lane_error:.4f},{self.state}\n"
                    )
                    log_file.flush()

                    # Console output
                    if time.time() - last_log >= 1.0:
                        tl_msg = "None"
                        if tl_actor is not None and tl_dist is not None:
                            try:
                                st = tl_actor.get_state()
                                tl_msg = f"{st.name[:3]} @ {tl_dist:.1f}m"
                            except Exception:
                                tl_msg = f"@ {tl_dist:.1f}m"
                        
                        lc_info = f"[LC: {self.lane_changes_completed}]" if self.lane_changes_completed > 0 else ""
                        
                        print(
                            f"[{tstamp:6.1f}s] F{frame_count:4d} | "
                            f"Speed:{speed_kmh:5.1f} | "
                            f"Lane:{lane_error:+.3f}m | "
                            f"Steer:{final_steer:+.3f} | "
                            f"TL:{tl_msg:15s} | "
                            f"{self.state:22s} {lc_info}"
                        )
                        last_log = time.time()

                    frame_count += 1

        except Exception as e:
            print("Evaluation error:", e)
            traceback.print_exc()
        finally:
            try:
                log_file.close()
            except Exception:
                pass
            
            # Create video
            try:
                imgs = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
                if imgs:
                    out_vid = f"eval_video_{int(self.start_time)}.mp4"
                    print(f"\nCreating video from {len(imgs)} frames...")
                    with imageio.get_writer(out_vid, fps=20) as writer:
                        for im in imgs:
                            writer.append_data(imageio.imread(im))
                    print("✓ Video saved:", out_vid)
            except Exception as e:
                print("Video creation warning:", e)

            self.print_statistics(frame_count)
            self.cleanup()

    def print_statistics(self, frame_count):
        elapsed = time.time() - (self.start_time or time.time())
        print("\n" + "="*70)
        print("EVALUATION STATISTICS")
        print("="*70)
        print(f"Duration: {elapsed:.2f} s")
        print(f"Frames: {frame_count}")
        if elapsed > 0:
            print(f"FPS: {frame_count / elapsed:.2f}")
        print(f"Collisions: {self.collision_count}")
        print(f"\nLane Changes:")
        print(f"  Completed: {self.lane_changes_completed}")
        
        if self.deviations:
            print(f"\nLane Keeping:")
            print(f"  Mean error: {np.mean(self.deviations):.3f} m")
            print(f"  Max error: {np.max(self.deviations):.3f} m")
            print(f"  Std dev: {np.std(self.deviations):.3f} m")
            within_half = sum(1 for d in self.deviations if d < 0.5) / len(self.deviations) * 100
            within_quarter = sum(1 for d in self.deviations if d < 0.25) / len(self.deviations) * 100
            print(f"  Within 0.5m: {within_half:.1f}%")
            print(f"  Within 0.25m: {within_quarter:.1f}%")
        
        if self.trajectory:
            distances = []
            for i in range(1, len(self.trajectory)):
                dx = self.trajectory[i]['x'] - self.trajectory[i-1]['x']
                dy = self.trajectory[i]['y'] - self.trajectory[i-1]['y']
                distances.append(math.sqrt(dx**2 + dy**2))
            total_dist = sum(distances)
            print(f"\nDistance traveled: {total_dist:.2f} m")
            if elapsed > 0:
                avg_speed = (total_dist / elapsed) * 3.6
                print(f"Average speed: {avg_speed:.2f} km/h")
        print("="*70)

    def cleanup(self):
        print("\nCleaning up...")
        try:
            if self.world is not None and self.original_settings is not None:
                self.world.apply_settings(self.original_settings)
        except Exception:
            pass
        
        actors = []
        for a in (self.camera, self.collision_sensor, self.vehicle):
            if a is not None:
                try:
                    a.stop()
                except Exception:
                    pass
                actors.append(a)
        
        for a in actors:
            try:
                a.destroy()
            except Exception:
                pass
        
        gc.collect()
        print("✓ Cleanup complete")


def main():
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    
    controller = ImprovedLaneChangeController(
        model_path=MODEL_PATH,
        host="localhost",
        port=2000
    )
    
    controller.run_evaluation()


if __name__ == "__main__":
    main()


SMOOTH LANE CHANGE WITH PRECISE STOPPING
Spawned at (-41.67, 89.75)
Device: cuda
Model loaded
Warming up sensors...
[   1.0s] F  17 | Speed:  0.0 | Lane:+0.115m | Steer:+0.032 | TL:None            | DRIVING                
[   2.0s] F  36 | Speed:  0.0 | Lane:+0.115m | Steer:+0.036 | TL:None            | DRIVING                
[   3.1s] F  55 | Speed: 15.9 | Lane:+0.038m | Steer:+0.019 | TL:None            | DRIVING                
[   4.1s] F  75 | Speed: 18.8 | Lane:-0.165m | Steer:-0.011 | TL:None            | DRIVING                
[   5.1s] F  95 | Speed: 18.5 | Lane:-0.220m | Steer:-0.015 | TL:None            | DRIVING                
[   6.2s] F 114 | Speed: 18.6 | Lane:-0.132m | Steer:-0.010 | TL:Red @ 39.7m     | DRIVING                
[   7.2s] F 132 | Speed: 18.7 | Lane:-0.049m | Steer:+0.031 | TL:Red @ 35.3m     | DRIVING                
[   8.2s] F 151 | Speed: 18.4 | Lane:-0.169m | Steer:+0.008 | TL:Red @ 30.6m     | DRIVING                
[   9.2s] F 169 | Speed: 18

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Shutdown requested.

Creating video from 3131 frames...
Video creation warning: [Errno 32] Broken pipe

FFMPEG COMMAND:
/home/robotuser/anaconda3/envs/carla_env/lib/python3.8/site-packages/imageio_ffmpeg/binaries/ffmpeg-linux64-v4.2.2 -y -f rawvideo -vcodec rawvideo -s 800x600 -pix_fmt rgb24 -r 20.00 -i - -an -vcodec libx264 -pix_fmt yuv420p -vf scale=800:608 -v warning /home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/eval_video_1760300841.mp4

FFMPEG STDERR OUTPUT:


EVALUATION STATISTICS
Duration: 172.01 s
Frames: 3131
FPS: 18.20
Collisions: 0

Lane Changes:
  Completed: 2

Lane Keeping:
  Mean error: 0.354 m
  Max error: 2.626 m
  Std dev: 0.429 m
  Within 0.5m: 83.1%
  Within 0.25m: 76.0%

Distance traveled: 240.45 m
Average speed: 5.03 km/h

Cleaning up...


Error initializing output stream 0:0 -- Error while opening encoder for output stream #0:0 - maybe incorrect parameters such as bit_rate, rate, width or height


✓ Cleanup complete


In [4]:
import carla
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import cv2
import gc
import math

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


def preprocess_image_stack(frame_stack, device):
    if len(frame_stack) != 3:
        return None
    processed_frames = []
    for frame in frame_stack:
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        frame = frame.astype(np.float32) / 255.0
        frame = (frame - 0.5) / 0.5
        processed_frames.append(frame)
    stacked = np.concatenate(processed_frames, axis=2)
    stacked = np.transpose(stacked, (2, 0, 1))
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)
    return tensor.to(device)


class SimpleSteeringController:
    """Very smooth and gentle steering control"""
    def __init__(self):
        self.Kp = 0.25      # Very gentle proportional
        self.Ki = 0.005     # Minimal integral
        self.Kd = 0.08      # Light derivative
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        self.integral_limit = 0.5

    def update(self, error, current_time):
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05
        else:
            dt = max(current_time - self.prev_time, 1e-3)
            self.prev_time = current_time
        
        P = self.Kp * error
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        D = self.Kd * derivative
        self.prev_error = error
        
        output = P + I + D
        return float(np.clip(output, -0.4, 0.4))

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


class StableLaneChangeController:
    def __init__(self, model_path, host="localhost", port=2000):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        self.steering_control = SimpleSteeringController()
        
        # VERY aggressive smoothing to prevent oscillations
        self.steer_smooth = 0.0
        self.throttle_smooth = 0.0
        self.steer_alpha = 0.06      # Very smooth (was 0.12)
        self.throttle_alpha = 0.15

        # Speed and stopping
        self.target_speed_kmh = 18.0
        self.stop_distance_from_line = 1.5  # Stop closer to line

        # State machine
        self.state = "DRIVING"
        self.stop_timer = None
        self.lane_change_active = False
        self.lane_change_direction = None

        # Stats
        self.collision_count = 0
        self.deviations = []
        self.lane_changes_completed = 0

        # CARLA
        self.client = None
        self.world = None
        self.map = None
        self.vehicle = None
        self.camera = None
        self.collision_sensor = None
        self.latest_image = None
        self.new_image_available = False
        self.start_time = None
        self.original_settings = None

    def connect_carla(self):
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        self.original_settings = self.world.get_settings()
        self.map = self.world.get_map()

        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)

    def spawn_vehicle(self):
        bp_lib = self.world.get_blueprint_library()
        vehicle_bp = bp_lib.filter("vehicle.tesla.model3")[0]
        spawn_points = self.map.get_spawn_points()
        sp = random.choice(spawn_points)
        sp.location.z += 0.5
        self.vehicle = self.world.spawn_actor(vehicle_bp, sp)
        print(f"Spawned at ({sp.location.x:.2f}, {sp.location.y:.2f})")
        return sp.location

    def setup_sensors(self):
        bp_lib = self.world.get_blueprint_library()
        
        cam_bp = bp_lib.find("sensor.camera.rgb")
        cam_bp.set_attribute("image_size_x", "800")
        cam_bp.set_attribute("image_size_y", "600")
        cam_bp.set_attribute("fov", "100")
        cam_transform = carla.Transform(carla.Location(x=1.3, z=1.8), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)
        self.camera.listen(self.camera_callback)

        col_bp = bp_lib.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(col_bp, carla.Transform(), attach_to=self.vehicle)
        self.collision_sensor.listen(self.collision_callback)

    def camera_callback(self, image):
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]
            small = cv2.resize(arr, (160, 120))
            self.latest_image = small
            self.new_image_available = True
        except Exception:
            pass

    def collision_callback(self, event):
        self.collision_count += 1
        print(f"\n*** COLLISION #{self.collision_count} ***")
        self.state = "COLLISION_RECOVERY"

    def load_model(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Device:", self.device)
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device)
        
        loaded = torch.load(self.model_path, map_location=self.device, weights_only=False)
        state = loaded['state_dict'] if isinstance(loaded, dict) and 'state_dict' in loaded else loaded
        
        try:
            self.model.load_state_dict(state)
        except RuntimeError:
            new_state = {k[7:] if k.startswith("module.") else k: v for k, v in state.items()}
            self.model.load_state_dict(new_state)
        
        print("Model loaded")
        self.model.eval()

    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        """Calculate pure cross-track error (no heading component)"""
        try:
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            if waypoint is None:
                return 0.0
            
            lane_center = waypoint.transform.location
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            cross_track = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            
            return cross_track
        except Exception:
            return 0.0

    def find_traffic_light_ahead(self):
        """Find nearest traffic light ahead"""
        if self.vehicle is None:
            return None, None
        
        try:
            veh_tf = self.vehicle.get_transform()
            veh_loc = veh_tf.location
            veh_fwd = veh_tf.get_forward_vector()
            
            lights = self.world.get_actors().filter("traffic.traffic_light*")
            best_light = None
            best_dist = None
            
            for tl in lights:
                tl_loc = tl.get_transform().location
                vec = np.array([tl_loc.x - veh_loc.x, tl_loc.y - veh_loc.y, tl_loc.z - veh_loc.z])
                dist = np.linalg.norm(vec)
                
                if dist > 50.0 or dist < 0.5:
                    continue
                
                vec_norm = vec / (dist + 1e-6)
                dot = vec_norm[0] * veh_fwd.x + vec_norm[1] * veh_fwd.y + vec_norm[2] * veh_fwd.z
                
                if dot > 0.7:
                    if best_light is None or dist < best_dist:
                        best_light = tl
                        best_dist = dist
            
            return best_light, best_dist
        except Exception:
            return None, None

    def check_lane_available(self, current_wp, direction):
        """Check if lane change is safe"""
        try:
            if direction == "left":
                target_wp = current_wp.get_left_lane()
            else:
                target_wp = current_wp.get_right_lane()
            
            if target_wp is None or target_wp.lane_type != carla.LaneType.Driving:
                return False
            
            # Check same direction
            if (current_wp.lane_id > 0) != (target_wp.lane_id > 0):
                return False
            
            # Check for vehicles
            veh_loc = self.vehicle.get_location()
            for other in self.world.get_actors().filter("vehicle.*"):
                if other.id == self.vehicle.id:
                    continue
                
                other_loc = other.get_location()
                dist = math.sqrt((other_loc.x - veh_loc.x)**2 + (other_loc.y - veh_loc.y)**2)
                
                if dist < 30.0:
                    other_wp = self.map.get_waypoint(other_loc, project_to_road=True)
                    if other_wp and other_wp.lane_id == target_wp.lane_id:
                        return False
            
            return True
        except Exception:
            return False

    def simple_lane_change(self, direction):
        """Execute simple lane change: gradual steering, maintain lane"""
        self.lane_change_active = True
        self.lane_change_direction = direction
        print(f"Initiating {direction.upper()} lane change...")

    def run_evaluation(self):
        print("\n" + "="*70)
        print("STABLE LANE CHANGE - ROCK SOLID LANE KEEPING")
        print("="*70)
        
        try:
            self.connect_carla()
            self.spawn_vehicle()
            self.setup_sensors()
            self.load_model()
        except Exception as e:
            print("Init failed:", e)
            return

        spectator = self.world.get_spectator()
        ctrl = carla.VehicleControl()

        print("Warming up sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)

        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log = time.time()

        try:
            while self.running and frame_count < 10000:
                self.world.tick()

                # Update spectator
                try:
                    tf = self.vehicle.get_transform()
                    spec_loc = tf.location + tf.get_forward_vector() * -8 + carla.Location(z=4.0)
                    spec_rot = carla.Rotation(pitch=-15, yaw=tf.rotation.yaw)
                    spectator.set_transform(carla.Transform(spec_loc, spec_rot))
                except Exception:
                    pass

                # Collect frames
                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False

                # Get traffic light
                tl_actor, tl_dist = self.find_traffic_light_ahead()
                tl_state = None
                if tl_actor is not None:
                    try:
                        tl_state = tl_actor.get_state()
                    except Exception:
                        pass

                # Main control
                if len(frame_stack) == 3:
                    tf = self.vehicle.get_transform()
                    loc = tf.location
                    rot = tf.rotation
                    vel = self.vehicle.get_velocity()
                    speed_mps = math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
                    speed_kmh = speed_mps * 3.6

                    lane_error = self.calculate_lane_error(loc, rot)
                    self.deviations.append(abs(lane_error))

                    # Get model prediction
                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    if input_tensor is None:
                        continue

                    speed_norm = min(speed_mps / 30.0, 1.0)
                    speed_tensor = torch.tensor([[speed_norm]], dtype=torch.float32, device=self.device)

                    with torch.no_grad():
                        output = self.model(input_tensor, aux=speed_tensor)
                    model_steer, model_throttle, model_brake = output[0].cpu().numpy()

                    final_steer = 0.0
                    final_throttle = 0.0
                    final_brake = 0.0

                    # ============ STATE MACHINE ============
                    
                    if self.state == "DRIVING":
                        # Check for red light
                        if (tl_actor is not None and tl_state == carla.TrafficLightState.Red and 
                            tl_dist is not None and tl_dist <= 30.0):
                            print(f"RED light @ {tl_dist:.1f}m -> STOPPING")
                            self.state = "APPROACHING_STOP"
                            self.stop_timer = time.time()
                        else:
                            # Normal lane keeping - VERY GENTLE
                            now = time.time()
                            pid_steer = self.steering_control.update(lane_error, now)
                            
                            # Blend model (70%) + PID (30%)
                            combined = 0.7 * model_steer + 0.3 * pid_steer
                            
                            # EXTREME smoothing
                            final_steer = self.steer_alpha * combined + (1 - self.steer_alpha) * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.25, 0.25))
                            
                            # Adaptive throttle
                            target_mps = self.target_speed_kmh / 3.6
                            speed_error = target_mps - speed_mps
                            final_throttle = np.clip(0.35 + speed_error * 0.08, 0.1, 0.7)
                            final_brake = 0.0
                    
                    elif self.state == "APPROACHING_STOP":
                        if tl_dist is not None:
                            stop_dist = tl_dist - self.stop_distance_from_line
                            
                            if stop_dist <= 0.3 or speed_mps < 0.2:
                                # At stop line
                                print("STOPPED at line")
                                self.state = "WAITING_GREEN"
                                self.stop_timer = time.time()
                                self.steering_control.reset()
                                final_steer = 0.0
                                final_throttle = 0.0
                                final_brake = 1.0
                            else:
                                # Smooth braking
                                brake_str = min(1.0, 0.3 + (speed_mps / 12.0) + (2.0 / max(stop_dist, 1.0)))
                                final_brake = float(np.clip(brake_str, 0.2, 0.9))
                                final_throttle = 0.0
                                
                                # GENTLE steering while braking
                                now = time.time()
                                pid_steer = self.steering_control.update(lane_error, now)
                                final_steer = 0.3 * pid_steer
                                final_steer = float(np.clip(final_steer, -0.15, 0.15))
                    
                    elif self.state == "WAITING_GREEN":
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                        
                        if tl_actor is not None and tl_state == carla.TrafficLightState.Green:
                            wait_time = time.time() - self.stop_timer
                            if wait_time > 0.3:
                                print("GREEN light!")
                                
                                # Try lane change
                                loc = self.vehicle.get_location()
                                current_wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
                                
                                direction = random.choice(["left", "right"])
                                if current_wp and self.check_lane_available(current_wp, direction):
                                    print(f"  Lane {direction.upper()} available -> CHANGING")
                                    self.simple_lane_change(direction)
                                    self.state = "LANE_CHANGING"
                                else:
                                    print(f"  Lane {direction.upper()} blocked -> GOING STRAIGHT")
                                    self.state = "DRIVING"
                                    self.steering_control.reset()
                    
                    elif self.state == "LANE_CHANGING":
                        # Lane change: gradual steering at low speed
                        steer_magnitude = 0.12 if self.lane_change_direction == "left" else -0.12
                        
                        # Very gradual increase
                        progress = min((time.time() - self.stop_timer - 0.3) / 2.0, 1.0)
                        
                        if progress < 1.0:
                            # During lane change
                            target_steer = steer_magnitude * (0.3 + 0.7 * progress)
                            
                            # Smooth it
                            final_steer = 0.05 * target_steer + 0.95 * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.25, 0.25))
                            
                            # Slow speed during change
                            target_lc_mps = 12.0 / 3.6
                            speed_error = target_lc_mps - speed_mps
                            final_throttle = np.clip(0.25 + speed_error * 0.1, 0.05, 0.4)
                            final_brake = 0.0
                        else:
                            # Lane change complete
                            print(f"Lane change COMPLETE")
                            self.lane_changes_completed += 1
                            self.lane_change_active = False
                            self.lane_change_direction = None
                            self.state = "DRIVING"
                            self.steering_control.reset()
                            final_steer = 0.0
                            final_throttle = 0.3
                            final_brake = 0.0
                    
                    elif self.state == "COLLISION_RECOVERY":
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                        if time.time() - (self.stop_timer or time.time()) > 2.0:
                            self.state = "DRIVING"

                    # ============ APPLY CONTROL ============
                    self.steer_smooth = final_steer
                    self.throttle_smooth = final_throttle
                    
                    ctrl.steer = float(final_steer)
                    ctrl.throttle = float(final_throttle)
                    ctrl.brake = float(final_brake)
                    ctrl.hand_brake = False
                    
                    try:
                        self.vehicle.apply_control(ctrl)
                    except Exception:
                        pass

                    # Logging
                    if time.time() - last_log >= 1.0:
                        tl_msg = "None"
                        if tl_actor is not None and tl_dist is not None:
                            st_name = "Red" if tl_state == carla.TrafficLightState.Red else "Green"
                            tl_msg = f"{st_name} @ {tl_dist:.1f}m"
                        
                        print(
                            f"[{time.time()-self.start_time:6.1f}s] F{frame_count:4d} | "
                            f"Speed:{speed_kmh:5.1f} | Lane:{lane_error:+.3f}m | "
                            f"Steer:{final_steer:+.3f} | {self.state:20s} | {tl_msg}"
                        )
                        last_log = time.time()

                    frame_count += 1

        except Exception as e:
            print("Error:", e)
        finally:
            self.print_statistics(frame_count)
            self.cleanup()

    def print_statistics(self, frame_count):
        elapsed = time.time() - (self.start_time or time.time())
        print("\n" + "="*70)
        print("STATISTICS")
        print("="*70)
        print(f"Duration: {elapsed:.2f}s | Frames: {frame_count} | FPS: {frame_count/elapsed:.1f}")
        print(f"Collisions: {self.collision_count}")
        print(f"Lane changes completed: {self.lane_changes_completed}")
        
        if self.deviations:
            print(f"\nLane Keeping:")
            print(f"  Mean error: {np.mean(self.deviations):.3f}m")
            print(f"  Max error: {np.max(self.deviations):.3f}m")
            print(f"  Within 0.3m: {sum(1 for d in self.deviations if d < 0.3)/len(self.deviations)*100:.1f}%")
        print("="*70)

    def cleanup(self):
        print("\nCleaning up...")
        try:
            if self.world is not None and self.original_settings is not None:
                self.world.apply_settings(self.original_settings)
        except Exception:
            pass
        
        for a in (self.camera, self.collision_sensor, self.vehicle):
            if a is not None:
                try:
                    a.stop()
                    a.destroy()
                except Exception:
                    pass
        
        gc.collect()
        print("Cleanup complete")


def main():
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    
    controller = StableLaneChangeController(
        model_path=MODEL_PATH,
        host="localhost",
        port=2000
    )
    
    controller.run_evaluation()


if __name__ == "__main__":
    main()


STABLE LANE CHANGE - ROCK SOLID LANE KEEPING
Spawned at (102.57, 43.97)
Device: cuda
Model loaded
Warming up sensors...
[   1.0s] F  59 | Speed: 13.9 | Lane:-0.054m | Steer:+0.012 | DRIVING              | None
[   2.0s] F 123 | Speed: 18.0 | Lane:-0.175m | Steer:-0.007 | DRIVING              | None
[   3.0s] F 190 | Speed: 18.0 | Lane:-0.059m | Steer:+0.003 | DRIVING              | None
[   4.0s] F 263 | Speed: 18.0 | Lane:+0.246m | Steer:+0.022 | DRIVING              | None
[   5.0s] F 334 | Speed: 18.0 | Lane:+0.989m | Steer:+0.079 | DRIVING              | None
[   6.0s] F 400 | Speed: 18.1 | Lane:+0.559m | Steer:+0.067 | DRIVING              | None
[   7.0s] F 465 | Speed: 18.1 | Lane:+1.121m | Steer:+0.092 | DRIVING              | None
[   8.1s] F 528 | Speed: 18.1 | Lane:+0.242m | Steer:+0.019 | DRIVING              | None
[   9.1s] F 595 | Speed: 18.1 | Lane:-0.540m | Steer:-0.025 | DRIVING              | None
[  10.1s] F 666 | Speed: 18.0 | Lane:+0.238m | Steer:+0.022 | DRIVING

KeyboardInterrupt: 

In [4]:
import carla
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import cv2
import gc
import math

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


def preprocess_image_stack(frame_stack, device):
    if len(frame_stack) != 3:
        return None
    processed_frames = []
    for frame in frame_stack:
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        frame = frame.astype(np.float32) / 255.0
        frame = (frame - 0.5) / 0.5
        processed_frames.append(frame)
    stacked = np.concatenate(processed_frames, axis=2)
    stacked = np.transpose(stacked, (2, 0, 1))
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)
    return tensor.to(device)


class SimpleSteeringController:
    """Very smooth and gentle steering control"""
    def __init__(self):
        self.Kp = 0.25      # Very gentle proportional
        self.Ki = 0.005     # Minimal integral
        self.Kd = 0.08      # Light derivative
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        self.integral_limit = 0.5

    def update(self, error, current_time):
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05
        else:
            dt = max(current_time - self.prev_time, 1e-3)
            self.prev_time = current_time
        
        P = self.Kp * error
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        D = self.Kd * derivative
        self.prev_error = error
        
        output = P + I + D
        return float(np.clip(output, -0.4, 0.4))

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


class StableLaneChangeController:
    def __init__(self, model_path, host="localhost", port=2000):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        self.steering_control = SimpleSteeringController()
        
        # VERY aggressive smoothing to prevent oscillations
        self.steer_smooth = 0.0
        self.throttle_smooth = 0.0
        self.steer_alpha = 0.06      # Very smooth (was 0.12)
        self.throttle_alpha = 0.15

        # Speed and stopping
        self.target_speed_kmh = 18.0
        self.stop_distance_from_line = 1.5  # Stop closer to line

        # State machine
        self.state = "DRIVING"
        self.stop_timer = None
        self.lane_change_active = False
        self.lane_change_direction = None

        # Stats
        self.collision_count = 0
        self.deviations = []
        self.lane_changes_completed = 0

        # CARLA
        self.client = None
        self.world = None
        self.map = None
        self.vehicle = None
        self.camera = None
        self.collision_sensor = None
        self.latest_image = None
        self.new_image_available = False
        self.start_time = None
        self.original_settings = None

    def connect_carla(self):
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        self.original_settings = self.world.get_settings()
        self.map = self.world.get_map()

        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)

    def spawn_vehicle(self):
        bp_lib = self.world.get_blueprint_library()
        vehicle_bp = bp_lib.filter("vehicle.tesla.model3")[0]
        spawn_points = self.map.get_spawn_points()
        sp = random.choice(spawn_points)
        sp.location.z += 0.5
        self.vehicle = self.world.spawn_actor(vehicle_bp, sp)
        print(f"Spawned at ({sp.location.x:.2f}, {sp.location.y:.2f})")
        return sp.location

    def setup_sensors(self):
        bp_lib = self.world.get_blueprint_library()
        
        cam_bp = bp_lib.find("sensor.camera.rgb")
        cam_bp.set_attribute("image_size_x", "800")
        cam_bp.set_attribute("image_size_y", "600")
        cam_bp.set_attribute("fov", "100")
        cam_transform = carla.Transform(carla.Location(x=1.3, z=1.8), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)
        self.camera.listen(self.camera_callback)

        col_bp = bp_lib.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(col_bp, carla.Transform(), attach_to=self.vehicle)
        self.collision_sensor.listen(self.collision_callback)

    def camera_callback(self, image):
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]
            small = cv2.resize(arr, (160, 120))
            self.latest_image = small
            self.new_image_available = True
        except Exception:
            pass

    def collision_callback(self, event):
        self.collision_count += 1
        print(f"\n*** COLLISION #{self.collision_count} ***")
        self.state = "COLLISION_RECOVERY"

    def load_model(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Device:", self.device)
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device)
        
        loaded = torch.load(self.model_path, map_location=self.device, weights_only=False)
        state = loaded['state_dict'] if isinstance(loaded, dict) and 'state_dict' in loaded else loaded
        
        try:
            self.model.load_state_dict(state)
        except RuntimeError:
            new_state = {k[7:] if k.startswith("module.") else k: v for k, v in state.items()}
            self.model.load_state_dict(new_state)
        
        print("Model loaded")
        self.model.eval()

    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        """Calculate pure cross-track error (no heading component)"""
        try:
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            if waypoint is None:
                return 0.0
            
            lane_center = waypoint.transform.location
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            cross_track = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            
            return cross_track
        except Exception:
            return 0.0

    def find_traffic_light_ahead(self):
        """Find nearest traffic light ahead"""
        if self.vehicle is None:
            return None, None
        
        try:
            veh_tf = self.vehicle.get_transform()
            veh_loc = veh_tf.location
            veh_fwd = veh_tf.get_forward_vector()
            
            lights = self.world.get_actors().filter("traffic.traffic_light*")
            best_light = None
            best_dist = None
            
            for tl in lights:
                tl_loc = tl.get_transform().location
                vec = np.array([tl_loc.x - veh_loc.x, tl_loc.y - veh_loc.y, tl_loc.z - veh_loc.z])
                dist = np.linalg.norm(vec)
                
                if dist > 50.0 or dist < 0.5:
                    continue
                
                vec_norm = vec / (dist + 1e-6)
                dot = vec_norm[0] * veh_fwd.x + vec_norm[1] * veh_fwd.y + vec_norm[2] * veh_fwd.z
                
                if dot > 0.7:
                    if best_light is None or dist < best_dist:
                        best_light = tl
                        best_dist = dist
            
            return best_light, best_dist
        except Exception:
            return None, None

    def check_lane_available(self, current_wp, direction):
        """Check if lane change is safe - SIMPLIFIED"""
        try:
            if direction == "left":
                target_wp = current_wp.get_left_lane()
            else:
                target_wp = current_wp.get_right_lane()
            
            # If target lane exists and is driving lane, it's available
            if target_wp is None:
                return False
            
            if target_wp.lane_type != carla.LaneType.Driving:
                return False
            
            return True
        except Exception:
            return False

    def simple_lane_change(self, direction):
        """Execute simple lane change: gradual steering, maintain lane"""
        self.lane_change_active = True
        self.lane_change_direction = direction
        print(f"Initiating {direction.upper()} lane change...")

    def run_evaluation(self):
        print("\n" + "="*70)
        print("STABLE LANE CHANGE - ROCK SOLID LANE KEEPING")
        print("="*70)
        
        try:
            self.connect_carla()
            self.spawn_vehicle()
            self.setup_sensors()
            self.load_model()
        except Exception as e:
            print("Init failed:", e)
            return

        spectator = self.world.get_spectator()
        ctrl = carla.VehicleControl()

        print("Warming up sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)

        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log = time.time()

        try:
            while self.running and frame_count < 10000:
                self.world.tick()

                # Update spectator
                try:
                    tf = self.vehicle.get_transform()
                    spec_loc = tf.location + tf.get_forward_vector() * -8 + carla.Location(z=4.0)
                    spec_rot = carla.Rotation(pitch=-15, yaw=tf.rotation.yaw)
                    spectator.set_transform(carla.Transform(spec_loc, spec_rot))
                except Exception:
                    pass

                # Collect frames
                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False

                # Get traffic light
                tl_actor, tl_dist = self.find_traffic_light_ahead()
                tl_state = None
                if tl_actor is not None:
                    try:
                        tl_state = tl_actor.get_state()
                    except Exception:
                        pass

                # Main control
                if len(frame_stack) == 3:
                    tf = self.vehicle.get_transform()
                    loc = tf.location
                    rot = tf.rotation
                    vel = self.vehicle.get_velocity()
                    speed_mps = math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
                    speed_kmh = speed_mps * 3.6

                    lane_error = self.calculate_lane_error(loc, rot)
                    self.deviations.append(abs(lane_error))

                    # Get model prediction
                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    if input_tensor is None:
                        continue

                    speed_norm = min(speed_mps / 30.0, 1.0)
                    speed_tensor = torch.tensor([[speed_norm]], dtype=torch.float32, device=self.device)

                    with torch.no_grad():
                        output = self.model(input_tensor, aux=speed_tensor)
                    model_steer, model_throttle, model_brake = output[0].cpu().numpy()

                    final_steer = 0.0
                    final_throttle = 0.0
                    final_brake = 0.0

                    # ============ STATE MACHINE ============
                    
                    if self.state == "DRIVING":
                        # Check for red light MUCH EARLIER (50m away)
                        if (tl_actor is not None and tl_state == carla.TrafficLightState.Red and 
                            tl_dist is not None and tl_dist <= 50.0):
                            print(f"🔴 RED light detected @ {tl_dist:.1f}m -> APPROACHING_STOP")
                            self.state = "APPROACHING_STOP"
                            self.stop_timer = time.time()
                        else:
                            # Normal lane keeping - VERY GENTLE
                            now = time.time()
                            pid_steer = self.steering_control.update(lane_error, now)
                            
                            # Blend model (70%) + PID (30%)
                            combined = 0.7 * model_steer + 0.3 * pid_steer
                            
                            # EXTREME smoothing
                            final_steer = self.steer_alpha * combined + (1 - self.steer_alpha) * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.25, 0.25))
                            
                            # Adaptive throttle
                            target_mps = self.target_speed_kmh / 3.6
                            speed_error = target_mps - speed_mps
                            final_throttle = np.clip(0.35 + speed_error * 0.08, 0.1, 0.7)
                            final_brake = 0.0
                    
                    elif self.state == "APPROACHING_STOP":
                        if tl_dist is not None:
                            stop_dist = tl_dist - self.stop_distance_from_line
                            
                            if stop_dist <= 0.5 or speed_mps < 0.15:
                                # At stop line - FULL BRAKE
                                self.state = "WAITING_GREEN"
                                self.stop_timer = time.time()
                                self.steering_control.reset()
                                final_steer = 0.0
                                final_throttle = 0.0
                                final_brake = 1.0
                            else:
                                # Apply strong, progressive braking
                                brake_str = 0.4 + (speed_mps / 15.0) * 0.5 + max(0, 3.0 / max(stop_dist, 0.5))
                                final_brake = float(np.clip(brake_str, 0.3, 1.0))
                                final_throttle = 0.0
                                
                                # GENTLE steering while braking
                                now = time.time()
                                pid_steer = self.steering_control.update(lane_error, now)
                                final_steer = 0.3 * pid_steer
                                final_steer = float(np.clip(final_steer, -0.15, 0.15))
                        else:
                            final_brake = 1.0
                            final_throttle = 0.0
                            final_steer = 0.0
                    
                    elif self.state == "WAITING_GREEN":
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                        
                        # Check if green light and waited at least 0.5 seconds
                        wait_elapsed = time.time() - self.stop_timer if self.stop_timer else 0
                        
                        if (tl_actor is not None and tl_state == carla.TrafficLightState.Green and 
                            wait_elapsed >= 0.5):
                            print(f"GREEN light! (waited {wait_elapsed:.1f}s)")
                            
                            # Try lane change
                            loc = self.vehicle.get_location()
                            current_wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
                            
                            if current_wp:
                                # Try left first, then right
                                for direction in ["left", "right"]:
                                    if self.check_lane_available(current_wp, direction):
                                        print(f"  ✓ {direction.upper()} lane available!")
                                        self.simple_lane_change(direction)
                                        self.state = "LANE_CHANGING"
                                        self.stop_timer = time.time()  # Reset timer for LC duration
                                        break
                                else:
                                    print(f"  ✗ No lanes available, going straight")
                                    self.state = "DRIVING"
                                    self.stop_timer = time.time()
                                    self.steering_control.reset()
                            else:
                                print(f"  ✗ Cannot get waypoint, going straight")
                                self.state = "DRIVING"
                                self.stop_timer = time.time()
                                self.steering_control.reset()
                    
                    elif self.state == "LANE_CHANGING":
                        # Lane change: gradual steering at low speed
                        steer_magnitude = 0.12 if self.lane_change_direction == "left" else -0.12
                        
                        # Very gradual increase
                        progress = min((time.time() - self.stop_timer - 0.3) / 2.0, 1.0)
                        
                        if progress < 1.0:
                            # During lane change
                            target_steer = steer_magnitude * (0.3 + 0.7 * progress)
                            
                            # Smooth it
                            final_steer = 0.05 * target_steer + 0.95 * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.25, 0.25))
                            
                            # Slow speed during change
                            target_lc_mps = 12.0 / 3.6
                            speed_error = target_lc_mps - speed_mps
                            final_throttle = np.clip(0.25 + speed_error * 0.1, 0.05, 0.4)
                            final_brake = 0.0
                        else:
                            # Lane change complete
                            print(f"Lane change COMPLETE")
                            self.lane_changes_completed += 1
                            self.lane_change_active = False
                            self.lane_change_direction = None
                            self.state = "DRIVING"
                            self.steering_control.reset()
                            final_steer = 0.0
                            final_throttle = 0.3
                            final_brake = 0.0
                    
                    elif self.state == "COLLISION_RECOVERY":
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                        if time.time() - (self.stop_timer or time.time()) > 2.0:
                            self.state = "DRIVING"

                    # ============ APPLY CONTROL ============
                    self.steer_smooth = final_steer
                    self.throttle_smooth = final_throttle
                    
                    ctrl.steer = float(final_steer)
                    ctrl.throttle = float(final_throttle)
                    ctrl.brake = float(final_brake)
                    ctrl.hand_brake = False
                    
                    try:
                        self.vehicle.apply_control(ctrl)
                    except Exception:
                        pass

                    # Logging
                    if time.time() - last_log >= 1.0:
                        tl_msg = "None"
                        if tl_actor is not None and tl_dist is not None:
                            st_name = "Red" if tl_state == carla.TrafficLightState.Red else "Green"
                            tl_msg = f"{st_name} @ {tl_dist:.1f}m"
                        
                        print(
                            f"[{time.time()-self.start_time:6.1f}s] F{frame_count:4d} | "
                            f"Speed:{speed_kmh:5.1f} | Lane:{lane_error:+.3f}m | "
                            f"Steer:{final_steer:+.3f} | {self.state:20s} | {tl_msg}"
                        )
                        last_log = time.time()

                    frame_count += 1

        except Exception as e:
            print("Error:", e)
        finally:
            self.print_statistics(frame_count)
            self.cleanup()

    def print_statistics(self, frame_count):
        elapsed = time.time() - (self.start_time or time.time())
        print("\n" + "="*70)
        print("STATISTICS")
        print("="*70)
        print(f"Duration: {elapsed:.2f}s | Frames: {frame_count} | FPS: {frame_count/elapsed:.1f}")
        print(f"Collisions: {self.collision_count}")
        print(f"Lane changes completed: {self.lane_changes_completed}")
        
        if self.deviations:
            print(f"\nLane Keeping:")
            print(f"  Mean error: {np.mean(self.deviations):.3f}m")
            print(f"  Max error: {np.max(self.deviations):.3f}m")
            print(f"  Within 0.3m: {sum(1 for d in self.deviations if d < 0.3)/len(self.deviations)*100:.1f}%")
        print("="*70)

    def cleanup(self):
        print("\nCleaning up...")
        try:
            if self.world is not None and self.original_settings is not None:
                self.world.apply_settings(self.original_settings)
        except Exception:
            pass
        
        for a in (self.camera, self.collision_sensor, self.vehicle):
            if a is not None:
                try:
                    a.stop()
                    a.destroy()
                except Exception:
                    pass
        
        gc.collect()
        print("Cleanup complete")


def main():
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    
    controller = StableLaneChangeController(
        model_path=MODEL_PATH,
        host="localhost",
        port=2000
    )
    
    controller.run_evaluation()


if __name__ == "__main__":
    main()


STABLE LANE CHANGE - ROCK SOLID LANE KEEPING
Spawned at (47.56, -57.23)
Device: cuda
Model loaded
Warming up sensors...
[   1.0s] F  60 | Speed: 14.0 | Lane:-0.107m | Steer:-0.005 | DRIVING              | None
[   2.0s] F 123 | Speed: 18.0 | Lane:-0.072m | Steer:+0.004 | DRIVING              | None
[   3.0s] F 189 | Speed: 18.0 | Lane:+1.300m | Steer:+0.122 | DRIVING              | None
[   4.0s] F 248 | Speed: 18.1 | Lane:+0.628m | Steer:+0.056 | DRIVING              | None
[   5.0s] F 307 | Speed: 18.1 | Lane:+1.018m | Steer:+0.084 | DRIVING              | None
🔴 RED light detected @ 49.8m -> APPROACHING_STOP
[   6.0s] F 362 | Speed:  0.0 | Lane:+1.242m | Steer:+0.000 | WAITING_GREEN        | Red @ 48.7m
[   7.0s] F 418 | Speed:  0.0 | Lane:+1.242m | Steer:+0.000 | WAITING_GREEN        | Red @ 48.7m
[   8.0s] F 478 | Speed:  0.0 | Lane:+1.242m | Steer:+0.000 | WAITING_GREEN        | Red @ 48.7m
GREEN light! (waited 3.5s)
  ✓ LEFT lane available!
Initiating LEFT lane change...
[   9.

KeyboardInterrupt: 

# lane change code

In [1]:
import carla
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import cv2
import gc
import imageio.v2 as imageio
import signal
import math
import traceback

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model (unchanged) -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


def preprocess_image_stack(frame_stack, device):
    if len(frame_stack) != 3:
        return None
    processed_frames = []
    for frame in frame_stack:
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        frame = frame.astype(np.float32) / 255.0
        frame = (frame - 0.5) / 0.5
        processed_frames.append(frame)
    stacked = np.concatenate(processed_frames, axis=2)
    stacked = np.transpose(stacked, (2, 0, 1))
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)
    return tensor.to(device)


# ----------------- Enhanced PID Controller -----------------
class StrongPIDController:
    """Ultra-precise lane keeping PID controller"""
    def __init__(self):
        self.Kp = 1.8  # Very strong proportional
        self.Ki = 0.08  # Moderate integral
        self.Kd = 0.45  # Strong derivative
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        self.integral_limit = 0.8
        self.output_limit = 1.0

    def update(self, error, current_time):
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05
        else:
            dt = max(current_time - self.prev_time, 1e-3)
            self.prev_time = current_time
        
        P = self.Kp * error
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = self.Ki * self.integral
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        D = self.Kd * derivative
        self.prev_error = error
        output = P + I + D
        return float(np.clip(output, -self.output_limit, self.output_limit))

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None


# ----------------- Waypoint Path Follower -----------------
class WaypointFollower:
    """Follow a pre-computed waypoint path smoothly"""
    def __init__(self):
        self.path = []
        self.current_idx = 0
        self.lookahead_distance = 8.0  # meters
        
    def set_path(self, waypoints):
        """Set new waypoint path"""
        self.path = waypoints
        self.current_idx = 0
        
    def clear_path(self):
        """Clear current path"""
        self.path = []
        self.current_idx = 0
        
    def get_target_point(self, vehicle_location):
        """Get lookahead target point on path"""
        if not self.path:
            return None
            
        # Find closest waypoint
        min_dist = float('inf')
        closest_idx = self.current_idx
        
        for i in range(self.current_idx, min(len(self.path), self.current_idx + 10)):
            wp_loc = self.path[i].transform.location
            dist = math.sqrt(
                (wp_loc.x - vehicle_location.x)**2 + 
                (wp_loc.y - vehicle_location.y)**2
            )
            if dist < min_dist:
                min_dist = dist
                closest_idx = i
        
        self.current_idx = closest_idx
        
        # Find lookahead point
        target_idx = closest_idx
        cumulative_dist = 0.0
        
        for i in range(closest_idx, len(self.path) - 1):
            wp1 = self.path[i].transform.location
            wp2 = self.path[i + 1].transform.location
            segment_dist = math.sqrt(
                (wp2.x - wp1.x)**2 + (wp2.y - wp1.y)**2
            )
            cumulative_dist += segment_dist
            
            if cumulative_dist >= self.lookahead_distance:
                target_idx = i + 1
                break
        else:
            target_idx = len(self.path) - 1
            
        if target_idx >= len(self.path):
            return None
            
        return self.path[target_idx].transform.location
    
    def is_path_complete(self, vehicle_location):
        """Check if we've reached the end of path"""
        if not self.path:
            return True
        
        end_loc = self.path[-1].transform.location
        dist = math.sqrt(
            (end_loc.x - vehicle_location.x)**2 + 
            (end_loc.y - vehicle_location.y)**2
        )
        return dist < 3.0


# ----------------- Main Controller -----------------
class ImprovedLaneChangeController:
    def __init__(self, model_path, host="localhost", port=2000):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        # Controllers
        self.lane_pid = StrongPIDController()
        self.path_follower = WaypointFollower()
        
        # Smoothing
        self.steer_smooth = 0.0
        self.throttle_smooth = 0.3
        self.steer_alpha = 0.25  # Smoother steering
        self.throttle_alpha = 0.20

        # Speed control
        self.target_speed_kmh = 20.0
        self.stop_distance_from_line = 3.5  # Stop 3.5m before traffic light

        # State machine
        self.state = "DRIVING"
        self.stop_start_time = None
        self.green_wait_time = 0.0
        self.lane_change_decision = None  # "left" or "right"
        self.lane_change_path_created = False

        # Stats
        self.collision_count = 0
        self.deviations = []
        self.controls_history = []
        self.trajectory = []
        self.lane_changes_completed = 0

        # CARLA objects
        self.client = None
        self.world = None
        self.map = None
        self.vehicle = None
        self.camera = None
        self.collision_sensor = None
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        self.start_time = None
        self.original_settings = None
        self._saved_frame_ema = None
        self._ema_alpha = 0.25

        signal.signal(signal.SIGINT, self.signal_handler)

    def signal_handler(self, sig, frame):
        print("\nShutdown requested.")
        self.running = False

    def connect_carla(self):
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        self.original_settings = self.world.get_settings()
        self.map = self.world.get_map()

        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)

    def spawn_vehicle(self):
        
        blueprint_library = self.world.get_blueprint_library()
        vehicle_bp = blueprint_library.filter("vehicle.mercedes.coupe_2020")[0]
        if not vehicle_bp:
            vehicle_bp = blueprint_library.filter("vehicle.*")[0]
        # bp_lib = self.world.get_blueprint_library()
        # vehicle_bp = bp_lib.filter("vehicle.tesla.model3")[0] if bp_lib.filter("vehicle.tesla.model3") else random.choice(bp_lib.filter("vehicle.*"))
        spawn_points = self.map.get_spawn_points()
        sp = random.choice(spawn_points)
        sp.location.z += 0.5
        self.vehicle = self.world.spawn_actor(vehicle_bp, sp)
        print(f"Spawned at ({sp.location.x:.2f}, {sp.location.y:.2f})")
        return sp.location

    def setup_sensors(self):
        bp_lib = self.world.get_blueprint_library()
        
        # Camera
        cam_bp = bp_lib.find("sensor.camera.rgb")
        cam_bp.set_attribute("image_size_x", "800")
        cam_bp.set_attribute("image_size_y", "600")
        cam_bp.set_attribute("fov", "120")
        cam_transform = carla.Transform(carla.Location(x=-4,y=0, z=2), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)
        self.camera.listen(self.camera_callback)

        # Collision sensor
        col_bp = bp_lib.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(col_bp, carla.Transform(), attach_to=self.vehicle)
        self.collision_sensor.listen(self.collision_callback)

    def camera_callback(self, image):
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]
            small = cv2.resize(arr, (800, 600))
            self.latest_image = small
            self.new_image_available = True

            save_frame = cv2.cvtColor(arr, cv2.COLOR_RGB2BGR)
            if self._saved_frame_ema is None:
                self._saved_frame_ema = save_frame.astype(np.float32)
            else:
                self._saved_frame_ema = (self._ema_alpha * save_frame.astype(np.float32) +
                                         (1 - self._ema_alpha) * self._saved_frame_ema)
        except Exception:
            pass

    def collision_callback(self, event):
        self.collision_count += 1
        print(f"\n*** COLLISION #{self.collision_count} ***")
        self.collision_occurred = True
        
        # Emergency stop and reset
        self.path_follower.clear_path()
        self.lane_pid.reset()
        self.state = "COLLISION_RECOVERY"
        
        try:
            ctrl = carla.VehicleControl()
            ctrl.steer = 0.0
            ctrl.throttle = 0.0
            ctrl.brake = 1.0
            self.vehicle.apply_control(ctrl)
        except Exception:
            pass
        
        time.sleep(1.5)
        self.collision_occurred = False
        self.state = "DRIVING"

    def load_model(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Device:", self.device)
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device)
        
        loaded = torch.load(self.model_path, map_location=self.device, weights_only=False)
        state = loaded['state_dict'] if isinstance(loaded, dict) and 'state_dict' in loaded else loaded
        
        try:
            self.model.load_state_dict(state)
        except RuntimeError:
            new_state = {k[7:] if k.startswith("module.") else k: v for k, v in state.items()}
            self.model.load_state_dict(new_state)
        
        print("Model loaded")
        self.model.eval()

    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        """Calculate cross-track and heading error"""
        try:
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            if waypoint is None:
                return 0.0
            
            # Cross-track error
            lane_center = waypoint.transform.location
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            cross_track = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            
            # Heading error
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            
            # Combined error with strong heading weight
            total_error = cross_track + 1.2 * heading_error
            return total_error
        except Exception:
            return 0.0

    def find_traffic_light_ahead(self):
        """Find nearest traffic light ahead of vehicle"""
        if self.vehicle is None:
            return None, None
        
        try:
            veh_tf = self.vehicle.get_transform()
            veh_loc = veh_tf.location
            veh_fwd = veh_tf.get_forward_vector()
            
            lights = self.world.get_actors().filter("traffic.traffic_light*")
            best_light = None
            best_dist = None
            
            for tl in lights:
                tl_loc = tl.get_transform().location
                vec = np.array([tl_loc.x - veh_loc.x, tl_loc.y - veh_loc.y, tl_loc.z - veh_loc.z])
                dist = np.linalg.norm(vec)
                
                if dist > 40.0 or dist < 0.5:
                    continue
                
                # Check if ahead
                vec_norm = vec / (dist + 1e-6)
                dot = vec_norm[0] * veh_fwd.x + vec_norm[1] * veh_fwd.y + vec_norm[2] * veh_fwd.z
                
                if dot > 0.7:  # Ahead of us
                    if best_light is None or dist < best_dist:
                        best_light = tl
                        best_dist = dist
            
            return best_light, best_dist
        except Exception:
            return None, None

    def check_lane_availability(self, current_wp, direction):
        """Check if lane change is safe and valid"""
        try:
            # Get target lane
            if direction == "left":
                target_wp = current_wp.get_left_lane()
            else:
                target_wp = current_wp.get_right_lane()
            
            if target_wp is None:
                return False
            
            # Must be driving lane
            if target_wp.lane_type != carla.LaneType.Driving:
                return False
            
            # Must be same direction (check lane_id sign)
            if (current_wp.lane_id > 0) != (target_wp.lane_id > 0):
                return False
            
            # Check path ahead is valid
            test_wp = target_wp
            valid_distance = 0.0
            for _ in range(30):
                nxts = test_wp.next(2.0)
                if not nxts:
                    break
                test_wp = nxts[0]
                if test_wp.lane_type != carla.LaneType.Driving:
                    break
                valid_distance += 2.0
            
            if valid_distance < 40.0:  # Need good path ahead
                return False
            
            # Check for vehicles in target lane
            veh_loc = self.vehicle.get_location()
            for other in self.world.get_actors().filter("vehicle.*"):
                if other.id == self.vehicle.id:
                    continue
                
                other_loc = other.get_location()
                dist = math.sqrt(
                    (other_loc.x - veh_loc.x)**2 + 
                    (other_loc.y - veh_loc.y)**2
                )
                
                if dist < 25.0:  # Check nearby vehicles
                    try:
                        other_wp = self.map.get_waypoint(other_loc, project_to_road=True)
                        if other_wp and other_wp.lane_id == target_wp.lane_id:
                            return False  # Vehicle in target lane
                    except Exception:
                        pass
            
            return True
            
        except Exception as e:
            print(f"Lane check error: {e}")
            return False

    def create_lane_change_path(self, direction):
        """Create smooth waypoint path for lane change"""
        try:
            loc = self.vehicle.get_location()
            current_wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
            
            if current_wp is None:
                return []
            
            # Get target lane
            if direction == "left":
                target_wp = current_wp.get_left_lane()
            else:
                target_wp = current_wp.get_right_lane()
            
            if target_wp is None:
                return []
            
            # Build path: current -> transition -> target
            path = []
            
            # Phase 1: Move forward in current lane (10m)
            wp = current_wp
            for _ in range(5):
                nxts = wp.next(2.0)
                if nxts:
                    wp = nxts[0]
                    path.append(wp)
            
            # Phase 2: Transition to target lane (20m smooth curve)
            start_wp = wp
            
            # Get corresponding point in target lane ahead
            end_wp = target_wp
            for _ in range(15):
                nxts = end_wp.next(2.0)
                if nxts:
                    end_wp = nxts[0]
            
            # Create interpolated waypoints
            for i in range(1, 11):
                t = i / 10.0
                # Smooth interpolation
                interp_loc = carla.Location(
                    x=start_wp.transform.location.x * (1 - t) + end_wp.transform.location.x * t,
                    y=start_wp.transform.location.y * (1 - t) + end_wp.transform.location.y * t,
                    z=start_wp.transform.location.z * (1 - t) + end_wp.transform.location.z * t
                )
                interp_wp = self.map.get_waypoint(interp_loc, project_to_road=True)
                if interp_wp:
                    path.append(interp_wp)
            
            # Phase 3: Continue in target lane (20m)
            wp = end_wp
            for _ in range(10):
                nxts = wp.next(2.0)
                if nxts:
                    wp = nxts[0]
                    path.append(wp)
            
            return path
            
        except Exception as e:
            print(f"Path creation error: {e}")
            return []

    def calculate_steering_to_point(self, vehicle_loc, vehicle_rot, target_point):
        """Pure pursuit steering to target point"""
        # Vector to target
        dx = target_point.x - vehicle_loc.x
        dy = target_point.y - vehicle_loc.y
        
        # Transform to vehicle frame
        yaw = math.radians(vehicle_rot.yaw)
        local_x = dx * math.cos(yaw) + dy * math.sin(yaw)
        local_y = -dx * math.sin(yaw) + dy * math.cos(yaw)
        
        # Steering angle
        if abs(local_x) < 0.1:
            return 0.0
        
        steering_angle = math.atan2(local_y, local_x)
        
        # Normalize to [-1, 1]
        steer_cmd = steering_angle / math.radians(30)  # Max 30 degrees
        return float(np.clip(steer_cmd, -1.0, 1.0))

    def adaptive_speed_control(self, current_speed_mps, lane_error, steering_mag):
        """Adaptive throttle based on conditions"""
        target_speed_mps = self.target_speed_kmh / 3.6
        
        # Reduce speed with lane error
        lane_factor = max(0.7, 1.0 - abs(lane_error) / 3.0)
        
        # Reduce speed with steering
        steer_factor = max(0.75, 1.0 - abs(steering_mag) / 0.6)
        
        adjusted_target = target_speed_mps * lane_factor * steer_factor
        speed_error = adjusted_target - current_speed_mps
        
        if speed_error > 0:
            throttle = min(0.7, 0.4 + speed_error * 0.3)
        else:
            throttle = max(0.0, 0.35 + speed_error * 0.25)
        
        return float(np.clip(throttle, 0.0, 0.8))

    def run_evaluation(self):
        print("\n" + "="*70)
        print("SMOOTH LANE CHANGE WITH PRECISE STOPPING")
        print("="*70)
        
        try:
            self.connect_carla()
            self.spawn_vehicle()
            self.setup_sensors()
            self.load_model()
        except Exception as e:
            print("Init failed:", e)
            traceback.print_exc()
            return

        spectator = self.world.get_spectator()
        ctrl = carla.VehicleControl()

        print("Warming up sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)

        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log = time.time()

        # Logging
        log_filename = f"eval_log_{int(self.start_time)}.csv"
        log_file = open(log_filename, "w")
        log_file.write("timestamp,frame,x,y,z,yaw,speed_kmh,final_steer,throttle,lane_error,state\n")

        image_dir = f"eval_images_{int(self.start_time)}"
        os.makedirs(image_dir, exist_ok=True)

        try:
            while self.running:
                self.world.tick()

                # Update spectator
                try:
                    tf = self.vehicle.get_transform()
                    spec_loc = tf.location + tf.get_forward_vector() * -8 + carla.Location(z=4.0)
                    spec_rot = carla.Rotation(pitch=-15, yaw=tf.rotation.yaw)
                    spectator.set_transform(carla.Transform(spec_loc, spec_rot))
                except Exception:
                    pass

                # Collect frames
                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False

                # Get traffic light info
                tl_actor, tl_dist = self.find_traffic_light_ahead()
                tl_state = None
                if tl_actor is not None:
                    try:
                        tl_state = tl_actor.get_state()
                    except Exception:
                        pass

                # Main control loop
                if len(frame_stack) == 3:
                    # Get vehicle state
                    tf = self.vehicle.get_transform()
                    loc = tf.location
                    rot = tf.rotation
                    vel = self.vehicle.get_velocity()
                    speed_mps = math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
                    speed_kmh = speed_mps * 3.6

                    # Lane error
                    lane_error = self.calculate_lane_error(loc, rot)

                    # Get model prediction
                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    if input_tensor is None:
                        continue

                    speed_norm = min(speed_mps / 30.0, 1.0)
                    speed_tensor = torch.tensor([[speed_norm]], dtype=torch.float32, device=self.device)

                    with torch.no_grad():
                        output = self.model(input_tensor, aux=speed_tensor)
                    model_steer, model_throttle, model_brake = output[0].cpu().numpy()

                    # Initialize control
                    final_steer = 0.0
                    final_throttle = 0.0
                    final_brake = 0.0

                    # ============ STATE MACHINE ============
                    
                    if self.state == "DRIVING":
                        # Check for red light
                        if (tl_actor is not None and tl_state == carla.TrafficLightState.Red and 
                            tl_dist is not None and tl_dist <= 25.0):
                            print(f"🔴 Red light @ {tl_dist:.1f}m -> STOPPING")
                            self.state = "APPROACHING_STOP"
                            self.stop_start_time = time.time()
                        else:
                            # Normal lane keeping with strong PID
                            now = time.time()
                            pid_correction = self.lane_pid.update(lane_error, now)
                            
                            # Blend model and PID (heavy PID weight)
                            error_weight = min(abs(lane_error) / 0.5, 0.90)
                            combined_steer = (1 - error_weight) * model_steer + error_weight * pid_correction
                            
                            # Smooth steering
                            final_steer = self.steer_alpha * combined_steer + (1 - self.steer_alpha) * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.7, 0.7))
                            
                            # Adaptive throttle
                            final_throttle = self.adaptive_speed_control(speed_mps, lane_error, abs(final_steer))
                            final_brake = 0.0
                    
                    elif self.state == "APPROACHING_STOP":
                        # Smooth braking to stop at correct position
                        if tl_dist is not None:
                            # Calculate brake based on distance and speed
                            stop_dist = tl_dist - self.stop_distance_from_line
                            
                            if stop_dist <= 0.5 or speed_mps < 0.3:
                                # Arrived at stop line
                                print("🛑 Stopped at line, waiting for green...")
                                self.state = "WAITING_GREEN"
                                self.green_wait_time = 0.0
                                self.lane_pid.reset()
                                final_steer = 0.0
                                final_throttle = 0.0
                                final_brake = 1.0
                            else:
                                # Proportional braking
                                brake_strength = min(1.0, 0.4 + (speed_mps / 15.0) + (1.0 / max(stop_dist, 1.0)))
                                final_brake = float(np.clip(brake_strength, 0.3, 1.0))
                                final_throttle = 0.0
                                
                                # Gentle steering to maintain lane while braking
                                now = time.time()
                                pid_correction = self.lane_pid.update(lane_error, now)
                                final_steer = 0.5 * pid_correction  # Gentle correction
                                final_steer = float(np.clip(final_steer, -0.3, 0.3))
                        else:
                            # Lost traffic light, stop anyway
                            final_brake = 0.8
                            final_throttle = 0.0
                            final_steer = 0.0
                    
                    elif self.state == "WAITING_GREEN":
                        # Wait at stop line
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                        
                        # Check for green light
                        if tl_actor is not None and tl_state == carla.TrafficLightState.Green:
                            self.green_wait_time += 0.05  # One tick
                            
                            if self.green_wait_time > 0.5:  # Wait 0.5s after green
                                print("🟢 Green light -> Deciding lane change...")
                                
                                # Decide lane change direction
                                loc = self.vehicle.get_location()
                                current_wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
                                
                                if current_wp:
                                    # Try random direction first
                                    directions = ["left", "right"] if random.random() < 0.5 else ["right", "left"]
                                    
                                    for direction in directions:
                                        if self.check_lane_availability(current_wp, direction):
                                            print(f"  ✓ {direction.upper()} lane is available")
                                            self.lane_change_decision = direction
                                            self.state = "INITIATING_LANE_CHANGE"
                                            break
                                    else:
                                        print("  ✗ No safe lane, going straight")
                                        self.state = "DRIVING"
                                        self.lane_pid.reset()
                                else:
                                    print("  ✗ Cannot get waypoint, going straight")
                                    self.state = "DRIVING"
                                    self.lane_pid.reset()
                    
                    elif self.state == "INITIATING_LANE_CHANGE":
                        # Create lane change path
                        if not self.lane_change_path_created:
                            path = self.create_lane_change_path(self.lane_change_decision)
                            
                            if path:
                                self.path_follower.set_path(path)
                                self.lane_change_path_created = True
                                print(f"  🔄 Lane change path created ({len(path)} waypoints)")
                                self.state = "EXECUTING_LANE_CHANGE"
                            else:
                                print("  ✗ Failed to create path, aborting")
                                self.state = "DRIVING"
                                self.lane_pid.reset()
                        
                        # While creating, stay stopped
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                    
                    elif self.state == "EXECUTING_LANE_CHANGE":
                        # Follow the waypoint path
                        target_point = self.path_follower.get_target_point(loc)
                        
                        if target_point is not None:
                            # Calculate steering to target
                            final_steer = self.calculate_steering_to_point(loc, rot, target_point)
                            
                            # Smooth steering
                            final_steer = self.steer_alpha * final_steer + (1 - self.steer_alpha) * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.6, 0.6))
                            
                            # Moderate speed during lane change
                            target_lc_speed = 15.0  # km/h
                            lc_speed_mps = target_lc_speed / 3.6
                            speed_error = lc_speed_mps - speed_mps
                            
                            if speed_error > 0:
                                final_throttle = min(0.5, 0.3 + speed_error * 0.2)
                            else:
                                final_throttle = 0.2
                            
                            final_brake = 0.0
                            
                            # Check if path complete
                            if self.path_follower.is_path_complete(loc):
                                print(f"  ✅ Lane change to {self.lane_change_decision.upper()} COMPLETE!")
                                self.lane_changes_completed += 1
                                self.path_follower.clear_path()
                                self.lane_change_path_created = False
                                self.lane_change_decision = None
                                self.state = "STABILIZING"
                                self.stop_start_time = time.time()
                        else:
                            # Lost path, abort
                            print("  ⚠️ Lost path target, aborting")
                            self.path_follower.clear_path()
                            self.lane_change_path_created = False
                            self.lane_change_decision = None
                            self.state = "DRIVING"
                            self.lane_pid.reset()
                            final_steer = 0.0
                            final_throttle = 0.0
                            final_brake = 0.3
                    
                    elif self.state == "STABILIZING":
                        # After lane change, stabilize in new lane
                        elapsed = time.time() - self.stop_start_time
                        
                        if elapsed < 2.0:
                            # Strong PID to center in lane
                            now = time.time()
                            pid_correction = self.lane_pid.update(lane_error, now)
                            
                            final_steer = pid_correction
                            final_steer = float(np.clip(final_steer, -0.5, 0.5))
                            
                            # Gentle acceleration
                            final_throttle = min(0.4, 0.15 + elapsed * 0.1)
                            final_brake = 0.0
                        else:
                            print("  📍 Stabilized in new lane, resuming normal driving")
                            self.state = "DRIVING"
                            self.lane_pid.reset()
                    
                    elif self.state == "COLLISION_RECOVERY":
                        # Recovery from collision
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0

                    # ============ APPLY CONTROL ============
                    
                    # Apply smoothing to steering
                    self.steer_smooth = final_steer
                    
                    # Apply smoothing to throttle
                    self.throttle_smooth = final_throttle
                    
                    # Apply to vehicle
                    ctrl.steer = float(final_steer)
                    ctrl.throttle = float(final_throttle)
                    ctrl.brake = float(final_brake)
                    ctrl.hand_brake = False
                    
                    try:
                        self.vehicle.apply_control(ctrl)
                    except Exception:
                        pass

                    # ============ LOGGING & RECORDING ============
                    
                    # Save frame
                    # try:
                    #     if self._saved_frame_ema is not None:
                    #         save_img = np.clip(self._saved_frame_ema, 0, 255).astype(np.uint8)
                    #         save_path = os.path.join(image_dir, f"{frame_count:06d}.png")
                    #         cv2.imwrite(save_path, save_img)
                    # except Exception:
                    #     pass
                    try:
                            frame_filename = os.path.join(image_dir, f"{frame_count:06d}.png")
                            cv2.imwrite(frame_filename, cv2.cvtColor(self.latest_image, cv2.COLOR_RGB2BGR))
                    except Exception as e:
                            print(f"Warning: failed to save frame {frame_count}: {e}")

                    # Statistics
                    self.deviations.append(abs(lane_error))
                    self.controls_history.append({
                        'frame': frame_count,
                        'final_steer': float(final_steer),
                        'throttle': float(final_throttle),
                        'speed': float(speed_mps),
                        'lane_error': float(lane_error),
                        'state': self.state
                    })
                    self.trajectory.append({
                        'x': loc.x,
                        'y': loc.y,
                        'z': loc.z,
                        'yaw': rot.yaw
                    })

                    # Log to CSV
                    tstamp = time.time() - self.start_time
                    log_file.write(
                        f"{tstamp:.3f},{frame_count},{loc.x:.2f},{loc.y:.2f},{loc.z:.2f},"
                        f"{rot.yaw:.2f},{speed_kmh:.1f},{final_steer:.4f},{final_throttle:.4f},"
                        f"{lane_error:.4f},{self.state}\n"
                    )
                    log_file.flush()

                    # Console output
                    if time.time() - last_log >= 1.0:
                        tl_msg = "None"
                        if tl_actor is not None and tl_dist is not None:
                            try:
                                st = tl_actor.get_state()
                                tl_msg = f"{st.name[:3]} @ {tl_dist:.1f}m"
                            except Exception:
                                tl_msg = f"@ {tl_dist:.1f}m"
                        
                        lc_info = f"[LC: {self.lane_changes_completed}]" if self.lane_changes_completed > 0 else ""
                        
                        print(
                            f"[{tstamp:6.1f}s] F{frame_count:4d} | "
                            f"Speed:{speed_kmh:5.1f} | "
                            f"Lane:{lane_error:+.3f}m | "
                            f"Steer:{final_steer:+.3f} | "
                            f"TL:{tl_msg:15s} | "
                            f"{self.state:22s} {lc_info}"
                        )
                        last_log = time.time()

                    frame_count += 1

        except Exception as e:
            print("Evaluation error:", e)
            traceback.print_exc()
        finally:
            try:
                log_file.close()
            except Exception:
                pass
            
            # Create video
            try:
                imgs = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
                if imgs:
                    out_vid = f"eval_video_{int(self.start_time)}.mp4"
                    print(f"\nCreating video from {len(imgs)} frames...")
                    with imageio.get_writer(out_vid, fps=30) as writer:
                        for im in imgs:
                            writer.append_data(imageio.imread(im))
                    print("✓ Video saved:", out_vid)
            except Exception as e:
                print("Video creation warning:", e)

            self.print_statistics(frame_count)
            self.cleanup()

    def print_statistics(self, frame_count):
        elapsed = time.time() - (self.start_time or time.time())
        print("\n" + "="*70)
        print("EVALUATION STATISTICS")
        print("="*70)
        print(f"Duration: {elapsed:.2f} s")
        print(f"Frames: {frame_count}")
        if elapsed > 0:
            print(f"FPS: {frame_count / elapsed:.2f}")
        print(f"Collisions: {self.collision_count}")
        print(f"\nLane Changes:")
        print(f"  Completed: {self.lane_changes_completed}")
        
        if self.deviations:
            print(f"\nLane Keeping:")
            print(f"  Mean error: {np.mean(self.deviations):.3f} m")
            print(f"  Max error: {np.max(self.deviations):.3f} m")
            print(f"  Std dev: {np.std(self.deviations):.3f} m")
            within_half = sum(1 for d in self.deviations if d < 0.5) / len(self.deviations) * 100
            within_quarter = sum(1 for d in self.deviations if d < 0.25) / len(self.deviations) * 100
            print(f"  Within 0.5m: {within_half:.1f}%")
            print(f"  Within 0.25m: {within_quarter:.1f}%")
        
        if self.trajectory:
            distances = []
            for i in range(1, len(self.trajectory)):
                dx = self.trajectory[i]['x'] - self.trajectory[i-1]['x']
                dy = self.trajectory[i]['y'] - self.trajectory[i-1]['y']
                distances.append(math.sqrt(dx**2 + dy**2))
            total_dist = sum(distances)
            print(f"\nDistance traveled: {total_dist:.2f} m")
            if elapsed > 0:
                avg_speed = (total_dist / elapsed) * 3.6
                print(f"Average speed: {avg_speed:.2f} km/h")
        print("="*70)

    def cleanup(self):
        print("\nCleaning up...")
        try:
            if self.world is not None and self.original_settings is not None:
                self.world.apply_settings(self.original_settings)
        except Exception:
            pass
        
        actors = []
        for a in (self.camera, self.collision_sensor, self.vehicle):
            if a is not None:
                try:
                    a.stop()
                except Exception:
                    pass
                actors.append(a)
        
        for a in actors:
            try:
                a.destroy()
            except Exception:
                pass
        
        gc.collect()
        print("✓ Cleanup complete")


def main():
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    
    controller = ImprovedLaneChangeController(
        model_path=MODEL_PATH,
        host="localhost",
        port=2000
    )
    
    controller.run_evaluation()


if __name__ == "__main__":
    main()


SMOOTH LANE CHANGE WITH PRECISE STOPPING
Spawned at (-111.12, 72.90)
Device: cuda
Model loaded
Warming up sensors...
[   1.0s] F  30 | Speed:  0.0 | Lane:-0.062m | Steer:+0.009 | TL:None            | DRIVING                
[   2.0s] F  67 | Speed: 17.1 | Lane:-0.102m | Steer:-0.043 | TL:None            | DRIVING                
[   3.1s] F 103 | Speed: 18.4 | Lane:-0.075m | Steer:-0.027 | TL:None            | DRIVING                
[   4.1s] F 138 | Speed: 18.2 | Lane:-0.047m | Steer:+0.000 | TL:None            | DRIVING                
[   5.1s] F 174 | Speed: 15.8 | Lane:-0.148m | Steer:-0.072 | TL:None            | DRIVING                
[   6.1s] F 211 | Speed: 17.6 | Lane:-0.073m | Steer:-0.020 | TL:None            | DRIVING                
[   7.1s] F 250 | Speed: 18.1 | Lane:+0.048m | Steer:+0.018 | TL:None            | DRIVING                
[   8.1s] F 285 | Speed: 12.0 | Lane:-0.006m | Steer:+0.005 | TL:Gre @ 29.8m     | DRIVING                
[   9.1s] F 322 | Speed: 1

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Shutdown requested.
[  33.5s] F1146 | Speed: 15.3 | Lane:-0.172m | Steer:-0.093 | TL:None            | DRIVING                [LC: 1]

Creating video from 1147 frames...
✓ Video saved: eval_video_1768080790.mp4

EVALUATION STATISTICS
Duration: 41.56 s
Frames: 1147
FPS: 27.60
Collisions: 0

Lane Changes:
  Completed: 1

Lane Keeping:
  Mean error: 0.192 m
  Max error: 2.191 m
  Std dev: 0.299 m
  Within 0.5m: 89.5%
  Within 0.25m: 81.2%

Distance traveled: 199.38 m
Average speed: 17.27 km/h

Cleaning up...
✓ Cleanup complete



# model efficient code

In [2]:
import carla
import torch
import torch.nn as nn
import numpy as np
import time
import os
import random
import cv2
import gc
import imageio.v2 as imageio
import signal
import math
import traceback

os.environ["OMP_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
torch.set_num_threads(1)


# ----------------- Model (unchanged) -----------------
class DrivingNet(nn.Module):
    def __init__(self, in_ch=3, n_stack=1, use_speed=True):
        super().__init__()
        C = in_ch * n_stack
        self.use_speed = use_speed
        self.conv = nn.Sequential(
            nn.Conv2d(C, 24, kernel_size=5, stride=2), nn.BatchNorm2d(24), nn.ReLU(),
            nn.Conv2d(24, 36, kernel_size=5, stride=2), nn.BatchNorm2d(36), nn.ReLU(),
            nn.Conv2d(36, 48, kernel_size=5, stride=2), nn.BatchNorm2d(48), nn.ReLU(),
            nn.Conv2d(48, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU(),
            nn.Conv2d(64, 64, kernel_size=3, stride=1), nn.BatchNorm2d(64), nn.ReLU()
        )
        self._feat = None
        self.fc = None
        self.speed_fc = nn.Sequential(nn.Linear(1, 16), nn.ReLU()) if use_speed else None

    def initialize(self, input_shape):
        with torch.no_grad():
            dummy = torch.zeros(1, *input_shape)
            out = self.conv(dummy)
            self._feat = int(out.view(1, -1).shape[1])
        fc_in = self._feat + (16 if self.use_speed else 0)
        self.fc = nn.Sequential(
            nn.Linear(fc_in, 100), nn.ReLU(), nn.Dropout(0.2),
            nn.Linear(100, 50), nn.ReLU()
        )
        self.steer_head = nn.Linear(50, 1)
        self.throttle_head = nn.Linear(50, 1)
        self.brake_head = nn.Linear(50, 1)

    def forward(self, x, aux=None):
        device = next(self.parameters()).device
        x = x.to(device)
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        if self.use_speed and aux is not None:
            aux = aux.to(device).float()
            if aux.dim() == 1:
                aux = aux.unsqueeze(1)
            aux = self.speed_fc(aux)
            x = torch.cat([x, aux], dim=1)
        x = self.fc(x)
        steer = torch.tanh(self.steer_head(x))
        throttle = torch.sigmoid(self.throttle_head(x))
        brake = torch.sigmoid(self.brake_head(x))
        return torch.cat([steer, throttle, brake], dim=1)


def preprocess_image_stack(frame_stack, device):
    if len(frame_stack) != 3:
        return None
    processed_frames = []
    for frame in frame_stack:
        if frame.shape[:2] != (120, 160):
            frame = cv2.resize(frame, (160, 120))
        frame = frame.astype(np.float32) / 255.0
        frame = (frame - 0.5) / 0.5
        processed_frames.append(frame)
    stacked = np.concatenate(processed_frames, axis=2)
    stacked = np.transpose(stacked, (2, 0, 1))
    tensor = torch.from_numpy(stacked).float().unsqueeze(0)
    return tensor.to(device)


# ----------------- ADAPTIVE PID Controller -----------------
class AdaptivePIDController:
    """PID controller with adaptive gains based on model confidence"""
    def __init__(self):
        # Gentler base gains - trust model more
        self.base_Kp = 0.6
        self.base_Ki = 0.02
        self.base_Kd = 0.15
        
        # Maximum gains (only when model really fails)
        self.max_Kp = 1.3
        self.max_Ki = 0.06
        self.max_Kd = 0.35
        
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        self.integral_limit = 0.6
        self.output_limit = 0.8
        
        # Track recent performance for confidence calculation
        self.recent_errors = []
        self.max_history = 30

    def calculate_model_confidence(self, lane_error, model_steer, pid_estimate):
        """
        Calculate confidence in model output
        Returns: 0.0 (no confidence) to 1.0 (full confidence)
        """
        # Factor 1: Recent lane keeping performance
        self.recent_errors.append(abs(lane_error))
        if len(self.recent_errors) > self.max_history:
            self.recent_errors.pop(0)
        
        if self.recent_errors:
            avg_error = np.mean(self.recent_errors)
            # Good performance (< 0.3m) → high score
            performance_score = max(0.0, 1.0 - avg_error / 0.4)
        else:
            performance_score = 0.5
        
        # Factor 2: Agreement between model and PID
        if abs(model_steer) > 0.01 and abs(pid_estimate) > 0.01:
            # Same direction = good agreement
            if (model_steer * pid_estimate) > 0:
                agreement = 1.0
            else:
                agreement = 0.3  # Disagreement
        else:
            agreement = 0.7  # Neutral when one is near zero
        
        # Factor 3: Sanity check - model shouldn't be extreme when centered
        if abs(lane_error) < 0.15 and abs(model_steer) > 0.6:
            magnitude_penalty = 0.5  # Model being too aggressive
        else:
            magnitude_penalty = 1.0
        
        # Combined confidence (weighted)
        confidence = (performance_score * 0.5 + 
                     agreement * 0.35 + 
                     magnitude_penalty * 0.15)
        
        return np.clip(confidence, 0.0, 1.0)

    def update(self, error, current_time, model_confidence=0.7):
        """
        Update PID with adaptive gains
        High confidence → weak PID
        Low confidence → strong PID
        """
        if self.prev_time is None:
            self.prev_time = current_time
            dt = 0.05
        else:
            dt = max(current_time - self.prev_time, 1e-3)
            self.prev_time = current_time
        
        # Adapt gains based on confidence
        # confidence=1.0 → use base gains
        # confidence=0.0 → use max gains
        confidence_factor = 1.0 - model_confidence
        
        Kp = self.base_Kp + (self.max_Kp - self.base_Kp) * confidence_factor
        Ki = self.base_Ki + (self.max_Ki - self.base_Ki) * confidence_factor
        Kd = self.base_Kd + (self.max_Kd - self.base_Kd) * confidence_factor
        
        # PID calculation
        P = Kp * error
        
        self.integral += error * dt
        self.integral = np.clip(self.integral, -self.integral_limit, self.integral_limit)
        I = Ki * self.integral
        
        derivative = (error - self.prev_error) / dt if dt > 0 else 0.0
        D = Kd * derivative
        
        self.prev_error = error
        output = P + I + D
        
        return float(np.clip(output, -self.output_limit, self.output_limit)), Kp

    def reset(self):
        self.integral = 0.0
        self.prev_error = 0.0
        self.prev_time = None
        self.recent_errors = []


# ----------------- Waypoint Path Follower -----------------
class WaypointFollower:
    def __init__(self):
        self.path = []
        self.current_idx = 0
        self.lookahead_distance = 8.0
        
    def set_path(self, waypoints):
        self.path = waypoints
        self.current_idx = 0
        
    def clear_path(self):
        self.path = []
        self.current_idx = 0
        
    def get_target_point(self, vehicle_location):
        if not self.path:
            return None
            
        min_dist = float('inf')
        closest_idx = self.current_idx
        
        for i in range(self.current_idx, min(len(self.path), self.current_idx + 10)):
            wp_loc = self.path[i].transform.location
            dist = math.sqrt(
                (wp_loc.x - vehicle_location.x)**2 + 
                (wp_loc.y - vehicle_location.y)**2
            )
            if dist < min_dist:
                min_dist = dist
                closest_idx = i
        
        self.current_idx = closest_idx
        
        target_idx = closest_idx
        cumulative_dist = 0.0
        
        for i in range(closest_idx, len(self.path) - 1):
            wp1 = self.path[i].transform.location
            wp2 = self.path[i + 1].transform.location
            segment_dist = math.sqrt(
                (wp2.x - wp1.x)**2 + (wp2.y - wp1.y)**2
            )
            cumulative_dist += segment_dist
            
            if cumulative_dist >= self.lookahead_distance:
                target_idx = i + 1
                break
        else:
            target_idx = len(self.path) - 1
            
        if target_idx >= len(self.path):
            return None
            
        return self.path[target_idx].transform.location
    
    def is_path_complete(self, vehicle_location):
        if not self.path:
            return True
        
        end_loc = self.path[-1].transform.location
        dist = math.sqrt(
            (end_loc.x - vehicle_location.x)**2 + 
            (end_loc.y - vehicle_location.y)**2
        )
        return dist < 3.0


# ----------------- Main Controller -----------------
class ModelEfficientLaneChangeController:
    def __init__(self, model_path, host="localhost", port=2000):
        self.model_path = model_path
        self.host = host
        self.port = port
        self.running = True

        # Controllers - NOW ADAPTIVE!
        self.lane_pid = AdaptivePIDController()
        self.path_follower = WaypointFollower()
        
        # Smoothing
        self.steer_smooth = 0.0
        self.throttle_smooth = 0.3
        self.steer_alpha = 0.25
        self.throttle_alpha = 0.20

        # Speed control
        self.target_speed_kmh = 20.0
        self.stop_distance_from_line = 3.5

        # State machine
        self.state = "DRIVING"
        self.stop_start_time = None
        self.green_wait_time = 0.0
        self.lane_change_decision = None
        self.lane_change_path_created = False

        # NEW: Model usage tracking
        self.model_confidence_history = []
        self.model_usage_stats = {
            'total_frames': 0,
            'model_dominant': 0,  # Model weight > 60%
            'balanced': 0,        # 40-60%
            'pid_dominant': 0,    # PID weight > 60%
            'model_throttle_used': 0,
            'classical_throttle_used': 0
        }

        # Stats
        self.collision_count = 0
        self.deviations = []
        self.controls_history = []
        self.trajectory = []
        self.lane_changes_completed = 0

        # CARLA objects
        self.client = None
        self.world = None
        self.map = None
        self.vehicle = None
        self.camera = None
        self.collision_sensor = None
        self.latest_image = None
        self.new_image_available = False
        self.collision_occurred = False
        self.start_time = None
        self.original_settings = None

        signal.signal(signal.SIGINT, self.signal_handler)

    def signal_handler(self, sig, frame):
        print("\nShutdown requested.")
        self.running = False

    def connect_carla(self):
        self.client = carla.Client(self.host, self.port)
        self.client.set_timeout(10.0)
        self.world = self.client.get_world()
        self.original_settings = self.world.get_settings()
        self.map = self.world.get_map()

        settings = self.world.get_settings()
        settings.synchronous_mode = True
        settings.fixed_delta_seconds = 0.05
        self.world.apply_settings(settings)

    def spawn_vehicle(self):
        blueprint_library = self.world.get_blueprint_library()
        vehicle_bp = blueprint_library.filter("vehicle.mercedes.coupe_2020")[0]
        if not vehicle_bp:
            vehicle_bp = blueprint_library.filter("vehicle.*")[0]
        spawn_points = self.map.get_spawn_points()
        sp = random.choice(spawn_points)
        sp.location.z += 0.5
        self.vehicle = self.world.spawn_actor(vehicle_bp, sp)
        print(f"Spawned at ({sp.location.x:.2f}, {sp.location.y:.2f})")
        return sp.location

    def setup_sensors(self):
        bp_lib = self.world.get_blueprint_library()
        
        cam_bp = bp_lib.find("sensor.camera.rgb")
        cam_bp.set_attribute("image_size_x", "800")
        cam_bp.set_attribute("image_size_y", "600")
        cam_bp.set_attribute("fov", "120")
        cam_transform = carla.Transform(carla.Location(x=-4,y=0, z=2), carla.Rotation(pitch=-5))
        self.camera = self.world.spawn_actor(cam_bp, cam_transform, attach_to=self.vehicle)
        self.camera.listen(self.camera_callback)

        col_bp = bp_lib.find("sensor.other.collision")
        self.collision_sensor = self.world.spawn_actor(col_bp, carla.Transform(), attach_to=self.vehicle)
        self.collision_sensor.listen(self.collision_callback)

    def camera_callback(self, image):
        try:
            arr = np.frombuffer(image.raw_data, dtype=np.uint8)
            arr = arr.reshape((image.height, image.width, 4))[:, :, :3]
            arr = arr[:, :, ::-1]
            small = cv2.resize(arr, (800, 600))
            self.latest_image = small
            self.new_image_available = True
        except Exception:
            pass

    def collision_callback(self, event):
        self.collision_count += 1
        print(f"\n*** COLLISION #{self.collision_count} ***")
        self.collision_occurred = True
        
        self.path_follower.clear_path()
        self.lane_pid.reset()
        self.state = "COLLISION_RECOVERY"
        
        try:
            ctrl = carla.VehicleControl()
            ctrl.steer = 0.0
            ctrl.throttle = 0.0
            ctrl.brake = 1.0
            self.vehicle.apply_control(ctrl)
        except Exception:
            pass
        
        time.sleep(1.5)
        self.collision_occurred = False
        self.state = "DRIVING"

    def load_model(self):
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print("Device:", self.device)
        self.model = DrivingNet(in_ch=3, n_stack=3, use_speed=True)
        self.model.initialize((9, 120, 160))
        self.model = self.model.to(self.device)
        
        loaded = torch.load(self.model_path, map_location=self.device, weights_only=False)
        state = loaded['state_dict'] if isinstance(loaded, dict) and 'state_dict' in loaded else loaded
        
        try:
            self.model.load_state_dict(state)
        except RuntimeError:
            new_state = {k[7:] if k.startswith("module.") else k: v for k, v in state.items()}
            self.model.load_state_dict(new_state)
        
        print("✓ Model loaded (Model-Efficient Mode)")
        self.model.eval()

    def calculate_lane_error(self, vehicle_location, vehicle_rotation):
        try:
            waypoint = self.map.get_waypoint(vehicle_location, project_to_road=True, lane_type=carla.LaneType.Driving)
            if waypoint is None:
                return 0.0
            
            lane_center = waypoint.transform.location
            dx = vehicle_location.x - lane_center.x
            dy = vehicle_location.y - lane_center.y
            lane_yaw = math.radians(waypoint.transform.rotation.yaw)
            cross_track = dx * math.sin(lane_yaw) - dy * math.cos(lane_yaw)
            
            vehicle_yaw = math.radians(vehicle_rotation.yaw)
            heading_error = vehicle_yaw - lane_yaw
            while heading_error > math.pi:
                heading_error -= 2 * math.pi
            while heading_error < -math.pi:
                heading_error += 2 * math.pi
            
            total_error = cross_track + 1.0 * heading_error  # Reduced from 1.2
            return total_error
        except Exception:
            return 0.0

    def find_traffic_light_ahead(self):
        if self.vehicle is None:
            return None, None
        
        try:
            veh_tf = self.vehicle.get_transform()
            veh_loc = veh_tf.location
            veh_fwd = veh_tf.get_forward_vector()
            
            lights = self.world.get_actors().filter("traffic.traffic_light*")
            best_light = None
            best_dist = None
            
            for tl in lights:
                tl_loc = tl.get_transform().location
                vec = np.array([tl_loc.x - veh_loc.x, tl_loc.y - veh_loc.y, tl_loc.z - veh_loc.z])
                dist = np.linalg.norm(vec)
                
                if dist > 40.0 or dist < 0.5:
                    continue
                
                vec_norm = vec / (dist + 1e-6)
                dot = vec_norm[0] * veh_fwd.x + vec_norm[1] * veh_fwd.y + vec_norm[2] * veh_fwd.z
                
                if dot > 0.7:
                    if best_light is None or dist < best_dist:
                        best_light = tl
                        best_dist = dist
            
            return best_light, best_dist
        except Exception:
            return None, None

    def check_lane_availability(self, current_wp, direction):
        try:
            if direction == "left":
                target_wp = current_wp.get_left_lane()
            else:
                target_wp = current_wp.get_right_lane()
            
            if target_wp is None:
                return False
            
            if target_wp.lane_type != carla.LaneType.Driving:
                return False
            
            if (current_wp.lane_id > 0) != (target_wp.lane_id > 0):
                return False
            
            test_wp = target_wp
            valid_distance = 0.0
            for _ in range(30):
                nxts = test_wp.next(2.0)
                if not nxts:
                    break
                test_wp = nxts[0]
                if test_wp.lane_type != carla.LaneType.Driving:
                    break
                valid_distance += 2.0
            
            if valid_distance < 40.0:
                return False
            
            veh_loc = self.vehicle.get_location()
            for other in self.world.get_actors().filter("vehicle.*"):
                if other.id == self.vehicle.id:
                    continue
                
                other_loc = other.get_location()
                dist = math.sqrt(
                    (other_loc.x - veh_loc.x)**2 + 
                    (other_loc.y - veh_loc.y)**2
                )
                
                if dist < 25.0:
                    try:
                        other_wp = self.map.get_waypoint(other_loc, project_to_road=True)
                        if other_wp and other_wp.lane_id == target_wp.lane_id:
                            return False
                    except Exception:
                        pass
            
            return True
            
        except Exception as e:
            print(f"Lane check error: {e}")
            return False

    def create_lane_change_path(self, direction):
        try:
            loc = self.vehicle.get_location()
            current_wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
            
            if current_wp is None:
                return []
            
            if direction == "left":
                target_wp = current_wp.get_left_lane()
            else:
                target_wp = current_wp.get_right_lane()
            
            if target_wp is None:
                return []
            
            path = []
            
            wp = current_wp
            for _ in range(5):
                nxts = wp.next(2.0)
                if nxts:
                    wp = nxts[0]
                    path.append(wp)
            
            start_wp = wp
            
            end_wp = target_wp
            for _ in range(15):
                nxts = end_wp.next(2.0)
                if nxts:
                    end_wp = nxts[0]
            
            for i in range(1, 11):
                t = i / 10.0
                interp_loc = carla.Location(
                    x=start_wp.transform.location.x * (1 - t) + end_wp.transform.location.x * t,
                    y=start_wp.transform.location.y * (1 - t) + end_wp.transform.location.y * t,
                    z=start_wp.transform.location.z * (1 - t) + end_wp.transform.location.z * t
                )
                interp_wp = self.map.get_waypoint(interp_loc, project_to_road=True)
                if interp_wp:
                    path.append(interp_wp)
            
            wp = end_wp
            for _ in range(10):
                nxts = wp.next(2.0)
                if nxts:
                    wp = nxts[0]
                    path.append(wp)
            
            return path
            
        except Exception as e:
            print(f"Path creation error: {e}")
            return []

    def calculate_steering_to_point(self, vehicle_loc, vehicle_rot, target_point):
        dx = target_point.x - vehicle_loc.x
        dy = target_point.y - vehicle_loc.y
        
        yaw = math.radians(vehicle_rot.yaw)
        local_x = dx * math.cos(yaw) + dy * math.sin(yaw)
        local_y = -dx * math.sin(yaw) + dy * math.cos(yaw)
        
        if abs(local_x) < 0.1:
            return 0.0
        
        steering_angle = math.atan2(local_y, local_x)
        steer_cmd = steering_angle / math.radians(30)
        return float(np.clip(steer_cmd, -1.0, 1.0))

    def process_model_throttle_brake(self, model_throttle, model_brake, speed_mps, 
                                     lane_error, confidence, state):
        """
        NEW: Actually use model throttle/brake with safety checks
        """
        # State-specific policies
        use_model = {
            'DRIVING': confidence > 0.5,
            'APPROACHING_STOP': False,  # Safety critical
            'EXECUTING_LANE_CHANGE': confidence > 0.6,
            'STABILIZING': confidence > 0.55
        }.get(state, False)
        
        if use_model:
            # Trust the model!
            target_speed_mps = self.target_speed_kmh / 3.6
            
            # Speed governor
            if speed_mps < target_speed_mps - 1.0:
                speed_factor = 1.0
            elif speed_mps > target_speed_mps + 1.0:
                speed_factor = 0.5
            else:
                speed_factor = 0.8
            
            # Lane error penalty
            lane_factor = max(0.75, 1.0 - abs(lane_error) / 2.0)
            
            # Apply model throttle with factors
            final_throttle = model_throttle * confidence * speed_factor * lane_factor
            final_throttle = np.clip(final_throttle, 0.15, 0.75)
            
            # Use model brake if confident enough
            if model_brake > 0.35 and confidence > 0.7:
                final_brake = model_brake
                final_throttle = 0.0
            else:
                final_brake = 0.0
            
            self.model_usage_stats['model_throttle_used'] += 1
            
        else:
            # Fallback to classical control
            target = 20.0 if state == 'DRIVING' else 15.0
            target_mps = target / 3.6
            
            if speed_mps < target_mps:
                final_throttle = min(0.6, 0.3 + (target_mps - speed_mps) * 0.25)
            else:
                final_throttle = 0.2
            
            final_brake = 0.0
            self.model_usage_stats['classical_throttle_used'] += 1
        
        return final_throttle, final_brake

    def calculate_model_weight(self, state, lane_error, confidence):
        """
        NEW: State-aware dynamic model weighting
        Returns higher model weights!
        """
        # Base weights per state (trust model MORE)
        base_weights = {
            'DRIVING': 0.70,              # 70% model normally
            'APPROACHING_STOP': 0.40,     # 40% during stopping
            'EXECUTING_LANE_CHANGE': 0.55, # 55% during LC
            'STABILIZING': 0.65            # 65% while stabilizing
        }
        
        base = base_weights.get(state, 0.60)
        
        # Adjust for confidence (±15%)
        conf_adjustment = (confidence - 0.5) * 0.30
        
        # Adjust for error (but less aggressively - only ±10%)
        error_adjustment = -min(abs(lane_error) / 1.0, 0.5) * 0.20
        
        # Calculate final weight
        model_weight = base + conf_adjustment + error_adjustment
        
        # Bounds: never below 25%, never above 85%
        model_weight = np.clip(model_weight, 0.25, 0.85)
        
        return model_weight

    def run_evaluation(self):
        print("\n" + "="*70)
        print("MODEL-EFFICIENT LANE CHANGE CONTROLLER")
        print("="*70)
        print("🎯 Goal: Trust the model 50-70% of the time")
        print("📊 Adaptive PID: Kp 0.6-1.3 (was fixed 1.8)")
        print("🚀 Model throttle/brake: ENABLED")
        print("="*70 + "\n")
        
        try:
            self.connect_carla()
            self.spawn_vehicle()
            self.setup_sensors()
            self.load_model()
        except Exception as e:
            print("Init failed:", e)
            traceback.print_exc()
            return

        spectator = self.world.get_spectator()
        ctrl = carla.VehicleControl()

        print("Warming up sensors...")
        for _ in range(10):
            self.world.tick()
            time.sleep(0.05)

        frame_stack = []
        frame_count = 0
        self.start_time = time.time()
        last_log = time.time()

        log_filename = f"model_efficient_log_{int(self.start_time)}.csv"
        log_file = open(log_filename, "w")
        log_file.write("timestamp,frame,x,y,z,yaw,speed_kmh,final_steer,throttle,lane_error,"
                      "state,model_weight,confidence,pid_kp\n")

        image_dir = f"model_efficient_images_{int(self.start_time)}"
        os.makedirs(image_dir, exist_ok=True)

        try:
            while self.running:
                self.world.tick()

                try:
                    tf = self.vehicle.get_transform()
                    spec_loc = tf.location + tf.get_forward_vector() * -8 + carla.Location(z=4.0)
                    spec_rot = carla.Rotation(pitch=-15, yaw=tf.rotation.yaw)
                    spectator.set_transform(carla.Transform(spec_loc, spec_rot))
                except Exception:
                    pass

                if self.new_image_available and self.latest_image is not None:
                    frame_stack.append(self.latest_image.copy())
                    if len(frame_stack) > 3:
                        frame_stack.pop(0)
                    self.new_image_available = False

                tl_actor, tl_dist = self.find_traffic_light_ahead()
                tl_state = None
                if tl_actor is not None:
                    try:
                        tl_state = tl_actor.get_state()
                    except Exception:
                        pass

                # Main control loop
                if len(frame_stack) == 3:
                    tf = self.vehicle.get_transform()
                    loc = tf.location
                    rot = tf.rotation
                    vel = self.vehicle.get_velocity()
                    speed_mps = math.sqrt(vel.x**2 + vel.y**2 + vel.z**2)
                    speed_kmh = speed_mps * 3.6

                    lane_error = self.calculate_lane_error(loc, rot)

                    input_tensor = preprocess_image_stack(frame_stack, self.device)
                    if input_tensor is None:
                        continue

                    speed_norm = min(speed_mps / 30.0, 1.0)
                    speed_tensor = torch.tensor([[speed_norm]], dtype=torch.float32, device=self.device)

                    with torch.no_grad():
                        output = self.model(input_tensor, aux=speed_tensor)
                    model_steer, model_throttle, model_brake = output[0].cpu().numpy()

                    final_steer = 0.0
                    final_throttle = 0.0
                    final_brake = 0.0
                    
                    # NEW: Variables for logging
                    model_weight = 0.0
                    confidence = 0.0
                    pid_kp = 0.0

                    # ============ STATE MACHINE (MODEL-EFFICIENT) ============
                    
                    if self.state == "DRIVING":
                        if (tl_actor is not None and tl_state == carla.TrafficLightState.Red and 
                            tl_dist is not None and tl_dist <= 25.0):
                            print(f"🔴 Red light @ {tl_dist:.1f}m -> STOPPING")
                            self.state = "APPROACHING_STOP"
                            self.stop_start_time = time.time()
                        else:
                            # NEW: Model-efficient lane keeping
                            now = time.time()
                            
                            # Get preliminary PID estimate for confidence calculation
                            prelim_pid = self.lane_pid.base_Kp * lane_error
                            
                            # Calculate model confidence
                            confidence = self.lane_pid.calculate_model_confidence(
                                lane_error, model_steer, prelim_pid
                            )
                            self.model_confidence_history.append(confidence)
                            
                            # Get adaptive PID correction
                            pid_correction, pid_kp = self.lane_pid.update(lane_error, now, confidence)
                            
                            # Calculate dynamic model weight
                            model_weight = self.calculate_model_weight('DRIVING', lane_error, confidence)
                            pid_weight = 1.0 - model_weight
                            
                            # Blend steering
                            combined_steer = model_weight * model_steer + pid_weight * pid_correction
                            
                            # Smooth steering
                            final_steer = self.steer_alpha * combined_steer + (1 - self.steer_alpha) * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.7, 0.7))
                            
                            # NEW: Use model throttle/brake with safety
                            final_throttle, final_brake = self.process_model_throttle_brake(
                                model_throttle, model_brake, speed_mps, lane_error, 
                                confidence, 'DRIVING'
                            )
                            
                            # Track statistics
                            self.model_usage_stats['total_frames'] += 1
                            if model_weight > 0.6:
                                self.model_usage_stats['model_dominant'] += 1
                            elif pid_weight > 0.6:
                                self.model_usage_stats['pid_dominant'] += 1
                            else:
                                self.model_usage_stats['balanced'] += 1
                    
                    elif self.state == "APPROACHING_STOP":
                        if tl_dist is not None:
                            stop_dist = tl_dist - self.stop_distance_from_line
                            
                            if stop_dist <= 0.5 or speed_mps < 0.3:
                                print("🛑 Stopped at line, waiting for green...")
                                self.state = "WAITING_GREEN"
                                self.green_wait_time = 0.0
                                self.lane_pid.reset()
                                final_steer = 0.0
                                final_throttle = 0.0
                                final_brake = 1.0
                            else:
                                brake_strength = min(1.0, 0.4 + (speed_mps / 15.0) + (1.0 / max(stop_dist, 1.0)))
                                final_brake = float(np.clip(brake_strength, 0.3, 1.0))
                                final_throttle = 0.0
                                
                                now = time.time()
                                pid_correction, pid_kp = self.lane_pid.update(lane_error, now, 0.3)
                                final_steer = 0.5 * pid_correction
                                final_steer = float(np.clip(final_steer, -0.3, 0.3))
                        else:
                            final_brake = 0.8
                            final_throttle = 0.0
                            final_steer = 0.0
                    
                    elif self.state == "WAITING_GREEN":
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                        
                        if tl_actor is not None and tl_state == carla.TrafficLightState.Green:
                            self.green_wait_time += 0.05
                            
                            if self.green_wait_time > 0.5:
                                print("🟢 Green light -> Deciding lane change...")
                                
                                loc = self.vehicle.get_location()
                                current_wp = self.map.get_waypoint(loc, project_to_road=True, lane_type=carla.LaneType.Driving)
                                
                                if current_wp:
                                    directions = ["left", "right"] if random.random() < 0.5 else ["right", "left"]
                                    
                                    for direction in directions:
                                        if self.check_lane_availability(current_wp, direction):
                                            print(f"  ✓ {direction.upper()} lane is available")
                                            self.lane_change_decision = direction
                                            self.state = "INITIATING_LANE_CHANGE"
                                            break
                                    else:
                                        print("  ✗ No safe lane, going straight")
                                        self.state = "DRIVING"
                                        self.lane_pid.reset()
                                else:
                                    print("  ✗ Cannot get waypoint, going straight")
                                    self.state = "DRIVING"
                                    self.lane_pid.reset()
                    
                    elif self.state == "INITIATING_LANE_CHANGE":
                        if not self.lane_change_path_created:
                            path = self.create_lane_change_path(self.lane_change_decision)
                            
                            if path:
                                self.path_follower.set_path(path)
                                self.lane_change_path_created = True
                                print(f"  🔄 Lane change path created ({len(path)} waypoints)")
                                self.state = "EXECUTING_LANE_CHANGE"
                            else:
                                print("  ✗ Failed to create path, aborting")
                                self.state = "DRIVING"
                                self.lane_pid.reset()
                        
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0
                    
                    elif self.state == "EXECUTING_LANE_CHANGE":
                        target_point = self.path_follower.get_target_point(loc)
                        
                        if target_point is not None:
                            final_steer = self.calculate_steering_to_point(loc, rot, target_point)
                            
                            final_steer = self.steer_alpha * final_steer + (1 - self.steer_alpha) * self.steer_smooth
                            final_steer = float(np.clip(final_steer, -0.6, 0.6))
                            
                            target_lc_speed = 15.0
                            lc_speed_mps = target_lc_speed / 3.6
                            speed_error = lc_speed_mps - speed_mps
                            
                            if speed_error > 0:
                                final_throttle = min(0.5, 0.3 + speed_error * 0.2)
                            else:
                                final_throttle = 0.2
                            
                            final_brake = 0.0
                            
                            if self.path_follower.is_path_complete(loc):
                                print(f"  ✅ Lane change to {self.lane_change_decision.upper()} COMPLETE!")
                                self.lane_changes_completed += 1
                                self.path_follower.clear_path()
                                self.lane_change_path_created = False
                                self.lane_change_decision = None
                                self.state = "STABILIZING"
                                self.stop_start_time = time.time()
                        else:
                            print("  ⚠️ Lost path target, aborting")
                            self.path_follower.clear_path()
                            self.lane_change_path_created = False
                            self.lane_change_decision = None
                            self.state = "DRIVING"
                            self.lane_pid.reset()
                            final_steer = 0.0
                            final_throttle = 0.0
                            final_brake = 0.3
                    
                    elif self.state == "STABILIZING":
                        elapsed = time.time() - self.stop_start_time
                        
                        if elapsed < 2.0:
                            # NEW: Use model with PID blend during stabilizing
                            now = time.time()
                            prelim_pid = self.lane_pid.base_Kp * lane_error
                            confidence = self.lane_pid.calculate_model_confidence(
                                lane_error, model_steer, prelim_pid
                            )
                            
                            pid_correction, pid_kp = self.lane_pid.update(lane_error, now, confidence)
                            model_weight = self.calculate_model_weight('STABILIZING', lane_error, confidence)
                            
                            # Blend model and PID for stabilizing
                            combined_steer = model_weight * model_steer + (1 - model_weight) * pid_correction
                            final_steer = float(np.clip(combined_steer, -0.5, 0.5))
                            
                            final_throttle = min(0.4, 0.15 + elapsed * 0.1)
                            final_brake = 0.0
                        else:
                            print("  📍 Stabilized in new lane, resuming normal driving")
                            self.state = "DRIVING"
                            self.lane_pid.reset()
                    
                    elif self.state == "COLLISION_RECOVERY":
                        final_steer = 0.0
                        final_throttle = 0.0
                        final_brake = 1.0

                    # ============ APPLY CONTROL ============
                    
                    self.steer_smooth = final_steer
                    
                    final_throttle = (self.throttle_alpha * final_throttle + 
                                     (1 - self.throttle_alpha) * self.throttle_smooth)
                    self.throttle_smooth = final_throttle
                    
                    ctrl.steer = float(final_steer)
                    ctrl.throttle = float(final_throttle)
                    ctrl.brake = float(final_brake)
                    ctrl.hand_brake = False
                    
                    try:
                        self.vehicle.apply_control(ctrl)
                    except Exception:
                        pass

                    # ============ LOGGING & RECORDING ============
                    
                    try:
                        frame_filename = os.path.join(image_dir, f"{frame_count:06d}.png")
                        cv2.imwrite(frame_filename, cv2.cvtColor(self.latest_image, cv2.COLOR_RGB2BGR))
                    except Exception as e:
                        print(f"Warning: failed to save frame {frame_count}: {e}")

                    self.deviations.append(abs(lane_error))
                    self.controls_history.append({
                        'frame': frame_count,
                        'final_steer': float(final_steer),
                        'throttle': float(final_throttle),
                        'speed': float(speed_mps),
                        'lane_error': float(lane_error),
                        'state': self.state,
                        'model_weight': float(model_weight),
                        'confidence': float(confidence)
                    })
                    self.trajectory.append({
                        'x': loc.x,
                        'y': loc.y,
                        'z': loc.z,
                        'yaw': rot.yaw
                    })

                    tstamp = time.time() - self.start_time
                    log_file.write(
                        f"{tstamp:.3f},{frame_count},{loc.x:.2f},{loc.y:.2f},{loc.z:.2f},"
                        f"{rot.yaw:.2f},{speed_kmh:.1f},{final_steer:.4f},{final_throttle:.4f},"
                        f"{lane_error:.4f},{self.state},{model_weight:.3f},{confidence:.3f},{pid_kp:.3f}\n"
                    )
                    log_file.flush()

                    if time.time() - last_log >= 1.0:
                        tl_msg = "None"
                        if tl_actor is not None and tl_dist is not None:
                            try:
                                st = tl_actor.get_state()
                                tl_msg = f"{st.name[:3]} @ {tl_dist:.1f}m"
                            except Exception:
                                tl_msg = f"@ {tl_dist:.1f}m"
                        
                        lc_info = f"[LC: {self.lane_changes_completed}]" if self.lane_changes_completed > 0 else ""
                        
                        # NEW: Enhanced logging with model info
                        if self.state == "DRIVING":
                            print(
                                f"[{tstamp:6.1f}s] F{frame_count:4d} | "
                                f"Speed:{speed_kmh:5.1f} | "
                                f"Lane:{lane_error:+.3f}m | "
                                f"Steer:{final_steer:+.3f} | "
                                f"Model:{model_weight:.0%} | "
                                f"Conf:{confidence:.2f} | "
                                f"Kp:{pid_kp:.2f} | "
                                f"{self.state:22s} {lc_info}"
                            )
                        else:
                            print(
                                f"[{tstamp:6.1f}s] F{frame_count:4d} | "
                                f"Speed:{speed_kmh:5.1f} | "
                                f"Lane:{lane_error:+.3f}m | "
                                f"Steer:{final_steer:+.3f} | "
                                f"TL:{tl_msg:15s} | "
                                f"{self.state:22s} {lc_info}"
                            )
                        last_log = time.time()

                    frame_count += 1

        except Exception as e:
            print("Evaluation error:", e)
            traceback.print_exc()
        finally:
            try:
                log_file.close()
            except Exception:
                pass
            
            try:
                imgs = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".png")])
                if imgs:
                    out_vid = f"model_efficient_video_{int(self.start_time)}.mp4"
                    print(f"\nCreating video from {len(imgs)} frames...")
                    with imageio.get_writer(out_vid, fps=30) as writer:
                        for im in imgs:
                            writer.append_data(imageio.imread(im))
                    print("✓ Video saved:", out_vid)
            except Exception as e:
                print("Video creation warning:", e)

            self.print_statistics(frame_count)
            self.cleanup()

    def print_statistics(self, frame_count):
        elapsed = time.time() - (self.start_time or time.time())
        print("\n" + "="*70)
        print("MODEL-EFFICIENT EVALUATION STATISTICS")
        print("="*70)
        print(f"Duration: {elapsed:.2f} s")
        print(f"Frames: {frame_count}")
        if elapsed > 0:
            print(f"FPS: {frame_count / elapsed:.2f}")
        print(f"Collisions: {self.collision_count}")
        print(f"\nLane Changes:")
        print(f"  Completed: {self.lane_changes_completed}")
        
        # NEW: Model usage statistics
        stats = self.model_usage_stats
        total = stats['total_frames']
        if total > 0:
            print(f"\n🎯 MODEL vs PID CONTROL DISTRIBUTION:")
            model_dom_pct = stats['model_dominant'] / total * 100
            balanced_pct = stats['balanced'] / total * 100
            pid_dom_pct = stats['pid_dominant'] / total * 100
            
            print(f"  Model-dominant (>60%):  {stats['model_dominant']:5d} frames ({model_dom_pct:5.1f}%)")
            print(f"  Balanced (40-60%):      {stats['balanced']:5d} frames ({balanced_pct:5.1f}%)")
            print(f"  PID-dominant (>60%):    {stats['pid_dominant']:5d} frames ({pid_dom_pct:5.1f}%)")
            
            print(f"\n🚀 THROTTLE/BRAKE CONTROL:")
            model_throttle = stats.get('model_throttle_used', 0)
            classical_throttle = stats.get('classical_throttle_used', 0)
            throttle_total = model_throttle + classical_throttle
            if throttle_total > 0:
                model_throttle_pct = model_throttle / throttle_total * 100
                print(f"  Model throttle:     {model_throttle:5d} frames ({model_throttle_pct:5.1f}%)")
                print(f"  Classical throttle: {classical_throttle:5d} frames ({100-model_throttle_pct:5.1f}%)")
        
        if self.model_confidence_history:
            avg_conf = np.mean(self.model_confidence_history)
            min_conf = np.min(self.model_confidence_history)
            max_conf = np.max(self.model_confidence_history)
            
            print(f"\n📊 MODEL CONFIDENCE:")
            print(f"  Average: {avg_conf:.3f}")
            print(f"  Range: {min_conf:.3f} to {max_conf:.3f}")
            
            high_conf = sum(1 for c in self.model_confidence_history if c > 0.7)
            low_conf = sum(1 for c in self.model_confidence_history if c < 0.4)
            
            print(f"  High confidence (>0.7): {high_conf/len(self.model_confidence_history)*100:.1f}%")
            print(f"  Low confidence (<0.4):  {low_conf/len(self.model_confidence_history)*100:.1f}%")
        
        if self.deviations:
            print(f"\n🛣️ LANE KEEPING:")
            print(f"  Mean error: {np.mean(self.deviations):.3f} m")
            print(f"  Max error: {np.max(self.deviations):.3f} m")
            print(f"  Std dev: {np.std(self.deviations):.3f} m")
            within_half = sum(1 for d in self.deviations if d < 0.5) / len(self.deviations) * 100
            within_quarter = sum(1 for d in self.deviations if d < 0.25) / len(self.deviations) * 100
            print(f"  Within 0.5m: {within_half:.1f}%")
            print(f"  Within 0.25m: {within_quarter:.1f}%")
        
        if self.trajectory:
            distances = []
            for i in range(1, len(self.trajectory)):
                dx = self.trajectory[i]['x'] - self.trajectory[i-1]['x']
                dy = self.trajectory[i]['y'] - self.trajectory[i-1]['y']
                distances.append(math.sqrt(dx**2 + dy**2))
            total_dist = sum(distances)
            print(f"\n📏 DISTANCE & SPEED:")
            print(f"  Distance traveled: {total_dist:.2f} m")
            if elapsed > 0:
                avg_speed = (total_dist / elapsed) * 3.6
                print(f"  Average speed: {avg_speed:.2f} km/h")
        
        print("="*70)
        print("\n✅ SUMMARY:")
        if total > 0 and model_dom_pct > 45:
            print("  SUCCESS: Model is now the primary controller!")
        elif total > 0 and model_dom_pct > 30:
            print("  GOOD: Model contributes significantly to control")
        else:
            print("  IMPROVEMENT NEEDED: PID still dominant")
        print("="*70)

    def cleanup(self):
        print("\nCleaning up...")
        try:
            if self.world is not None and self.original_settings is not None:
                self.world.apply_settings(self.original_settings)
        except Exception:
            pass
        
        actors = []
        for a in (self.camera, self.collision_sensor, self.vehicle):
            if a is not None:
                try:
                    a.stop()
                except Exception:
                    pass
                actors.append(a)
        
        for a in actors:
            try:
                a.destroy()
            except Exception:
                pass
        
        gc.collect()
        print("✓ Cleanup complete")


def main():
    MODEL_PATH = "/home/robotuser/carla_0.9.12/PythonAPI/examples/checkpoints/new_best_model.pth"
    
    controller = ModelEfficientLaneChangeController(
        model_path=MODEL_PATH,
        host="localhost",
        port=2000
    )
    
    controller.run_evaluation()


if __name__ == "__main__":
    main()


MODEL-EFFICIENT LANE CHANGE CONTROLLER
🎯 Goal: Trust the model 50-70% of the time
📊 Adaptive PID: Kp 0.6-1.3 (was fixed 1.8)
🚀 Model throttle/brake: ENABLED

Spawned at (-20.12, 16.75)
Device: cuda
✓ Model loaded (Model-Efficient Mode)
Warming up sensors...
🔴 Red light @ 12.3m -> STOPPING
🛑 Stopped at line, waiting for green...
[   1.0s] F  32 | Speed:  0.0 | Lane:+0.121m | Steer:+0.000 | TL:Red @ 12.3m     | WAITING_GREEN          
[   2.0s] F  66 | Speed:  0.0 | Lane:+0.121m | Steer:+0.000 | TL:Gre @ 12.3m     | WAITING_GREEN          
🟢 Green light -> Deciding lane change...
  ✓ RIGHT lane is available
  🔄 Lane change path created (25 waypoints)
[   3.0s] F 101 | Speed:  2.6 | Lane:+0.121m | Steer:+0.023 | TL:Gre @ 12.3m     | EXECUTING_LANE_CHANGE  
[   4.0s] F 138 | Speed:  9.9 | Lane:+0.055m | Steer:+0.526 | TL:Gre @ 8.9m      | EXECUTING_LANE_CHANGE  
[   5.1s] F 175 | Speed: 12.0 | Lane:+1.560m | Steer:+0.032 | TL:Red @ 33.6m     | EXECUTING_LANE_CHANGE  
[   6.1s] F 215 | Spe

IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (800, 600) to (800, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).



Shutdown requested.

Creating video from 2506 frames...
✓ Video saved: model_efficient_video_1768080845.mp4

MODEL-EFFICIENT EVALUATION STATISTICS
Duration: 83.50 s
Frames: 2506
FPS: 30.01
Collisions: 0

Lane Changes:
  Completed: 1

🎯 MODEL vs PID CONTROL DISTRIBUTION:
  Model-dominant (>60%):   2045 frames (100.0%)
  Balanced (40-60%):          0 frames (  0.0%)
  PID-dominant (>60%):        0 frames (  0.0%)

🚀 THROTTLE/BRAKE CONTROL:
  Model throttle:      2045 frames (100.0%)
  Classical throttle:     0 frames (  0.0%)

📊 MODEL CONFIDENCE:
  Average: 0.794
  Range: 0.794 to 0.794
  High confidence (>0.7): 100.0%
  Low confidence (<0.4):  0.0%

🛣️ LANE KEEPING:
  Mean error: 0.257 m
  Max error: 2.280 m
  Std dev: 0.364 m
  Within 0.5m: 92.2%
  Within 0.25m: 90.9%

📏 DISTANCE & SPEED:
  Distance traveled: 48.83 m
  Average speed: 2.11 km/h

✅ SUMMARY:
  SUCCESS: Model is now the primary controller!

Cleaning up...
✓ Cleanup complete
